In [26]:
import re


def find_page_by_paragraph(content_list, search_paragraph, threshold=0.8):
    """
    Find the page number(s) that contain a given paragraph.
    
    Args:
        content_list (list): List of dictionaries with 'page' and 'content' keys
        search_paragraph (str): The paragraph text to search for
        threshold (float): Similarity threshold (0.0 to 1.0) for fuzzy matching
                          1.0 means exact match, lower values allow for partial matches
    
    Returns:
        list: List of page numbers where the paragraph was found
    """
    search_paragraph = "\n".join([line.strip() for line in search_paragraph.lower().split("\n")])
    result_pages = []
    # Check for exact matches first
    for item in content_list:
        page = item['page']
        content = item['content'].strip().lower().replace("\n", " ")
        
        if search_paragraph.replace("\n", " ") in content:
            result_pages.append(page)

    # If exact match found, return results
    if result_pages:
        return result_pages
    
    # If no exact match, try fuzzy matching
    if threshold < 1.0:
        for item in content_list:
            page = item['page']
            content = item['content'].strip().lower().replace("\n", " ")
            
            # Check if significant portion of search_paragraph is in content
            common_words = set(search_paragraph.split()) & set(content.split())
            similarity = len(common_words) / len(set(search_paragraph.split()))
            
            if similarity >= threshold:
                result_pages.append(page)
    
    # Line-by-line cross-page detection
    if not result_pages:
        # Sort content list by page number
        sorted_content = sorted(content_list, key=lambda x: x['page'])
        
        # Split the search paragraph into lines
        search_lines = search_paragraph.split('\n')
        if len(search_lines) == 1:  # If no newlines, try to split by periods
            potential_lines = re.split(r'(?<=[.!?])\s+', search_paragraph.strip())
            if len(potential_lines) > 1:
                search_lines = potential_lines
        
        # If there's only one line, we can't use this method effectively
        if len(search_lines) <= 1:
            return result_pages
        
        # Initialize tracking variables
        matched_pages = []
        remaining_lines = search_lines.copy()
        start_page_idx = 0
        
        # Continue until all lines are matched or we've tried all pages
        while remaining_lines and (start_page_idx < len(sorted_content)):
            found_match = False
            current_page_idx = start_page_idx
            current_page = sorted_content[current_page_idx]
            current_content = current_page['content'].strip().lower()
            
            # Try matching with different numbers of lines
            for num_lines in range(len(remaining_lines), 0, -1):
                chunk = ' '.join(remaining_lines[:num_lines]).lower()
                
                # Check if this chunk is in the current page
                if chunk in current_content.replace("\n", " "):
                    # Match found on this page
                    matched_pages.append(current_page['page'])
                    remaining_lines = remaining_lines[num_lines:]  # Remove matched lines
                    found_match = True
                    start_page_idx = current_page_idx + 1  # Move to next page for next iteration
                    break
            
            # If no match found with any number of lines on the current page
            if not found_match:
                start_page_idx += 1  # Try the next page
        
        # Check if we matched all lines across multiple pages
        if not remaining_lines and len(matched_pages) > 0:
            # Check for consecutive pages
            is_consecutive = True
            for i in range(len(matched_pages) - 1):
                if matched_pages[i] + 1 != matched_pages[i + 1]:
                    is_consecutive = False
                    break
            
            if is_consecutive:
                return matched_pages
        
        # If we reach here with no results, try one more approach for two-page spans
        # This is a fallback for cases where the line-by-line approach doesn't work
        if len(sorted_content) >= 2:
            for i in range(len(sorted_content) - 1):
                current_page = sorted_content[i]
                next_page = sorted_content[i + 1]
                
                # Only consider consecutive pages
                if next_page['page'] != current_page['page'] + 1:
                    continue
                
                # Get content from both pages
                current_content = "\n".join([line for line in current_page['content'].lower().split("\n")])
                next_content = "\n".join([line for line in next_page['content'].lower().split("\n")])
                
                # Check if first part is at the end of current page
                # and second part is at the beginning of next page
                if search_paragraph.replace("\n", " ") in (current_content + "\n" + next_content).replace("\n", " "):
                    return [current_page['page'], next_page['page']]
    
    return result_pages

# Example usage:
if __name__ == "__main__":
    # Example content list
    content_list = [
  {
    "page": 1,
    "content": "TERMA DAN SYARAT KAD DEBIT HONG LEONG (English Version)\nDikemaskini pada 6 Januari 2025\nSebagai pertimbangan Hong Leong Bank Berhad atau Hong Leong Islamic Bank Berhad (\"Bank\")bersetuju\nuntuk menyediakan Perkhidmatan dan bersetuju untuk mengeluarkan Kad Debit (seperti yang ditentukan\ndalam Klausa 1) termasuk Gabungan (sebagaimana didefinisikan dalam Klausa 1.2) kepada Pemegang\nKad atas permohonan Pemegang Kad, Pemegang Kad berjanji dan bersetuju bahawa penurunan\ntandatangan pada Kad Debit, penggunaan Kad Debit dan/atau pengekalan Akaun oleh Pemegang Kad\nakan membentuk perjanjian Pemegang Kad dengan terma dan syarat (\u201cT&S\u201d) di bawah.\nT&S ini hendaklah dibaca bersama sebagai keseluruhan dengan Terma dan Syarat Am Akaun Bank,\nTerma dan Syarat Penggunaan HLB Connect dan sebarang terma dan syarat akaun yang berkaitan, serta\nsyarat dan peraturan lain yang mengikat Bank. Definisi berikut berkuatkuasa kecuali dinyatakan sebaliknya:\n1.0\nDefinisi dan Interpretasi\n1.1\n\u201cAkaun\u201d bermaksud akaun atau akaun-akaun Pelanggan dengan HLB / HLISB dan hendaklah\ntermasuk mana-mana akaun baharu lain yang dibuka dari semasa ke semasa\n1.2\n\u201cGabungan\u201d bermaksud Kad Debit yang ditawarkan oleh institusi kewangan dalam perkongsian\ndengan institusi lain.\n1.3\n\u201cFi Tahunan\u201d bermaksud fi yang dikenakan ke atas Pemegang Kad setiap tahun.\n1.4\n\"Pihak Berkuasa Berkenaan\" bermaksud mana-mana kerajaan atau pihak berkuasa percukaian.\n1.5\n\u201cATM\u201d bermaksud mesin juruwang automatik:\n(a) yang dipasang oleh Bank atau mana-mana ahli Rangkaian ATM Kongsi (Shared ATM Network\n(SAN)); dan/atau\n(b) ditetapkan oleh Bank atau Visa/Mastercard,\nuntuk penggunaan Pemegang Kad.\n1.6\n\u201cTransaksi Kad ATM\u201d bermaksud penggunaan Kad Debit untuk pengeluaran tunai dan transaksi\nelektronik atau mana-mana kad yang mungkin diluluskan oleh Bank dari semasa ke semasa.\n1.7\n\u201cRangkaian Tunai Dibenarkan\u201d bermaksud cawangan, pejabat dan/atau lokasi yang ditetapkan\noleh Rangkaian ATM Kongsi, Visa/Mastercard untuk melaksanakan pengeluaran wang tunai.\n1.8\n\u201cPeniaga Sah\u201d bermaksud mana-mana peruncit atau perbadanan yang mana selaras dengan\nPerjanjian Peniaga bersetuju untuk menerima atau menyebabkan outletnya menerima Kad Debit\nuntuk\npembayaran\natau\nmenurut\nkepada\nperkiraan\nundang-undang\ndengan\nMyDebit/Visa/Mastercard bersetuju untuk menerima atau menyebabkan rangkaiannya untuk\nmenerima kemudahan yang ditawarkan bergabung atau berkongsi jenama Visa/Mastercard untuk\npembayaran."
  },
  {
    "page": 2,
    "content": "1.9\n\u201cTransaksi Auto Debit\u201d bermaksud bayaran berulang melalui Kad Debit untuk caj utiliti, insurans\natau takaful yang diluluskan oleh Bank sahaja.\n1.10\n\u201cBank\u201d bermaksud sama ada Hong Leong Bank Berhad (193401000023 (97141-X)) atau Hong\nLeong Islamic Bank Berhad (200501009144 (686191-W)) dan termasuk pewaris-namanya dan\npemegang serah-hak dibenarkan.\n1.11\n\u201cHari Perniagaan\u201d bermaksud hari apabila HLB / HLISB dibuka untuk perniagaan di semenanjung\nMalaysia, Sabah dan Sarawak (tidak termasuk hari kelepasan bank, negeri dan hari kelepasan am),\nmengikut mana yang berkenaan, dan hari apabila transaksi yang jenisnya dicadang bagi Akaun\ndijalankan.\n1.12\n\u201cPemegang Kad\u201d bermaksud Pemegang Kad Bank yang mana Kad Debit diterbitkan kepadanya.\n1.13\n\u201cFi Penggantian Kad\u201d bermaksud fi yang dikenakan ke atas Pemegang Kad jika kad hilang, dicuri\natau rosak.\n1.14\n\u201cTransaksi Kad\u201d bermaksud transaksi yang dilaksanakan dengan menggunakan Kad Debit untuk\nurus niaga tempatan dan luar negara; Transaksi dengan Kad secara bersemuka sertaTransaksi\nTanpa Kad dan Transaksi Tanpa Sentuhan secara tidak bersemuka (di mana berkenaan).\n1.15\n\u201cTransaksi Card-Present (CP)\u201d bermaksud pembayaran Transaksi Runcit di mana Pemegang Kad\ndan Kad Debit hadir secara fizikal di Peniaga Sah apabila pembayaran dilakukan.\n1.16\n\u201cTransaksi Card-Not-Present (CNP)\u201d bermaksud pembayaran Transaksi Runcit di mana\nPemegang Kad dan Kad Debit tidak hadir secara fizikal di Peniaga Sah apabila pembayaran\ndilakukan.\n1.17\n\u201cBaki Semasa\u201d bermaksud baki atau tunai terkini yang ada di dalam akaun simpanan atau\nsemasa Pemegang Kad.\n1.18\n\u201cFi Pengeluaran Tunai\u201d bermaksud fi yang dikenakan ke atas Pemegang Kad untuk pengeluaran\ntunai berjaya dari ATM.\n1.19\n\u201cTransaksi Tanpa Sentuhan\u201d bermaksud kaedah pembayaran yang cepat dan mudah yang\nmenggunakan teknologi komunikasi medan dekat (NFC) bagi membuat pembayaran dengan\nmenyentuh/mengimbas Kad Debit pada pembaca selamat.\n1.20\n\u201cKad Karakter\u201d merujuk kepada Kad Debit seperti Kad Debit Hello Kitty atau Transformers yang\ndikeluarkan oleh Bank.\n1.21\n\u201cHad Pengeluaran Tunai Harian\u201d bermaksud had maksimum harian dibenarkan yang ditetapkan\noleh Bank berhubung pengeluaran tunai melalui ATM.\n1.22\n\u201cHad Belian Dalam Talian Harian\u201d bermaksud had maksimum belian harian dalam talian\ndibenarkan yang ditetapkan oleh Bank dalam Klausa 12.4 di sini.\n1.23\n\u201cHad Belian Runcit Harian\u201d bermaksud had maksimum belian runcit harian dibenarkan yang\nditetapkan oleh Bank dalam Klausa 12.3 di sini."
  },
  {
    "page": 3,
    "content": "1.24\n\u201cHad Pemindahan Harian\u201d bermaksud had maksimum harian yang ditetapkan oleh Bank\nberkenaan dengan pemindahan wang melalui ATM dan/atau Hong Leong Connect.\n1.25\n\u201cKad Debit\u201d bermaksud Kad Debit MyDebit/Visa/Mastercard Hong Leong yang dikeluarkan oleh\nBank.\n1.26\n\u201cDCC\u201d merujuk kepada Penukaran Mata Wang Dinamik, iaitu perkhidmatan pilihan yang\nditawarkan oleh peniaga atau tersedia untuk pembelian luar negara tertentu seperti yang\nditentukan dalam Klausa 20.1 di bawah (termasuk Transaksi Kad ATM), yang memberi pilihan\npembayaran / pengeluaran tunai dalam Ringgit Malaysia (termasuk pembelian di luar negara dan\npembelian dalam talian dalam mata wang asing) untuk transaksi luar negara, sebagaimana yang\ndihuraikan dalam Klausa 20.\n1.27\n\u201cFi\u201d termasuk fi perlu dibayar oleh Pemegang Kad semasa permohonan, tahunan atau sebarang\njarak waktu yang mungkin ditentukan oleh Bank untuk menggunakan Perkhidmatan yang akan\ndidebit dari Akaun pada setiap tarikh ulangtahun Kad Debit dikeluarkan dan akan merangkumi\nsemua fi lain, caj perkhidmatan, komisen dan lain-lain bayaran yang dicaj oleh Bank di bawah T&S\nini. Bank berhak untuk mengubah Fi dengan memberi notis awal dua puluh satu (21) hari kalendar\nterlebih dahulu kepada Pemegang Kad.\n1.28\n\u201cPemegang Kad Biasa\u201d bermaksud Pemegang Kad bukan Perbankan Prioriti.\n1.29\n\u201cTarikh Baik Sehingga\u201d bermaksud tarikh luput yang dicetak pada Kad Debit.\n1.30\n\u201cBarangan dan Perkhidmatan yang mematuhi Syariah\u201d bermaksud barangan & perkhidmatan\nyang boleh diterima dan mematuhi Syariah.\n1.31\n\u201cHLB Connect App\u201d bermaksud Perbankan Mudah Alih Hong Leong Connect.\n1.32\n\u201cHong Leong Connect\u201d bermaksud perkhidmatan perbankan internet dan perkhidmatan\naplikasi mudah alih yang disediakan oleh HLB / HLISB.\n1.33\n\u201cPemegang Akaun Bersama\u201d bermaksud mana-mana Akaun yang ada dua (2) atau lebih nama\nbersama atau mempunyai dua (2) atau lebih yang diberi kuasa penandatangan.\n1.34\n\u201cMastercard\u201d bermaksud MasterCard International Incorporated, sebuah syarikat yang\ndikendalikan di bawah Negeri Delaware dengan pejabat dan pusat perniagaan utamanya di 2000\nPurchase Street, Purchase, N.Y. 10577-2509, Amerika Syarikat di mana Bank adalah ahli\ninstitusinya.\n1.35\n\"MCCS\" bermaksud Spesifikasi Kad Cip Malaysia, iaitu ialah skim kad debit EMV (Euro,\nMastercard, dan Visa) domestik yang menyokong kedua-dua aplikasi sentuhan dan tanpa\nsentuhan.\n1.36\n\u201cAkaun Diaktifkan MCF\u201d bermaksud Akaun Pemegang Kad di mana MCF (seperti yang ditakrifkan\ndalam Klausa 1.45 di sini) telah diupayakan, yang mungkin termasuk akaun yang dipautkanke Kad\nDebit. telah diaktifkan, yang mungkin termasuk Akaun Pembelian Runcit.\n1.37\n\u201cMEPS\u201d bermaksud Malaysian Electronic Payment System Sdn Bhd, pembekal perkhidmatan\nantara bank di Malaysia."
  },
  {
    "page": 4,
    "content": "1.38\n\u201cPerjanjian Peniaga\u201d bermaksud perjanjian yang ditandatangani di antara Peniaga Sah dan Bank.\n1.39\n\u201cFi Penyata Bulanan\u201d bermaksud fi bulanan yang dikenakan ke atas Pemegang Kad untuk\npenyata bulanan Kad Debit yang dicetak.\n1.40\n\"MOTO\" ertinya Pesanan Mel Pesanan Telefon yang merupakan jenis Transaksi Kad-Tidak-Hadir\ndi mana Perkhidmatan diselesaikan dan dibayar melalui telefon, mel, faks atau e-mel.\n1.41\n\u201cMyKad\u201d bermaksud kad yang dikeluarkan kepada Pemegang Kad oleh Jabatan Pendaftaran\nNegara Malaysia kepada Pemegang Kad untuk mengenalpasti and megesahkan identiti\nPemegang Kad.\n1.42\n\"MyDebit\" bermaksud suatu skim kad debit domestik yang membenarkan Pemegang Kad untuk\nmembuat bayaran bagi pembelian barangan dan/atau perkhidmatan dengan mendebit secara\nlangsung dari Akaun Pembelian Runcit Pemegang Kad.\n1.43\n\u201cKad Debit Multi Currency\u201d atau \u201cKad MCF\u201d atau \u201cKad Debit Multi Currency Visa Hong\nLeong\u201d merujuk kepada Kad Debit dengan asas warna hitam yang dikeluarkan oleh Bank kepada\npemegang Akaun Pay&Save Hong Leong dan hanya boleh dipautkan kepada Akaun Pembelian\nRuncit ke Akaun Pay&Save Hong Leong.\n1.44\n\u201cCiri Mata Wang Pelbagai\u201d atau \u201cMCF\u201d bermaksud mata wang asing yang disimpan dalam Akaun\nyang dibenarkan oleh Bank untuk MCF yang membolehkan Pemegang Kad, antara lain:\n(a) memegang mata wang asing yang dibenarkan oleh Bank dari semasa ke semasa;\n(b) melakukan Transaksi Luar Negara dalam mata wang asing yang dipilih oleh Pemegang Kad;\n(c) menukar satu (1) mata wang asing ke mata wang asing yang lain dalam Akaun Diaktifkan MCF\nPemegang Kad melalui cawangan Bank atau Hong Leong Connect.\nBagi mengelakkan keraguan, MCF hanya tersedia untuk Akaun Diaktifkan MCF yang dipegang\noleh Pemegang Akaun tunggal. Mata wang asing terkini yang diluluskan untuk MCF boleh didapati\nhttps://www.hlb.com.my/ms/personal-banking/deposits/savings-account/pay-and-save-\naccount.html.\n1.45\n\u201cPaparan Notifikasi HLB Connect App\u201d bermaksud ciri notifikasi tolak yang membolehkan\nPemegang Kad menerima notifikasi melalui HLB Connect App.\n1.46\n\u201cOne-Time Password (OTP)\u201d bermaksud kod 6-digit yang unik dan sekali guna yang dihantar\nmelalui SMS atau Kad OTP dalam Aplikasi HLB Connect untuk melakukan transaksi dalam talian\ntertentu atau tinkdakan.\n1.47\n\u201cHad Belian Dalam Talian\u201d bermaksud had transaksi yang terpakai ke atas Transaksi Kad yang\ndilaksanakan dalam talian melalui internet.\n1.48\n\u201cPembelian Petrol\u201d bermaksud semua transaksi yang dibuat dengan Kad Debit untuk\npembayaran petrol dan mana-mana pembayaran lain yang dilakukan di kiosk petrol.\n1.49\n\u201cPIN\u201d bermaksud kod berangka yang bertindak sebagai kata laluan yang digunakan untuk\nmengesahkan Pemegang Kad ke akaun mereka\n1.50\n\u201cTarikh Pengeposan\u201d bermaksud tarikh di mana sebarang Transaksi Kad yang dilakukan oleh"
  },
  {
    "page": 5,
    "content": "Pemegang Kad didebitkan dan direkodkan dalam Penyata.\n1.51\n\u201cPemegang Kad Perbankan Prioriti\u201d bermaksud Pemegang Kad dengan status Perbankan\nPrioriti yang diperakui oleh Bank.\n1.52\n\u201cNotis Privasi\u201d bermaksud dasar dan prinsip Bank yang berkaitan dengan pengumpulan,\npenggunaan dan penyimpanan maklumat peribadi individu dan entiti sedia ada dan prospektif yang\nberurusan dengan Bank yang mana mungkin akan dipinda dari semasa ke semasa dan diterbitkan\ndi laman sesawang Bank atau dengan cara yang dianggap wajar oleh Bank.\n1.53\n\u201cHad Belian Runcit\u201d bermaksud had transaksi yang dikuatkuasakan bagi Transaksi Kad termasuk\nTransaksi Tanpa Sentuhan yang dibuat di kedai runcit, atau mana-mana kedai lain di mana Kad\nDebit boleh diterima untuk pembayaran.\n1.54\n\u201cAkaun Pembelian Runcit\u201d bermaksud akaun yang dikaitkan dengan Kad Debit untuk membuat\nTransaksi Runcit.\n1.55\n\"Transaksi Runcit\" bermaksud transaksi yang menggunakan Kad Debit di Peniaga Sah di mana\nKad Debit boleh diterima untuk pembayaran di dalam dan luar negara termasuk Pembelian Dalam\nTalian, tambah nilai Touch 'n Go secara automatik, Pembelian Petrol, Auto Bill/transaksi urusniaga\nyang berulang dan tidak termasuk pengeluaran wang tunai, pemindahan dana, pembayaran yuran\ntahunan dan perkhidmatan lain/yuran pelbagai seperti yang ditakrifkan oleh Bank dari semasa ke\nsemasa dengan memberi notis terlebih dahulu.\n1.56\n\u201cFi Pencarian Rekod Jualan\u201d bermaksud fi yang dikenakan ke atas Pemegang Kad apabila\nPemegang Kad memohon untuk mendapatkan rekod jualan yang diisu oleh Peniaga Sah dari\nBank. Fi ini juga dikenakan semasa penyiasatan kes pertikaian apabila Pemegang Kad memohon\nuntuk melihat rekod jualan yang berkaitan.\n1.57\n\u201cKod Keselamatan\u201d bermaksud kod keselamatan yang diberikan oleh Bank kepada Pemegang\nKad untuk mengakses Perkhidmatan berkaitan yang merangkumi PIN (untuk Perkhidmatan ATM),\nIPIN (untuk Hong Leong Connect), HLB Connect Code (untuk Hong Leong Connect) dan termasuk\nlain-lain nama pengguna, kata laluan, nombor pengenalan peribadi, sijil digital atau lain-lain kod\nkeselamatan yang mungkin dikeluarkan oleh Bank dari semasa ke semasa untuk mengakses\nsemua atau mana-mana Perkhidmatan dan rujukan untuk terma \u201cKod Keselamatan\u201d\nakanbermaksud kod atau kod-kod keselamatan berkaitan dengan Perkhidmatan tersebut\nsebagaimanadiperlukan.\n1.58\n\u201cPerkhidmatan\u201d bermaksud perkhidmatan yang disediakan oleh Bank untuk memudahkan\nTransaksi Kad, Transaksi Tanpa Sentuhan, transaksi Cip dan PIN, Perbankan Telefon,\nPerkhidmatan Pembayaran Bil, atau mana-mana perkhidmatan elektronik, digital, visual, video\natau lain-lain seperti perkhidmatan baru atau tambahan yang ditawarkan oleh Bank dari semasa\nke semasa bagi membolehkan Pemegang Kad untuk melaksanakan transaksi perbankan dan lain-\nlain transaksi dan Perkhidmatan itu boleh ditambah, diubah atau ditarik balik oleh Bank pada bila-\nbila masa dan dari semasa ke semasa dengan memberi notis awal dua puluh satu (21) hari\nkalendar terlebih dahulu.\n1.59\n\u201cRangkaian ATM Kongsi\u201d bermaksud bank dan institusi kewangan yang mengambil bahagian di\ndalam Rangkaian ATM Kongsi yang dikendalikan oleh Payments Network Malaysia Sdn Bhd.\n1.60\n\u201cPenyata\u201d bermaksud penyata akaun mencerminkan urusniaga yang dilaksanakan dengan Kad"
  },
  {
    "page": 6,
    "content": "Debit.\n1.61\n\"Cukai\" bermaksud sebarang cukai semasa atau masa hadapan, langsung atau tidak langsung,\ncukai, levi, impos, duti, caj, yuran, sebarang bentuk potongan atau pegangan dalam sebarang\nbentuk, Malaysia atau asing, yang dikenakan oleh Pihak Berkuasa Berkenaan, termasuk tanpa\nhad, sebarang cukai penggunaan dan lain-lain cukai dengan apa sahaja nama yang dipanggil, dan\nsebarang faedah, denda atau penalti yang berkenaan dengannya.\n1.62\n\"Tarikh Transaksi\" bermaksud tarikh di mana Transaksi Pembelian Runcit tersebut\nmenggunakan Kad Debit di Peniaga Sah di mana Kad Debit adalah diterima untuk pembayaran.\n1.63\n\u201cResit Transaksi\u201d bermaksud slip bayaran yang berkaitan, borang atau kertas yang dibekalkan\nkepada/oleh Bank kepada/oleh Peniaga Sah atau dijana oleh ATM untuk tujuan merekod,\nmengesah dan membuktikan pembelian atau transaksi lain yang dilakukan oleh Pemegang Kad\nmelalui penggunaan Kad Debit yang akan dicajkan dan didebitkan daripada Akaun atau\npengesahan yang diberikan kepada Pemegang Kad untuk transaksi kewangan yang dilakukan\nmelalui Hong Leong Connect.\n1.64\n\u201cVisa\u201d bermaksud Visa International Service Association, sebuah syarikat yang dikendalikan di\nbawah Negeri Delaware dengan ibu pejabatnya di 3125 Clearview Way, San Mateo, CA 94402,\nAmerika Syarikat di mana Bank adalah ahli institusinya.\n1.65\nPerkataan yang bermaksud tunggal juga bermaksud majmuk dan sebaliknya serta serta perkataan\nyang bermaksud jantina maskulin hendaklah termasuk jantina feminin dan neuter dan sebaliknya.\n2.0\nTanggungjawab Pemegang Kad\n2.1\nDengan menandatangani Kad Debit, menggunakan Kad Debit dan/atau mengekalkan Akaun,\nPemegang Kad bersetuju untuk terikat sepenuhnya dengan T&S ini yang mungkin diubah oleh\nBank dari semasa ke semasa dengan memberi notis awal dua puluh satu (21) hari kalendar. Notis\nmengenai perubahan atau pindaan tersebut akan dimaklumkan kepada Pemegang Kad dalam\nsebarang bentuk atau kaedah sebagaimana yang ditetapkan oleh Bank.\n2.2\nPemegang Kad harus mengambil semua langkah wajar untuk memastikan dan mencegah\npenggunaan tanpa kebenaran dan secara penipuan dan meyalahi undang-undang ke atas\nPerkhidmatan atau sebahagian daripadanya dan Kod Keselamatan Pemegang Kad, termasuk\ntetapi tidak terhad kepada yang berikut:\n(a) Kod Keselamatan hendaklah dirahsiakan dan apabila Pemegang Kad menerima Kod\nKeselamatan ia hendaklah ditukar sertamerta apabila telah diterima dan dibaca, dan hanya\nboleh digunakan oleh Pemegang Kad sahaja dan bukan orang lain. Jika Kod Keselamatan\ntidak dikeluarkan kepada Pemegang Kad, Pemegang Kad dinasihatkan agar mencipta Kod\nKeselamatan sendiri sebagai syarat untuk mengakses Perkhidmatan.\n(b) Bahawa tidak ada orang lain selain daripada Pemegang Kad mempunyai akses atau mampu\nmempunyai akses kepada Kod Keselamatan untuk membolehkan dia melakukan sebarang\ntransaksi melalui mana-mana Perkhidmatan dan Kod Keselamatan tidak ditulis dalam apa jua\nbentuk atau cara termasuk tetapi tidak terhad kepada e-mel yang tidak diminta atau pada\nmana-mana laman sesawang selain daripada laman sesawang Bank yang boleh ditafsirkan\noleh pihak ketiga atau disimpan bersama atau didedahkan kepada mana-mana orang\n(termasuk pekerja Bank) di bawah apa-apa keadaan atau pada bila-bila masa."
  },
  {
    "page": 7,
    "content": "(c) Pemegang Kad tidak boleh mendedahkan Kod Keselamatan kepada mana-mana pihak di\nbawah apa jua sebab atau cara sama ada secara sukarela atau sebaliknya dan harus\nmengambil segala langkah wajar untuk mencegah Kod Keselamatan daripada diketahui oleh\npihak lain. Pemegang Kad memahami dan bersetuju bahawa kegagalan untuk mematuhi\nperaturan ini boleh mendedahkan Pemegang Kad kepada akibat kecurian dan/atau\npenggunaan tanpa kebenaran Kad Debit, yang mana Bank tidak akan bertanggungjawab.\nPemegang Kad mesti melaporkan sebarang pelanggaran Kod Keselamatan atau kehilangan\nKod Keselamatan kepada Bank dengan secepat yang mungkin, apabila Pemegang Kad\nmenyedari tentang pelanggaran atau kerugian masing-masing. Pemegang Kad dengan ini\nbersetuju untuk membayar balik dan membayar kepada Bank atas permintaan bertulis Bank\nsemua tuntutan dan liabiliti yang ditanggung oleh Bank yang timbul daripada penggunaan\ntanpa kebenaran tersebut.\n(d) Bahawa baki Akaun, Penyata Akaun dan Transaksi Kad diperiksa dan dipantau oleh\nPemegang Kad secara kerap dan teliti dan diperiksa setiap kali sebelum memberikan apa-apa\narahan kepada Bank. Pemegang Kad hendaklah memaklumkan kepada Bank dengan segera\ntentang apa-apa yang tidak dibenarkan, penipuan, menyalahi undang-undang urus niaga,\nkesilapan atau percanggahan dalam Akaun.\n(e) Pemegang Kad harus memastikan bahawa dana di dalam Akaun adalah mencukupi untuk\nmelaksanakan sebarang Transaksi Kad yang diarahkan oleh Pemegang Kad.\n(f) Pemegang Kad akan mencipta PIN sendiri pada ATM Bank untuk tujuan melaksanakan\ntransaksi ATM atau transaksi elektronik. Pemegang Kad dikehendaki untuk tidak mendedahkan\nPIN kepada mana-mana pihak ketiga dan akan bertanggungjawab kepada Bank untuk sebarang\nkemasukan debit di dalam Akaun dengan Bank yang diakibatkan oleh Transaksi Kad tanpa\nkebenaran.\n(g) Untuk melakukan pembelian dengan menggunakan Kad Debit di mana-mana Peniaga Sah,\nPemegang Kad boleh mengesahkan Transaksi Kad dengan (i) menandatangan Resit\nTransaksi yang disediakan oleh Peniaga Sah; (ii) memasukkan PIN pada terminal Tempat\nJualan; atau (iii) melaksanakan Transaksi Tanpa Sentuhan. Penerimaan Resit Transaksi atau\napa-apa dokumen pulangan dengan tandatangan Pemegang Kad oleh Bank (jika ada) akan\nmenjadi bukti yang mengikat dan muktamad bahawa Transaksi Kad dan amaun yang direkod\npadanya adalah dilakukan oleh Pemegang Kad selaras dengan T&S ini. Untuk pembelian atau\npenggunaan Kad Debit yang dibuat secara transaksi dalam talian, atau transaksi bukan\nberasaskan tandatangan termasuk tetapi tidak terhad kepada pembelian petrol, transaksi\ndalam talian, Pemegang Kad bersetuju bahawa resit pengesahan dan/atau penerimaan yang\ndikeluarkan oleh mana-mana Peniaga Sah atau sekutunya untuk transaksi tersebut akan\ndianggap sebagai bukti dokumen yang memuaskan sebagai penggunaan dan Pemegang Kad\nharus mematuhi T&S ini.\n(h) Walapun peruntukan yang dinyatakan di bawah Klausa 2.2(g) di atas, tandatangan/PIN bukan\nsatu-satunya syarat yang boleh didahulukan kepada liability Pemegang Kad berkenaan\ndengan perkhidmatan yang dibekalkan, dan Pemegang Kad akan bertanggungjawab untuk\nTransaksi Kad jika Bank berpendapat, berdasarkan bukti dokumen yang memuaskan, bahawa\nketidaksempurnaan tandatangan atau ketiadaan tandatangan/kemasukan PIN adalah\ndisebabkan oleh keadaan dan/atau kaedah transaksi yang digunakan atau akibat kesilapan\ndari pihak Pemegang Kad dan/atau Peniaga Sah dan ahli bank MyDebit/Visa/ Mastercard.\n(i) Pemegang Kad harus mematuhi semua keperluan, arahan, suruhan, dan garis panduan"
  },
  {
    "page": 8,
    "content": "tentang penggunaan Kad Debit yang dikeluarkan oleh Bank dari semasa ke semasa\nberkenaan dengan semua Perkhidmatan kepada Pemegang Kad.\n(j) Pemegang Kad bersetuju untuk menandatangan bahagian belakang Kad Debit sebaik sahaja\nkad tersebut diterima dan sebelum ia digunakan, sebagai langkah mencegah penggunaan\ntanpa kebenaran Kad Debit tersebut.\n(k) Kad Debit adalah sah sehingga Tarikh Baik Sehingga. Surat pembaharuan kepada Pemegang\nKad akan dikirimkan tiga puluh (30) hari sebelum Tarikh Baik Sehingga. Bank akan mengirim\nsurat pembaharuan kepada Pemegang Kad untuk memaklumkan Pemegang Kad untuk\nmengunjungi cawangan Bank yang terdekat atau mana-mana cawangan Bank untuk\nmemperolehi Kad Debit Penggantian. Pemegang Kad harus memastikan bahawa sebaik\nsahaja Kad Debit luput, ia dimusnahkan, dengan memotong dua secara melintang dan\nmengembalikannya kepada Bank untuk penggantian Kad Debit.\n(l) Kad Debit tidak boleh digunakan selepas pembatalan, tamat tempoh, atau pengeluaran atau\napabila Pemegang Kad tidak lagi menjadi Pemegang Kad Bank.\n(m) Pemegang Kad tidak boleh menggunakan Kad Debit untuk pengeluaran tunai, bayaran atau\npemindahan dana melainkan terdapat dana yang mencukupi di dalam Akaun. Sebaran\npengeluaran tunai, bayaran atau pindahan dana akan ditolak jika tidak terdapat dana yang\nmencukupi di dalam Akaun.\n(n) Sekiranya tiada terdapat dana mencukupi di dalam Akaun dan Pemegang Kad tidak\nmempunyai kemudahan overdraf atau mana-mana kemudahan kredit tersedia, semua\nTransaksi Kad di Peniaga Sah akan ditolak.\n(o) Pemegang Kad bersetuju untuk tidak menggunakan Kad Debit untuk sebarang aktiviti yang\nmenyalahi undang-undang. Jika Bank mendapati, mengesyaki atau mempunyai sebab untuk\nmempercayai bahawa Kad Debit telah digunakan untuk sebarang aktiviti yang menyalahi\nundang-undang, Bank boleh mengambil apa-apa tindakan dianggap sesuai untuk memenuhi\napa-apa kewajipan yang berkaitan dengan pencegahan apa-apa aktiviti haram termasuk tetapi\ntidak terhad kepada penipuan, pengubahan wang haram, aktiviti/pembiayaan pengganas,\nrasuah, penyalahgunaan wang dan/atau pengelakan cukai.\n(p) Sekiranya Pemegang Kad menjadikan Akaun Simpanan-i atau Akaun Semasa-i sebagai\nAkaun Pembelian Runcit, Pemegang Kad mengakui dan bersetuju bahawa Kad Debit tidak\nboleh digunakan untuk sebarang aktiviti atau tujuan tidak patuh Syariah seperti yang\ndinyatakan di Klausa 13 di bawah dan yang boleh menyebabkan transaksi tersebut ditolak.\n(q) Pemegang Kad dengan ini memberi representasi dan memberi waranti bahawa Pemegang\nKad telah mendapat persetujuan semua orang yang dinamakan dalam permohonan\nPemegang Kad untuk Akaun atau apa-apa dokumen lain yang dihantar kepada Bank bagi\nmenyokong permohonan itu dan/atau wakil yang diberi kuasa mereka, termasuk tetapi tidak\nterhad kepada pengarah, pemegang saham, penandatangan yang diberi kuasa Pemegang\nKad atau mana-mana orang lain yang ditetapkan oleh Bank (\"Subjek Data Relevan\"), untuk\nkoleksi Bank, pegangan dan penggunaan maklumat peribadi daripada subjek Data Relevan\nmengikut Notis Privasi Bank sebagaimana yang dipinda dari semasa ke semasa.\n(r) Sekiranya Pemegang Kad menggantikan Kad Debit beliau dengan Kad Debit baru kerana\nhilang/dicuri/pembaharuan atau apa-apa sebab lain, Pemegang Kad bertanggungjawab untuk\nmemaklumkan kepada semua Peniaga Sah berkaitan yang beliau telah melanggan auto\ndebit/transaksi debit langsung untuk mengemaskini nombor Kad Debit baru untuk"
  },
  {
    "page": 9,
    "content": "mengelakkan transaksi daripada ditolak.\n(s) Pemegang Kad dengan ini bersetuju dan memberi kebenaran kepada pegangan,\npengumpulan dan penggunaan semua data peribadi yang diberikan kepada Bank oleh\nPemegang Kad dan/atau diperoleh oleh Bank dari domain awam, dan juga data peribadi yang\ntimbul akibat daripada penyediaan perkhidmatan kepada Pemegang Kad berhubung dengan\nAkaun-Akaun yang selaras dengan Notis Privasi Bank sebagaimana yang dipinda dari semasa\nke semasa.\n(t) Pemegang Kad yang ingin dikecualikan daripada pekongsian data peribadi mereka di dalam\nBank dan/atau kumpulan syarikat Hong Leong Financial Group Berhad bagi tujuan pemasaran\ndan promosi perlu mengunjungi mana-mana cawangan Bank atau menghubungi Pusat\nPanggilan Bank di 03-7626 8899 untuk mendaftar permintaan mengecualikan pekongsian\ntersebut.\n(u) Pemegang Kad hendaklah segera memberitahu Bank jika ada apa-apa perubahan dalam\nbutir-butir perhubungan dan/atau peribadi Pemegang Kad sama ada secara bertulis,\nmengunjungi cawangan-cawangan Bank atau menghubungi Pusat Hubungan Bank di 03-7626\n8899.\n(v) Bank tidak menyatakan atau menjamin bahawa penggunaan Kad Debit tidak akan berlaku\ntanpa gangguan, gangguan atau kesilapan. Bank tidak akan bertanggungjawab ke atas apa-\napa kerugian, kerosakan, kesulitan, keaiban, kos atau perbelanjaan dalam apa jua bentuk yang\ndalam apa-apa cara mungkin dialami atau ditanggung oleh Pemegang Kad atau oleh mana-\nmana orang lain berkenaan dengan atau berkaitan dengan penggunaan Perkhidmatan\ndisebabkan oleh sebarang kejadian di luar kawalan munasabah Bank dan Pemegang Kad\ntidak akan mempunyai tuntutan terhadap Bank jika Transaksi Kad dan/atau Transaksi Tanpa\nSentuhan tidak diterima atau dihormati oleh mana-mana Peniaga Sah atau atas apa jua sebab\nsekalipun.\n(w) Pemegang Kad mengakui dan bersetuju bahawa penggunaan Kad Debit atau prestasi\nTransaksi Kad dan/atau Transaksi Tanpa Sentuhan adalah atas risiko mutlak Pemegang Kad.\nBank tidak akan bertanggungjawab ke atas sebarang kerosakan dan/atau kerugian yang\nditanggung berhubung dengan penggunaan Kad Debit atau prestasi Transaksi Kad dan/atau\nTransaksi Tanpa Sentuhan bagi barangan atau perkhidmatan yang dibekalkan oleh mana-\nmana Peniaga Sah atau kualiti atau pelaksanaan apa-apa barangan atau perkhidmatan\nselaras dengan atau berhubung dengan apa-apa transaksi yang dibuat pada Kad Debit.\nPemegang Kad akan menyelesaikan secara langsung dengan Peniaga Sah bagi apa-apa\naduan yang timbul daripada Transaksi Kad dan/atau Transaksi Tanpa Sentuhan.\n(x) Pemegang Kad harus memastikan bahawa jika beliau mempunyai lebih daripada satu (1) kad\natau instrumen pembayaran dengan teknologi tanpa sentuh, iaitu instrumen pembayaran\ndengan keupayaan untuk melaksanakan Transaksi Tanpa Sentuhan, Pemegang Kad harus\nmemastikan bahawa kad tersebut disimpan secara berasingan daripada Kad Debit untuk\nmengelakkan urus niaga perlanggaran yang akan menyebabkan kedua-dua kad didebitkan\nuntuk jumlah urus niaga yang berkaitan.\n(y) Di mana debit atau kedudukan terlebih pengeluaran yang tidak diatur sebelumnya timbul dalam\nakaun semasa atas apa jua sebab, Pelanggan bertanggungjawab dan hendaklah membayar\namaun terlebih keluar ditambah faedah yang boleh dikenakan iaitu 4.0% p.a. melebihi BLR pada\nbaki harian atas baki yang terlebih keluar."
  },
  {
    "page": 10,
    "content": "(z) Pemegang Kad harus memastikan semua Transaksi Kad (termasuk Transaksi Luar Negara\nseperti yang ditakrifkan di bawah Klausa 20.1 di bawah) mematuhi Notis Dasar Pertukaran\nAsing (\u201cNotis FEP\u201d). Pemegang Kad disarankan untuk membaca dan memahami Notis FEP\ndi laman sesawang Bank Negara Malaysia (\u201cBNM\u201d) di http://www.bnm.gov.my \u2013 tab \u2018Notis\nDasar Pertukaran Asing\u2019 untuk memastikan semua aktiviti yang dilakukan menggunakan Kad\nDebit mematuhi Notis FEP setiap masa.\n3.0\nTransaksi Tanpa Sentuhan\n3.1\nTerdapat tiga (3) jenis Transaksi Tanpa Sentuhan iaitu:\n(a) MCCS Tanpa Sentuhan \u2013 kaedah pembayaran tanpa sentuhan melalui rangkaian MyDebit\nyang boleh dilakukan di mana-mana terminal Point-of-Sales yang memaparkan logo MCCS\nTanpa Sentuhan;\n(b) Visa Tanpa Sentuhan \u2013 kaedah pembayaran tanpa sentuhan melalui rangkaian Visa yang\nboleh dilakukan di mana-mana terminal Point-of-Sales yang memaparkan logo Visa Tanpa\nSentuhan; dan\n(c) Mastercard Tanpa Sentuhan \u2013 kaedah pembayaran tanpa sentuhan melalui rangkaian\nMastercard yang boleh dilakukan di mana-mana terminal Point-of-Sales yang memaparkan\nlogo Mastercard Tanpa Sentuhan.\n3.2\nTransaksi Tanpa Sentuhan tanpa pengesahan PIN dihadkan kepada Ringgit Malaysia Dua Ratus\nLima Puluh (RM250) setiap transaksi (\u201cHad Transaksi Tanpa Sentuhan\u201d). Pemegang Kad akan\ndikehendaki mengesahkan PIN bagi Transaksi Tanpa Sentuhan melebihi Ringgit Malaysia Dua\nRatus Lima Puluh (RM250).\n3.3\nTransaksi Tanpa Sentuhan tanpa pengesahan PIN boleh dilakukan secara berasingan tetapi\ntransaksi kumulatif dalam satu (1) hari hendaklah dihadkan kepada Ringgit Malaysia Lapan Ratus\n(RM800) sahaja atau had lain sebagaimana yang ditetapkan oleh Pemegang Kad mengikut Klausa\n3.4 (\u201cHad Kumulatif Transaksi Harian Tanpa Sentuhan\u201d). Apa-apa Transaksi Tanpa Sentuhan\ntanpa pengesahan PIN yang melebihi Had Kumulatif Transaksi Harian Tanpa Sentuhan akan\ndibatalkan dan Pemegang Kad akan diminta untuk melakukan pengesahan PIN untuk meneruskan\nbayaran. Had Kumulatif Transaksi Harian Tanpa Sentuhan akan ditetapkan semula setiap hari atau\napabila Pemegang Kad melakukan transaksi disahkan PIN pada terminal Point-of-Sales.\n3.4\nHad Kumulatif Transaksi Harian Tanpa Sentuhan ditetapkan kepada Ringgit Malaysia Lapan Ratus\n(RM800). Pemegang Kad boleh menukarkan Had Kumulatif Transaksi Harian Tanpa Sentuhan\nmereka bila-bila masa dari RM0 \u2013 RM800 melalui Hong Leong Connect atau di mana-mana\ncawangan Bank di seluruh negara. Jika Pemegang Kad tidak ingin menggunakan fungsi Transaksi\nTanpa Sentuhan tanpa pengesahan PIN, Had Kumulatif Transaksi Harian Tanpa Sentuhan\nhendaklah ditetapkan kepada sifar, iaitu RM0.\n3.5\nHad Kumulatif Transaksi Harian Tanpa Sentuhan Pemegang Kad adalah subset Had Belian Runcit\nPemegang Kad. Oleh sebab itu, Had Kumulatif Transaksi Harian Tanpa Sentuhan tidak akan lebih\ndaripada Had Belian Runcit yang ditetapkan oleh Pemegang Kad. Had Kumulatif Transaksi Harian\nTanpa Sentuhan Pemegang Kad secara automatik akan ditetapkan kepada Had Belian Runcit\nsekiranya Had Belian Runcit kurang daripada Had Kumulatif Transaksi Harian Tanpa Sentuhan.\nContohnya, jika Pemegang Kad telah menetapkan Had Belian Runcit kepada Ringgit Malaysia\nSeratus (RM100), Had Kumulatif Transaksi Harian Tanpa Sentuhan akan menjadi Ringgit Malaysia\nSeratus (RM100)."
  },
  {
    "page": 11,
    "content": "4.0\nTransaksi Pembelian Dalam Talian\n4.1\nApabila Pemegang Kad membuat pembelian dalam talian dengan Kad Debit, Pemegang Kad akan\nmenerima OTP (One-Time Password) 6-digit dengan salah satu cara berikut:\n(i)\nMelalui OTP Kad Debit dalam HLB Connect App (jika Pemegang Kad berdaftar) atau\n(ii)\nMelalui SMS (jika Pemegang Kad tidak berdaftar dengan HLB Connect App)\n4.2\nUntuk melengkapkan pembelian dalan talian, Pemegang Kad dikehendaki::\n(i)\nSemak Kad Debit OTP melalui HLB Connect App atau SMS; dan\n(ii)\nMasukkan kod OTP 6-digit pada halaman pembayaran Peniaga\n5.0\nPenyata Pemegang Kad\n5.1\nPenyata akan dikemas kini ke dalam penyata/buku Akaun Simpanan/Akaun Simpanan-i atau\nAkaun Semasa/Akaun Semasa-i yang ditetapkan, dan Pemegang Kad boleh menyemak\nPenyatanya secara percuma melalui Hong Leong Connect yang meliputi, antara lain, Transaksi\nKad dan Tarikh Pengeposan Transaksi Kad yang dilakukan oleh Pemegang Kad untuk tempoh\nyang dinyataka.\n5.2\nAtas permintaan oleh Pemegang Kad, Penyata bulanan bercetak akan dihantar ke alamat surat-\nmenyurat Pemegang Kad yang terakhir dicatatkan di dalam rekod Bank. Caj perkhidmatan bulanan akan\ndidebitkan dari dalam Akaun. Penyata bulanan yang dicetak menunjukkan semua TransaksiKad,\nTarikh Pengeposan dan Tarikh Transaksi untuk bulan yang berkenaan. Pemegang Kad boleh\nmeminta Penyata dicetak pada bila-bila masa dan Bank akan mengenakan bayaran perkhidmatan\nsebanyak Ringgit Malaysia Sepuluh (RM10) dan Ringgit Malaysia Dua (RM2) bagi setiap halaman\nuntuk Penyata sehingga satu (1) tahun yang lalu atau Ringgit Malaysia Sepuluh (RM10) dan\nRinggit Malaysia Lima (RM5) bagi setiap halaman untuk Penyata lebih daripada satu (1) tahun\nyang lalu untuk setiap permintaan.\n5.3\nRekod dan catatan dalam Akaun yang ditetapkan dengan Bank yang terdapat pada Penyata\nbulanan akan dianggap sebagai tepat dan mengikat ke atas Pemegang Kad kecuali makluman\nbertulis bertentangan dengannya diberi oleh Pemegang Kad kepada Bank dalam masa empat\nbelas (14) hari kalendar selepas menerima.\n5.4\nJika atas sebarang sebab Pemegang Kad tidak memaklumkan kepada Bank secara bertulis dalam\nmasa empat belas (14) hari tentang sebarang percanggahan di dalam Penyata dan sekiranya\ntiada sebarang kesilapan yang jelas pada muka penyata atau penipuan oleh Bank,maka\nPemegangKad akan dianggap telah menerima rekod dan catatan di dalam Penyata sebagai betul,\nakhir danmuktamad. Penyata itu harus dianggap sebagai muktamad dan terikat kepada Pemegang\nKad, wakil perundangan dan pengganti Pemegang Kad, dan sebarang tuntutan atau dakwa\nterhadap Bank yang mendakwa Penyata itu adalah salah adalah tidak sah.\n5.5\nApabila Bank menerima notis bertulis yang diberi oleh Pemegang Kad dalam tempoh yang\ndinyatakan, Bank akan menyiasat Akaun untuk membuat pelarasan dan pembetulan yang\nsewajarnya, jika ada. Sebarang wang yang perlu dibayar kepada atau dibayar oleh Pemegang Kad\nakan dikredit atau didebit ke dalam Akaun berkaitan.\n5.6\nHanya Transaksi Kad yang telah disahkan dengan sewajarnya oleh Bank akan muncul di dalam\nPenyata Akaun Pemegang Kad.\n5.7\nPemegang Kad juga boleh mendapatkan rekod sepuluh (10) transaksi terakhir Akaun Simpanan"
  },
  {
    "page": 12,
    "content": "atau Semasa/Akaun Simpanan-i atau Semasa -i melalui ATM. Di dalam rekod transaksi ATM yang\ndikeluarkan oleh ATM, semua Transaksi Belian Runcit diwakili oleh perkataan tiga (3)-huruf \u201cPOS\u201d.\n6.0\nPemegang Akaun Bersama\n6.1\nUntuk Pemegang Akaun Bersama, salah seorang pemegang akaun boleh mengeluarkan arahan\ndan membenarkan Bank untuk melaksanakan sebarang Transaksi Kad ke atas akaun bersama.\nSemua Transaksi Kad yang dilakukan akan mengikat ke atas semua pemegang kad, yang\nbertanggungjawab secara bersama dan bersendiri. Mandat pemegang akaun bersama tidak akan\nditarik-balik kecuali dan sehingga pemegang akaun bersama yang mana mandatnya akan ditarik-\nbalik menyerahkan Kad Debitnya kepada Bank.\n7.0\nAkaun Pegangan\n7.1\nBank mungkin akan mendebit atau mengenakan pegangan ke atas Akaun Pembelian Runcit\n(\u201cAkaun Pegangan\u201d) untuk amaun Transaksi Kad sama ada pada hari ianya diserahkan kepada\nBank untuk bayaran atau pada hari Bank menerima makluman mengenai Transaksi Kad, yang\nmana lebih awal.\n7.2\nAkaun Pegangan ke atas akaun adalah sah untuk semua transaksi runcit yang dilakukan dengan\nPeniaga Sah yang memerlukan pegangan sedemikian. Jika Peniaga Sah meminta kebenaran\nuntuk sebarang Transaksi Kad, Bank mungkin akan mengenakan pegangan ke atas Akaun\nPegangan untuk amaun Transaksi Kad yang ditetapkan oleh Peniaga Sah and dipersetujui oleh\nPemegang Kad. Baki tersedia di dalam Akaun Pembelian Runcit untuk digunakan oleh Pemegang\nKad akan berkurangan sejajar dengan amaun yang dipegang.\n7.3\nTertakluk kepada Had Pembelian Runcit Harian dan Baki Semasa dalam Akaun Belian Runcit dan\nTerma & Syarat di sini, Bank hendaklah meletakkan Akaun Pegangan untuk pembelian petrol oleh\nPemegang Kad di dispenser bahan api automatik di stesen minyak di mana sistem akan secara\nautomatik memperuntukkan Ringgit Malaysia Dua Ratus (RM200). Amaun pra-kuasa tersebut\nakan dikeluarkan secara automatik sebaik sahaja bank pemeroleh menuntut nilai pembelian\nsebenar kepada bank dalam tempoh tiga (3) Hari Perniagaan atau tempoh lain yang mungkin\ndimaklumkan oleh Bank.\n7.4\nSelain daripada yang dinyatakan seperti Klausa 7.3 di atas, Bank mempunyai budi bicara untuk\nmenahan jumlah tersebut sehingga dua puluh satu (21) hari kalendar atau untuk tempoh yang\ndifikirkan sesuai. Bank hendaklah mendebitkan jumlah yang ditahan ke Akaun Belian Runcit\napabila Transaksi Kad yang sepadan dikemukakan kepada Bank untuk pembayaran. Bank\nhendaklah mengeluarkan jumlah yang ditahan jika Transaksi Kad yang sepadan tidak\ndikemukakan kepada Bank untuk pembayaran dalam tempoh yang dianggap sesuai oleh Bank.\nPemegang Kad selanjutnya dengan nyata bersetuju bahawa Bank berhak untuk menahan Akaun\nBelian Runcit dan mendebit Akaun Belian Runcit jika Transaksi Kad berkemungkinan akan atau\ndibentangkan untuk pembayaran kemudiannya oleh Pedagang Dibenarkan apabila tamat tempoh\ndua puluh satu satu (21) hari kalendar. Pemegang Kad bersetuju bahawa semua resit yang\ndikemukakan oleh Pedagang Sah untuk Urus Niaga Kad melalui Resit Transaksi termasuk resit\npengesahan dan pengakuan hendaklah dianggap sebagai bukti penggunaan muktamad dan\nkonklusif oleh Pemegang Kad.\n7.5\nApabila Bank menerima bukti jumlah transaksi sebenar daripada Peniaga Sah, Bank akan\nmelepaskan Jumlah Pra-Kuasa dan Akaun Pembelian Runcit Pemegang Kad akan didebitkan"
  },
  {
    "page": 13,
    "content": "dengan jumlah transaksi sebenar tersebut.\n7.6\nPemegang Kad tidak boleh memberhentikan pembayaran ke atas Transaksi Kad. Untuk\nmengelakkan keraguan, dengan ini dinyatakan bahawa hak Bank untuk mendebit Akaun\nPembelian Runcit berhubung sebarang Transaksi Kad tidak terhad kepada amaun yang diletakkan\ndalam pegangan yang berkaitan dengan Transaksi Kad dan Bank berhak untuk mendebit Akaun\nPembelian Runcit untuk amaun penuh atas Transaksi Kad. Bank juga berhak untuk menaikkan\namaun yang perlu dipegang berkaitan mana-mana Transaksi Kad yang berdenominasi dalam mata\nwang lain selain daripada Ringgit Malaysia jika Bank berpendapat amaun asal yang dipegang tidak\nmencukupi untuk membayar Transaksi Kad sepenuhnya apabila ditukar kepada mata wang asing\ntersebut.\n7.7\nPemegang Kad tidak boleh menggunakan jumlah yang ditahan, walau apa pun terma dan syarat\nlain yang mengatur Akaun Pembelian Runcit.\n7.8\nBank hanya akan mengkreditkan Akaun Pembelian Runcit dengan kembalian yang dibuat oleh\nPeniaga Sah atau penentuan yang berkaitan dengan mana-mana transaksi runcit/transaksi dalam\ntalian selepas Bank menerima baucer kredit yang dikeluarkan dengan betul. Kembalian ini\ntermasuk tetapi tidak terhad kepada Pemegang Kad memulangkan semula barangan kepada\nperuncit selepas pembelian.\n8.0\nPemilikan dan Kad Debit Hilang atau Dicuri\n8.1\nPemegang Kad harus mengambil segala langkah keselamatan untuk mengelakkan Kad Debit\ndaripada kehilangan atau kecurian dan Pemegang Kad tidak boleh meninggalkan Kad Debit tanpa\ndijaga atau mendedahkan PIN dan/atau butiran Kad Debit kepada mana-mana pihak ketiga.\nSekiranya berlaku kehilangan dan/atau kecurian Kad Debit dan/atau pendedahan PIN dan/atau\nbutiran kepada pihak yang tidak dibenarkan, Pemegang Kad apabila menyedarinya hendaklah\nmemaklumkan kepada Bank dengan secepat yang munasabah boleh dilaksanakan selepas itu\n(jika perkara tersebut berlaku di Malaysia) atau Visa Travel Service Centre atau mana-mana ahli\nMastercard atau sekutu terdekatnya (jika perkara tersebut berlaku di luar negara). Pemegang Kad\nmemahami sepenuhnya bahawa kegagalan untuk menjaga dan mengambil langkah keselamatan\nyang munasabah dalam penyimpanan Kad Debit boleh mendedahkan Pemegang Kad kepada\nrisiko kecurian dan/atau penggunaan tanpa kebenaran Kad Debit.\n8.2\nBank berhak bertindak atas arahan telefon yang diterima oleh Bank daripada Pemegang Kad\ndan/atau wakilnya yang diberikuasa oleh Pemegang Kad (jika berkenaan dan mengikut mana-\nmana yang berkenaan) (\"Orang yang Diberi Kuasa\") dengan niat yang baik dan mengikut operasi\nyang biasa. Berhubung dengan ini, Bank tidak akan bertanggungjawab untuk bertindak atas arahan\ntelefon yang diberikan oleh Orang yang Diberi Kuasa tersebut tidak kira keadaan yang wujud pada\nmasa arahan tersebut.\n8.3\nJika berlaku kecurian/kehilangan Kad Debit dan Kad Debit tersebut telah digunakan oleh orang\nyang tidak dibenarkan untuk sebarang Transaksi Kad, Pemegang Kad bersetuju bahawa beliau\nakan bertanggungjawab kepada Bank ke atas semua caj tanpa kebenaran yang ditanggung\ntermasuk caj kewangan untuk sebarang barangan atau perkhidmatan yang diberikan oleh Peniaga\nSah, akibat penggunaan Kad Debit oleh pihak yang tidak dibenarkan tersebut sehingga tarikh dan\nmasa penerimaan oleh Bank makluman dari Pemegang Kad yang dinyatakan di dalam Klausa 8.1\ndi atas dan tertakluk kepada terma dan syarat Klausa 8.6 di dalam ini.\n8.4\nPemegang Kad bersetuju untuk memberi kuasa kepada Bank supaya tidak memperakui Kad Debit"
  },
  {
    "page": 14,
    "content": "asal yang tersebut di atas sekiranya Kad Debit yang sama diserahkan kepada Bank untuk\npembayaran setelah penerimaan oleh Bank pemberitahuan Pemegang Kad yang dinyatakan di\nbawah Klausa 8.1 diatas.\n8.5\nSelepas pengeluaran penggantian Kad Debit asal atas permintaan Pemegang Kad, Pemegang Kad\nbersetuju untuk membayar balik dan membayar kepada Bank jika Kad Debit asal digunakan dalam\napa jua cara, yang mana Pemegang Kad bersetuju bahawa Bank boleh menolak atau mendebit\ndaripada Akaun Pemegang Kad untuk jumlah tersebut.\n8.6\nPemegang Kad tidak akan dipertanggungjawab ke atas urus niaga tanpa kebenaran kad-hadir\nyang memerlukan pengesahan PIN atau tandatangan yang telah disahkan atau penggunaan kad\ntanpa-sentuh, dengan syarat Pemegang Kad tidak:\n8.6.1 melakukan penipuan;\n8.6.2 tangguh dalam memaklum Bank secepat mungkin setelah diketahui kehilangan atau\npenggunaan tanpa kebenaran Kad Kredit;\n8.6.3 secara sukarela mendedahkan PIN kepada orang lain;\n8.6.4 mencatatkan PIN pada Kad Kredit atau pada apa-apa yang disimpan berdekatan dengan Kad;\n8.6.5 meninggalkan Kad Kredit atau apa-apa yang mengandungi Kad Kredit tanpa jagaan di mana-\nmana tempat yang boleh dilihat dan diakses oleh orang lain; atau\n8.6.6 secara sukarela membenarkan orang lain untuk menggunakan Kad.\nUntuk mengelakkan keraguan, transaksi tanpa kebenaran yang dinyatakan di atas bermaksud\ntransaksi pembelian runcit dan bukan transaksi ATM. Urus niaga ATM akan merangkumi kedua-\ndua pengeluaran dan transaksi wang antara bank yang dilakukan dengan Kad Debit. Adalah\ndipersetujui dengan nyata bahawa Bank tidak bertanggungjawab untuk memastikan Baki Semasa\nyang ditetapkan oleh Bank tidak melebihi.\n8.7\nBank boleh, atas permintaan Pemegang Kad, tetapi tanpa diwajibkan di sisi undang-undang,\nmengganti Kad Debit yang hilang, dicuri atau rosak dengan mengenakan bayaran fi. Kad Debit\ngentian adalah tertakluk kepada terma dan syarat di dalam T&S ini sebagaimana Kad Debit asal.\n8.8\nSetelah Kad Debit tamat tempoh atau apabila diminta oleh Bank selepas pembatalan, pemansuhan\natau penggantungan Kad Debit oleh Bank atau apabila diketahui Kad Debit selepas pemberitahuan\nkehilangannya, Pemegang Kad tidak mempunyai apa-apa hak lagi penggunaan Kad Debit dan\nakan segera dipotong setengah dan memulangkan Kad Debit tersebut kepada Bank.\n9.0\nPenamatan oleh Pemegang Kad\n9.1\nPemegang Kad boleh pada bila-bila masa, menamatkan penggunaan Kad Debit dengan\nmemberikan makluman bertulis kepada Bank dan mengembalikan Kad Debit yang dipotong dua\nkepada Bank. Tiada kembalian Fi atau sebahagiannya akan dibuat kepada Pemegang Kad dan\nPemegang Kad akan dan kekal bertanggungjawab ke atas sebarang transaksi yang dilakukan\nmenggunakan Kad Debit sebelum Bank menerima makluman bertulis mengenai penamatan dan\npengembalian Kad Debit dipotong dua kepada Bank.\n10.0\nFi, Komisyen dan Caj\n10.1\nBank berhak untuk mengenakan Fi untuk Perkhidmatan termasuk tetapi tidak terhad kepada\npengeluaran Kad Debit, Penyata bulanan dicetak, Yuran Penggantian Kad, Fi Pengeluaran Tunai\nmelalui rangkaian MEPS, terminal Visa PLUS/Mastercard Cirrus dan Bank Antarabangsa, Dapatan\nSemula Draf Jualan dan Fi Bayaran Tahunan. Selain daripada Bayaran Fi tersebut, transaksi yang\ndijalankan melalui Perkhidmatan akan dikenakan caj dan komisen biasa. Bagi tujuan pengutipan\nYuran, komisen dan/atau caj, Pemegang Kad bersetuju untuk memberi kuasa kepada Bank untuk"
  },
  {
    "page": 15,
    "content": "mendebitkan Akaun dengan Yuran, komisyen dan/atau caj yang berkenaan. Sekiranya Pemegang\nKad tidak memberi kuasa kepada Bank untuk mendebitkan Akaun dengan Yuran, komisen\ndan/atau caj, Pemegang Kad bersetuju bahawa kegagalan pembayaran Yuran, komisen dan/atau\ncaj bagi Perkhidmatan boleh menjejaskan Perkhidmatan yang disediakan sehingga semua yuran,\nkomisen dan/atau caj yang telah dibayar dengan sewajarnya oleh Pemegang Kad.\n10.2\nPemegang Kad bersetuju untuk membayar semua fi, komisen dan/atau caj yang dikenakan dalam\nklausa ini dan membenarkan Bank untuk mendebit Akaun Pemegang Kad, tanpa mengambilkira\npendebitan tersebut mungkin menyebabkan Akaun terlebihguna. Bayaran, komisen dan/atau caj\nberikut dikenakan pada kadar yang dinyatakan atau kadar lain yang ditetapkan, yang boleh dipinda\noleh Bank untuk membuat pemindahan dari semasa ke semasa dengan memberi dua puluh satu\n(21) hari kalendar notis terlebih dahulu kepada Pemegang Kad untuk transaksi yang dilaksanakan\nmelalui penggunaan Kad Debit. Untuk senarai fi dan caj yang lengkap, sila layari laman web kami\nwww.hlb.com.my/dc2 atau imbas di sini:\n10.3\nFi Tahunan tidak akan dikenakan semasa pengeluaran Kad Debit dan hanya akan dikenakan pada\ntarikh ulang tahun. Fi Tahunan boleh diubah oleh Bank dari semasa ke semasa. Fi Tahunan tidak\nakan dikembalikan.\n11.0\nCukai Kerajaan, Fi dan Caj Berkanun dan/atau Kawal Selia\n11.1\nSebarang yuran perkhidmatan dan sebarang pembayaran yang kena dibuat oleh Pemegang Kad\nkepada Bank dibawah T&S ini, termasuk sebarang jumlah yang mewakili pembayaran ganti untuk\ndibayar oleh Pemegang Kad kepada Bank, adalah tidak termasuk Cukai, dan perlu dijelaskan tanpa\nsebarang pengecualian, syarat dan tanpa sebarang potongan bagi sebarang jumlah tuntutan balas atau\nsebarang potongan atau sekatan.\n11.2\nPada sebarang masa atau keadaan dimana Pemegang Kad dituntut oleh undang-undang untuk\nmembuat sebarang potongan atau menahan dari yuran perkhidmatan dan/atau semua wang lain\nyang kena dibayar kepada di bawah T&S ini berkaitan dengan Cukai atau sebaliknya, jumlah yang\nkena dibayar oleh Pemegang Kad dalam hal di mana potongan atau penahanan cukai diperlukan\nakan ditambah supaya yuran perkhidmatan dan/atau jumlah bersih wang yang diterima oleh Bank\nkekal dengan apa yang Bank sepatutnya menerima jika sebarang potong atau pengecualian tidak\ndikenakan.\n11.3\nSebagai penambahan atas yuran perkhidmatan dan sebarang bayaran yang perlu dibayar,\nPemegang Kad kena bayar kepada Bank semua bayaran Cukai pada kadar terkini dan/atau jumlah\nyang\nditentukan\noleh\nBank\nuntuk\nmerangkumi\nsebarang\npembayaran\nCukai/liabilti\nCukai/kewajipan-kewajipan Cukai yang berhubung kait, tanpa sebarang penolakan, sekatan atau\nsyarat-syarat dan tanpa apa-apa potongan untuk atau sebarang jumlah tuntutan balas atau apa-\napa potongan atau pengecualian, selain sebarang Cukai-Cukai yang mana mungkin diperlukan\ndi bawah sebarang undang-undang kena dibayar oleh anda secara terus kepada Pihak Berkuasa\nYang Berkenaan, dimana anda akan membayar secara terus kepada Pihak Berkuasa Yang\nBerkenaan."
  },
  {
    "page": 16,
    "content": "11.4\nJika pada bila-bila masa satu pelarasan dibuat atau dikehendaki untuk dibuat antara Bank\ndan Pihak Berkuasa Berkenaan bagi sebarang jumlah Cukai berkaitan sebarang bekalan yang\ndilakukan atau dinilai sebagai telah berlaku, yang berkaitan dengan T&S ini daripada Bank atau\nsegala yang berkaitan dengannya, satu pelarasan, tertakluk pada budi bicara, boleh dibuat diantara\nBank and Pemegang Kad dan pada keadaan sedemikian, sebarang pembayaran berkaitan\npelarasan tersebut, haruslah dijelaskan sepenuhnya.\n11.5\nSemua Cukai yang mesti dibayar oleh Pelanggan kepada Bank seperti yang diperuntukkan di ini\nadalah perlu dibayar di masa-masa sedemikian dan di sedemikian cara seperti yang akan diminta\noleh Bank.\n11.6\nPemegang Kad dengan ini bersetuju untuk memenuhi segala permintaan oleh Bank bagi\nmembantu Bank dalam mematuhi dengan kewajipan-kewajipannya di bawah undang-undang\npercukaian yang berkaitan. Jika terdapat, Cukai baru yang telah berkuatkuasa dan Cukai tersebut\nadalah diwajibkan untuk dikenakan terhadap sebarang urus niaga yang termaktub di dalam T&S\nini, Pemegang Kad bersetuju untuk memberi kerjasama dengan sepenuhnya kepada Bank untuk\nmemudahkan Bank untuk mematuhi tanggungjawab Bank di bawah undang-undang tersebut.\n11.7\nBagi mengelakkan sebarang keraguan, kesemua pihak terbabit dengan ini bersetuju bahawa\nmana-mana jumlah yang perlu dibayar atau jumlah yang digunakan untuk menentukan jumlah yang\nperlu dibayar berdasarkan Terma dan Syarat ini adalah muktamad tanpa mengambil kira untuk\nCukai dan tidak termasuk jumlah-jumlah yang perlu ditambah di bawah Klausa ini bekenaan\ndengan Cukai.\n12.0\nHad Ke Atas Transaksi Kad\n12.1\nKecuali diarahkan sebaliknya oleh Pemegang Kad atau Pemegang Kad tidak memilih suatu had\nuntuk pelaksanaan transaksi melalui Perkhidmatan, Bank berhak untuk menentukan dan\nmenetapkan sebarang had sama ada dari segi amaun, kekerapan atau kegunaan mana-mana\nPerkhidmatan dan/atau Kod Keselamatan. Pemegang Kad boleh mengubah had ke atas Transaksi\nKad yang dilakukan dengan memberikan makluman awal secara bertulis kepada Bank. Bank tidak\nbertanggungjawab untuk memastikan Pemegang Kad tidak melebihi had yang ditetapkan\nmahupun mematuhi permintaan Pemegang Kad supaya Bank mengubah had tersebut.\n12.2\nHad Pengeluaran Tunai Harian untuk Pemegang Kad Biasa dan Perbankan Prioriti ditetapkan pada\nRinggit Malaysia Dua Ribu (RM2,000), dengan had maksimum dibenarkan sebanyak Ringgit\nMalaysia Sepuluh Ribu (RM10,000) untuk Pemegang Kad Biasa dan Pemegang Kad Perbankan\nPrioriti, atau suatu amaun lain yang ditentukan oleh Bank dari semasa ke semasa dengan memberi\ndua puluh satu (21) hari kalendar terlebih dahulu makluman awal kepada Pemegang Kad.\nPemegang Kad boleh melakukan penetapan Had Pengeluaran Tunai Harian di mana-mana\ncawangan Bank atau ATM atau melalui Hong Leong Connect.\n12.3\nHad Belian Runcit Harian untuk Pemegang Kad Biasa dan Perbankan Prioriti ditetapkan pada\nRinggit Malaysia Tiga Ribu (RM3,000), dengan had maksimum dibenarkan sebanyak Ringgit\nMalaysia Dua Puluh Ribu (RM20,000) untuk Pemegang Kad Biasa dan Pemegang Kad Perbankan\nPrioriti, atau suatu amaun lain yang ditentukan oleh Bank dari semasa ke semasa dengan\nmemberikan notis awal dua puluh satu (21) hari kepada Pemegang Kad. Pemegang Kad boleh\nmelakukan penetapan Had Belian Runcit Harian di mana-mana cawangan Bank atau melalui Hong\nLeong Connect. Selain daripada itu, Pemegang Kad boleh memohon Had Belian Runcit Harian\nyang lebih tinggi (\u201cHad Belian Runcit Harian Sementara\u201d) melalui Hong Leong Connect dan Had\nBelian Runcit Harian Sementara tersebut akan berkuat kuasa sepanjang tempoh masa tertentu"
  },
  {
    "page": 17,
    "content": "yang dipilih oleh Pemegang Kad (\u201cJulat Tarikh\u201d). Had Belian Runcit Harian Sementara\nmemberikan Pemegang Kad had maksimum yang dibenarkan iaitu Ringgit Malaysia Sembilan\nPuluh Sembilan Ribu Sembilan Ratus Sembilan Puluh Sembilan (RM99,999) semasa Julat Tarikh.\nHad Belian Runcit Harian Sementara akan ditamatkan apabila Julat Tarikh berakhir dan selepas\nitu, Had Belian Runcit akan kembali kepada Had Belian Runcit Harian.\n12.4\nHad Belian Dalam Talian Harian untuk Pemegang Kad Biasa dan Pemegang Kad Perbankan\nPrioriti ditetapkan pada Ringgit Malaysia Satu Ribu (RM1,000), dengan had maksimum dibenarkan\nsebanyak Ringgit Malaysia Dua Puluh Ribu (RM20,000) untuk Pemegang Kad Biasa dan\nPemegang Kad Perbankan Prioriti, atau suatu amaun lain yang ditentukan oleh Bank dari semasa\nke semasa dengan memberikan notis awal dua puluh satu (21) hari kalendar kepada Pemegang\nKad. Pemegang Kad boleh melakukan Had Belian Dalam Talian Harian di mana-mana cawangan\nBank atau melalui Hong Leong Connect. Selain daripada itu, berkuat kuasa 24 September 2022,\nPemegang Kad boleh memohon Had Belian Dalam Talian Harian yang lebih tinggi (\u201cHad Belian\nDalam Talian Harian Sementara\u201d) melalui Hong Leong Connect dan Had Belian Dalam Talian\nSeharian Sementara tersebut akan berkuat kuasa sepanjang tempoh masa tertentu yang dipilih\noleh Pemegang Kad (\u201cJulat Tarikh\u201d). Had Belian Dalam Talian Harian Sementara memberikan\nPemegang Kad had maksimum yang dibenarkan iaitu Ringgit Malaysia Tiga Puluh Ribu\n(RM30,000) semasa Julat Tarikh. Had Belian Dalam Talian Harian Sementara akan ditamatkan\napabila Julat Tarikh berakhir dan selepas itu, Had Belian Runcit akan Kembali kepada\nHad BelianDalam Talian Harian.\n12.5 Had Pemindahan Harian melalui ATM untuk Pemegang Kad Biasa dan Perbankan Prioriti\nditetapkan pada Ringgit Malaysia Dua Ribu (RM2,000), dengan had maksimum dibenarkan\nsebanyak Ringgit Malaysia Lima Puluh Ribu (RM50,000) untuk Pemegang Kad Biasa dan\nPerbankan Prioriti, atau suatu amaun lain yang ditentukan oleh Bank dari semasa ke semasa\ndengan memberikan makluman awal dua puluh satu (21) hari kalendar terlebih dahulu kepada\nPemegang Kad. Pemegang Kad boleh melakukan penetapan Had Pemindahan Harian di mana-\nmana cawangan Bank atau ATM daripada Bank.\n12.6 Bagi mengelakkan keraguan, Had Pemindahan Harian melalui ATM adalah berasingkan daripada\nHad Pemindahan Harian melalui Hong Leong Connect.\n12.7 Pemegang Kad yang ingin menukar had amaran transaksi melalui Paparan Notifikasi Aplikasi HLB\nConnect atau khidmat pesanan ringkas (\u201cSMS\u201d) dikehendaki membuat permintaan bertulis untuk\nmenukar had yang telah ditetapkan sebelum ini atau menarik diri daripada perkhidmatan amaran\ntransaksi melalui Paparan Notifikasi Aplikasi HLB Connect atau SMS tersebut. Pemegang Kad\nboleh mendapatkan Borang Penyelenggaraan Perkhidmatan Amaran Transaksi Kad Debit/Kad\nDebit-i (\u201cBorang\u201d) di mana-mana cawangan Bank atau di Laman Sesawang Bank di\nwww.hlb.com.my. Pemegang Kad perlu melengkapkan Borang tesebut dan mematuhi arahan yang\nterdapat di dalamnya untuk menukar had amaran transaksi melalui Paparan Notifikasi Aplikasi HLB\nConnect atau SMS atau menarik diri daripada perkhidmatan amaran transaksi melalui Paparan\nNotifikasi Aplikasi HLB Connect atau SMS tersebut.\n13.0\nPematuhan Syariah\n13.1\nApabila Akaun Pembelian Runcit yang dipautkan dengan Kad Debit adalah Akaun Simpanan atau\nAkaun Semasa-i, Kad Debit hanya boleh digunakan untuk pembelian barang dan perkhidmatan\nyang mematuhi Syariah. Kad Debit tidak boleh digunakan di mana-mana peniaga runcit yang\nmenyediakan barang dan perkhidmatan yang tidak mematuhi Syariah dan/atau untuk apa-apa"
  },
  {
    "page": 18,
    "content": "transaksi yang tidak mematuhi Syariah yang dikategorikan mengikut Kategori peniaga seperti yang\nberikut:\n(i) Bar, Ruang Koktel, Disko, Kelab Malam dan Kedai Minuman Keras\n(ii) Pakej Bir, Wain dan Minuman Keras\n(iii) Kedai dan gerai cerut\n(iv) Tranksaksi Perjudian\n(v) Petaruhan-Perlumbaan Kuda, Perlumbaan Anjing, Petaruhan Bukan Sukan Dalam Talian\nAntara Negeri\n(vi) Perkhidmatan Janji Temu dan Teman Social\nNota: Senarai peniaga runcit dan aktiviti tidak mematuhi Syariah di atas akan dikemas kini dari\nsemasa ke semasa.\n14.0\nKewajipan Pemegang Kad\n14.1\nKecuali jika kerugian tersebut disebabkan oleh salah laku sengaja atau kecuaian melampau oleh\nBank, Pemegang Kad dengan ini bersetuju dan wajib untuk membayar balik kepada Bank atas\npermintaan bertulis Bank semua tuntutan, kerugian dan perbelanjaan (termasuk kos perundangan\nsebagaimana peguam cara dengan pelanggan sendiri) dan semua liabiliti dalam apa jua bentuk atau\njenis yang ditanggung atau dialami oleh Bank berhubung dengan atau apa jua cara yang timbul\ndaripada peruntukan Perkhidmatan atau penerimaan sebarang arahan yang diberikan oleh\nPemegang Kad atau pelanggaran mana-mana Terma dan Syarat ini oleh Pemegang Kad. Liabiliti\nPemegang Kad adalah merupakan liabiliti berterusan dan akan kekal berkuat kuasa sepenuhnya\nsehingga liabiliti Bank, jika ada, dilepaskan sepenuhnya.\n15.0\nPenggangtungan, Pembatalan dan Penamatan\n15.1\nBank berhak menarik atau menggantung mengikut budi bicaranya Kad Debit dan/atau mana-mana\nPerkhidmatan dan dalam keadaan seperti itu jika Bank dianggap sesuai untuk menghentikan\npenggunaan Kad Debit oleh Pemegang Kad dengan pemberitahuan bertulis kepada Pemegang\nKad. Selanjutnya disepakati bahawa Bank tidak berkewajiban untuk menyatakan alasan\npenamatan penggunaan Kad Debit.\n16.0\nBudi Bicara Bank\n16.1\nBank berhak pada bila-bila masa menurut budi bicaranya tidak akan meluluskan sebarang\nTransaksi Kad walaupun Baki Semasa di dalam Akaun Pemegang Kad membolehkan mana-mana\nTransaksi Kad (iaitu terdapat dana yang mencukupi di dalam Akaun).\n16.2\nBank berhak untuk:\n(a) Menggantung hak Pemegang Kad untuk menggunakan Kad Debit sepenuhnya atau\nberkenaan keistimewaan tertentu.\n(b) Menolak untuk menerbit-semula, memperbaharui atau mengganti Kad Debit, tanpa dalam apa\njua perkara, menjejaskan tanggungjawab Pemegang Kad di bawah T&S ini yang akan terus\nberkuatkuasa dan tiada pengembalian akan dibuat untuk sebarang fi tahunan atau fi lain yang\ntelah dibayar jika hak untuk menggunakan Kad Debit digantung oleh Bank atau jika Kad Debit\ntidak diperbaharui atau diganti.\n16.3\nTanpa prejudis kepada peruntukan lain di dalam T&S ini, Bank berhak pada bila-bila masa dengan\nmemberi dua puluh satu (21) hari kalendar terlebih dahulu makluman awal, untuk\nmemperkenalkan, menukar, mengubah, mengehad atau menarik balik semua atau mana-mana"
  },
  {
    "page": 19,
    "content": "manfaat, perkhidmatan, kemudahan atau kelebihan yang berkaitan dengan atau berhubung\ndengan penggunaan Kad Debit dan/atau Terma dan Syarat ini.\n17.0\nAkaun Gantung\n17.1\nUntuk tujuan membolehkan Bank mengekalkan keseluruhan liabiliti Pemegang Kad, apabila writ\nsaman telah dikeluarkan atau untuk membuktikan kebankrapan atau ketakmampuan bayar\nPemegang Kad atau untuk tujuan lain yang dianggap wajar oleh Bank, Bank boleh pada bila-bila\nmasa dan tempat, menyimpan untuk suatu tempoh yang wajar, sebarang wang yang diterima,\ndiperoleh semula atau direalisasi selaras dengan T&S ini atau di bawah mana-mana cagaran atau\njaminan untuk dikreditkan dalam akaun berasingan yang tidak menawarkan keuntungan di bawah\nnama Pemegang Kad sebagaimana dianggap wajar oleh Bank. Bank tidak mempunyai kewajipan\nsegera untuk memanfaatkan wang tersebut atau sebahagian daripadanya untuk atau ke arah\nmelunaskan jumlah perlu bayar dan terhutang kepada Bank.\n18.0\nTolak Selesai\n18.1\nPemegang Kad bersetuju bahawa Bank boleh, pada bila-bila masa dan dengan memberikan\nmakluman awal, menyatu dan menggabung semua atau mana-mana Akaun Pemegang Kad\ndengan Bank dalam apa jua bentuk dan di mana-mana lokasi, sama ada di dalam Ringgit Malaysia\natau mata wang lain, atau menolak selesai atau memindah sebarang jumlah yang sedia untuk\ndikredit ke dalam mana-mana Akaun tersebut termasuk akaun bersama di mana akaun bersama\ntersebut adalah di atas nama Pemegang Kad untuk atau ke arah melunaskan semua jumlah\nterhutang kepada Bank di bawah mana-mana akaun Pemegang Kad dengan Bank, dalam apa jua\nbentuk dan di mana-mana lokasi, sama ada di dalam Ringgit Malaysia atau mata wang lain dan\nBank boleh berbuat demikian tanpa mengambil kira baki di dalam akaun tersebut dan jumlah\nterhutang mungkin bukan di dalam mata wang yang sama, dan Pemegang Kad dengan ini\nmembenarkan Bank menguatkuasakan sebarang penyatuan, gabungan, tolak selesai atau\npemindahan dengan pertukaran mata wang yang sewajarnya pada kadar tukaran semasa Bank\nyang akan ditentukan oleh Bank mengikut budi bicaranya.\n19.0\nCaj-Balik\n19.1\nBank tidak boleh dipertanggungjawabkan atas sebarang tindakan atau pengabaian oleh mana-\nmana peniaga runcit atau dalam talian walau apapun puncanya, termasuk tetapi tidak terhad\nkepada keengganan menerima Kad Debit atau sebarang Penyata atau komunikasi lain yang dibuat\nberkaitan dengannya, atau sebarang kecacatan atau kekurangan pada produk atau perkhidmatan.\nSekiranya terdapat sebarang kecacatan atau kekurangan pada produk atau perkhidmatan,\nPemegang Kad hendaklah berhubung secara langsung dengan Peniaga Sah mengenai sebarang\ntuntutan atau pertikaian dan tidak akan mempertikaikan sebarang bayaran yang dibuat oleh Bank\ndari Akaun Pemegang Kad ke atas tuntutan atau pertikaian tersebut.\n19.2\nDalam keadaan tertentu, peraturan skim kad membenarkan Pemegang Kad untuk mencaj-balik\nTransaksi Kad kepada Peniaga Sah bagi transaksi yang telah dibuat (jika hak caj-balik disediakan)\ntertakluk kepada aduan atau pertikaian telah dibuat oleh Pemegang Kad kepada Bank secara lisan\natau bertulis dalam masa dua puluh satu (21) hari kalendar daripada Tarikh Pengeposan\nTransaksi Kad tersebut seperti dalam Penyata dan Pemegang Kad telah menyediakan maklumat\nyang diperlukan oleh Bank (iaitu: nama Pemegang Kad, Akaun yang terlibat, tarikh dan jumlah Kad\ntransaksi yang dipertikaian dan sebab Pemegang Kad percaya bahawa ia adalah Transaksi Kad\nyang dipertikaikan) untuk menyokong caj balik dalam tempoh masa yang ditetapkan."
  },
  {
    "page": 20,
    "content": "19.3\nSekiranya Bank melanjutkan tempoh masa untuk menyelesaikan siasatan selepas empat belas\n(14) hari kalendar dari tarikh laporan terhadap Transaksi Kad yang dipertikaikan, sama ada secara\nlisan atau bertulis, oleh Pemegang Kad kepada Bank, Bank boleh: sekurang-kurangnya,\nsementara mengkreditkan jumlah penuh transaksi yang dipertikaikan atau Ringgit Malaysia Lima\nRibu (RM5,000), yang mana lebih rendah (termasuk apa-apa keuntungan yang berkenaan), ke\ndalam Akaun Pembelian Runcit tidak lewat daripada empat belas (14) hari kalendar dari tarikh\nPemegang Kad mengemukakan maklumat yang dinyatakandalam Klausa 19.2 kepada Bank;\n(a) mengkredit baki Transaksi Kad yang dipertikaikan (termasuk apa-apa keuntungan yang\nberkenaan) tidak lewat daripada tiga puluh (30) hari kalendar daripada tarikh perkreditan\npertama dana sementara ke dalam Akaun Pembelian Runcit oleh Bank mengikut Klausa 19.3\n(a) di mana dana tersebut adalah kurang daripada amaun Transaksi Kad yang dipertikaikan;\ndan\n(b) membenarkan Pemegang Kad menggunakan sepenuhnya dana sementara yang dikreditkan.\n19.4\nSekiranya Pemegang Kad dikehendaki membayar jumlah transaksi atau Peniaga Sah telah\nmengembalikan jumlah transkasi tersebut secara langsung kepada Pemegang Kad, Bank berhak\nmendebitkan amaun berkenaan daripada Akaun Pembelian Runcit.\n19.5\nTanpa mengambilkira dan tanpa prejudis kepada sebutan umum Klausa di dalam T&S ini,\nPemegang Kad bersetuju bahawa:\n(a) penggunaan Kad Debit adalah atas risiko Pemegang Kad sendiri dan Pemegang Kad harus\nmenanggung semua risiko yang berkaitan atau yang timbul daripada penggunaan Kad Debit;\ndan\n(b) Pemegang Kad tidak boleh membuat atau cuba untuk membuat apa-apa dakwaan palsu,\npenipuan atau menyalahi undang-undang berkenaan dengan Transaksi Kad yang\ndipertikaikan sama ada secara langsung atau tidak langsung. Bank berhak pada bila-bila masa\nuntuk memulakan dan memulakan dan mengambil tindakan undang-undang dan/atau apa-apa\nprosiding lain sebagaimana yang Bank fikirkan perlu terhadap Pemegang Kad setelah Bank\nmenemui atau menyedari tentang apa-apa tuntutan yang palsu atau penipuan atau menyalahi\nundang-undang berkaitan dengan atau yang timbul daripada Transaksi Kad yang dipertikaikan.\n19.6\nBank mempunyai hak untuk mengambil tindakan perundangan dan/atau apa-apa prosiding lain\nyang dianggap perlu oleh Bank termasuk melaporkan apa-apa laporan yang sesuai atau perlu\ndengan pihak berkuasa yang berkaitan, terhadap Pemegang Kad yang melengahkan, menghalang\ndan/atau menyembunyikan maklumat penting daripada Bank, membuat atau cuba untuk membuat\ntuntutan palsu berkenaan dengan apa-apa transaksi, menyiarkan tuntutan palsu di media\ntradisional atau sosial, dan/atau membuat laporan polis palsu mengenai apa-apa transaksi.\n19.7\nTertakluk kepada kepatuhan Pemegang Kad kepada tanggungjawab yang dinyatakan dalam\nKlausa 19.2, sekiranya berlaku caj balik akibat aduan atau pertikaian oleh Pemegang Kad\nmengenai Transaksi Luar Negara (seperti yang ditakrifkan di bawah Klausa 19.1 di bawah) yang\ndilakukan dalam mata wang asing melalui Akaun Yang Diupayakan Dengan MCF, amaun yang\ndicaj balik akan dikreditkan ke dalam Akaun Yang Diupayakan Dengan MCF Pemegang Kad dalam\nmata wang transaksi asal.\n20.0\nTransaksi Luar Negara\n20.1\nPemegang Kad boleh menggunakan Kad Debit untuk melakukan Transaksi Kad dan Transaksi"
  },
  {
    "page": 21,
    "content": "Kad ATM di luar Malaysia (\u201cTransaksi Luar Negara\u201d) di mana terdapat Peniaga Sah dan/atau\nSaluaran Tunai yang dibenarkan dengan syarat bahawa Pemegang Kad telah memilih untuk\nmembenarkan Transaksi Luar Negara yang akan dilaksanakan pada Kad Debit yang berkaitan\nKlausa 20.1 di dalam ini.\n20.2\nSemua Transaksi Luar Negara melalui transaksi ATM dan Transaksi Kad adalah tertakluk kepada\nundang-undang sedia ada di negara di mana transaksi dilakukan. Untuk semua Transaksi Kad,\nkadar tukaran, di mana berkaitan, adalah kadar tukaran serta-merta semasa pada tarikh transaksi.\n20.3\nPemegang Kad boleh menggunakan Kad Debit untuk pengeluaran tunai melalui ATM tertentu yang\ndipasang di negara yang diluluskan sebagaimana ditetapkan oleh Bank dan/atau mana-mana\nVisa/Mastercard tertakluk bahawa semua caj yang dikenakan oleh Visa International/MasterCard\nInternational dan/atau Bank termasuk semua caj yang dinyatakan di dalam Klausa 20.5 dan 20.6.\n20.4\nSekiranya Pemegang Kad menggunakan Kad Debit di ATM dan outlet Peniaga di bawah rangkaian\nVisa/Mastercard di luar Malaysia, transaksi akan dicaj menggunakan matawang rasmi negara\ntersebut dan ditukar ke Ringgit Malaysia pada kadar tukaran yang mungkin ditentukan oleh Visa\n/Mastercard.\n20.5\nTransaksi Kad yang dibuat dalam mata wang selain dari Ringgit Malaysia akan ditukarkan kepada\nRinggit Malaysia pada tarikh ia diproses oleh Visa atau Mastercard pada kadar pertukaran asing\nsebagaimana yang ditentukan oleh Visa atau Mastercard. Fi tambahan berikut akan dikenakan ke\natas penukaran:\n(i) 1% fi transaksi dikenakan oleh Visa atau Mastercard; dan\n(ii) 1% penambahan kepada penukaran tukaran asing dikenakan oleh Bank\n20.6\nUntuk Transaksi Luar Negara tertentu, Pemegang Kad boleh memilih untuk menggunakan\nperkhidmatan DCC untuk menukarkan transaksi mata wang asing kepada Ringgit Malaysia. Jika\nDCC dipilih:\n(i) Kadar pertukaran asing yang digunakan oleh peniaga luar negara bagi urusniaga DCCmungkin\nlebih tinggi daripada kadar pertukaran yang ditentukan oleh Visa atau Mastercard.\n(ii) Pemegang Kad akan dikenakan fi transaksi sebanyak 1% oleh Visa atau Mastercard ke atas\namaun Ringgit Malaysia yang ditukarkan, yang merupakan fi yang dikenakan oleh Visa atau\nMastercard.\n20.7\nSekiranya Pemegang Kad memilih untuk melakukan transaksi dalam mata wang asing\nmenggunakan Akaun Yang Diupayakan Dengan MCF, transaksi tersebut akan berjaya sekiranya\nmata wang asing yang dipilih mempunyai baki yang mencukupi di dalam Akaun Yang Diupayakan\nDengan MCF. Transaksi tersebut tidak akan dikenakan fi yang dinyatakan dalam Klausa 20.5.\nWalau bagaimanapun, sekiranya mata wang asing yang dipilih oleh Pemegang Kad tidak\nmencukupi, transaksi tersebut akan disempurnakan dalam Ringgit Malaysia dan akan didebit dari\nbaki Ringgit Malaysia yang terdapat dalam Akaun Yang Diupayakan Dengan MCF Pemegang Kad\ndan fi yang dinyatakan dalam Klausa 20.5 akan dikenakan.\n20.8\nMeskipun\nadanya\nKlausa\n20.7,\nsekiranya\ninstitusi\nkewangan\npeniaga\nluar\nnegara\nmenyempurnakan / membenarkan Transaksi Luar Negara dalam Ringgit Malaysia, bukannya\ndalam mata wang asing yang dipilih oleh Pemegang Kad, transaksi tersebut akan didebit dari"
  },
  {
    "page": 22,
    "content": "Akaun Pembelian Runcit Pemegang Kad dalam Ringgit Malaysia walaupun Pemegang\nKadmempunyai baki mata wang asing yang mencukupi dalam Akaun Yang Diupayakan\nDengan MCF\n20.9\nBagi Transaksi Kad yang disempurnakan dari Akaun Yang Diupayakan Dengan MCF dalam mata\nwang asing:\n(i) sekiranya terdapat kembalian oleh Peniaga Sah dalam tempoh enam puluh (60) hari daripada\ntarikh transaksi, dan Bank berjaya memadankan kembalian tersebut kepada transaksi asal,\nkembalian tersebut akan dikreditkan ke dalam Akaun Yang Diupayakan Dengan MCF dalam\nmata wang transaksi asal. Sekiranya padanan tidak berjaya, kembalian tersebut akan\nditukarkan kepada Ringgit Malaysia pada kadar tukaran semasa Bank pada masa Bank\nmengkredit kembalian tersebut ke dalam Akaun Yang Diupayakan Dengan MCF Pemegang\nKad.\n(ii)\nsekiranya Peniaga Sah mengemukakan permintaan untuk kembalian selepas tempoh enam\npuluh (60) hari daripada tarikh transaksi, kembalian kepada Pemegang Kad akan\ndisempurnakan dalam Ringgit Malaysia and tukaran daripada mata wang yang berkenaan\nkepada Ringgit Malaysia adalah pada kadar tukaran semasa Bank pada masa Bank\nmengkredit kembalian tersebut ke dalam Akaun Yang Diupayakan Dengan MCF Pemegang\nKad.\n20.10\nPemegang Kad perlu mematuhi Notis FE yang dikeluarkan oleh BNM berkenaan dengan sebarang\ntransaksi, termasuk Transaksi Luar Negara, yang boleh didapati di laman sesawang BNM.\nPemegang Kad adalah bertanggungjawab atas sebarang pengingkaran atau ketidakpatuhan\nperaturan tersebut dan pindaan yang dibuat terhadapnya.\n21.0\nKeperluan Pilih Masuk bagi Transaksi Luar Negara dan Transaksi Tanpa Kad\n21.1\nPemegang Kad tidak dibenarkan untuk melaksanakan apa-apa Transaksi Luar Negara dan/atau\nTransaksi Tanpa Kad melainkan Pemegang Kad telah mendaftarkan arahannya untuk memilih\nmasuk untuk melaksanakan Transaksi Luar Negara dan/atau Transaksi Tanpa Kad melalui Hong\nLeong Connect, ATM Bank, kunjungi mana-mana cawangan Bank atau hubungi Pusat Panggilan\nBank di 03-7626 8899, jika tidak, Transaksi Luar Negara dan/atau Transaksi Tanpa Kad akan\nditolak.\n22.0\nPlatform Pembayaran atau Rangkaian\n22.1\nWalaupun Kad Debit Bank menyokong MyDebit yang hanya boleh digunakan di dalam Malaysia,\ndan Visa/Mastercard yang diterima di dalam Malaysia dan di luar negara, apabila Kad Debit\ndigunakan di Peniaga Sah di Malaysia, ia adalah tertakluk kepada pilihan rangkaian oleh Peniaga\nSah tersebut (iaitu sama ada MyDebit atau Visa/Mastercard) bagi tujuan memproses pembayaran.\n22.2\nPemegang Kad tidak dibenarkan untuk meminta Peniaga Sah yang dibenarkan untuk menukar\nrangkaian Kad Debit yang telah dipilih.\n23.0\nPengganti\n23.1\nT&S ini akan digunakan pakai untuk waris, wakil peribadi dan pengganti\nPemegang Kad dan Bank masing-masing."
  },
  {
    "page": 23,
    "content": "24.0\nUndang-undang dan Bidang Kuasa yang Terpakai\n24.1\nT&S ini akan tertakluk kepada dan ditafsirkan menurut undang-undang Malaysia tetapi dalam\nmenguatkuasakan haknya, Bank bebas untuk memulakan dan mengambil tindakan atau prosiding\natau selainnya terhadap Pemegang Kad dan mana-mana pihak di Malaysia dan/atau di tempat lain\nseperti yang dianggap sesuai oleh Bank dan Pemegang Kad bersetuju bahawa apabila apa-apa\ntindakan atau prosiding dimulakan dan diambil di Malaysia, ia hendaklah mematuhi bidang kuasa\neksklusif Mahkamah Malaysia dalam semua hal yang berkaitan dengan kewajipan dan liabiliti di\nbawah atau yang timbul daripada Akaun dan T&S ini.\nJika anda mempunyai sebarang pertanyaan mengenai T&S ini, anda boleh mendapatkan penjelasan\ndaripada kakitangan kami yang membantu anda. Sebagai alternatif, sila e-mel kepada kami di\nhlonine@hlbb.hongleong.com.my atau hubungi 03-7626 8899\n[TAMAT]"
  }
]
    
    # Test cases
    test_paragraphs = [
      """1.36 “Akaun Diaktifkan MCF” bermaksud Akaun Pemegang Kad di mana MCF (seperti yang ditakrifkan dalam Klausa 1.45 di sini) telah diupayakan, yang mungkin termasuk akaun yang dipautkanke Kad Debit. telah diaktifkan, yang mungkin termasuk Akaun Pembelian Runcit.
      1.37 “MEPS” bermaksud Malaysian Electronic Payment System Sdn Bhd, pembekal perkhidmatan antara bank di Malaysia.
      1.38 “Perjanjian Peniaga” bermaksud perjanjian yang ditandatangani di antara Peniaga Sah dan Bank.
      1.39 “Fi Penyata Bulanan” bermaksud fi bulanan yang dikenakan ke atas Pemegang Kad untuk penyata bulanan Kad Debit yang dicetak.
      1.40 "MOTO" ertinya Pesanan Mel Pesanan Telefon yang merupakan jenis Transaksi Kad-Tidak-Hadir di mana Perkhidmatan diselesaikan dan dibayar melalui telefon, mel, faks atau e-mel.
      1.41 “MyKad” bermaksud kad yang dikeluarkan kepada Pemegang Kad oleh Jabatan Pendaftaran Negara Malaysia kepada Pemegang Kad untuk mengenalpasti and megesahkan identiti Pemegang Kad.
      1.42 "MyDebit" bermaksud suatu skim kad debit domestik yang membenarkan Pemegang Kad untuk membuat bayaran bagi pembelian barangan dan/atau perkhidmatan dengan mendebit secara langsung dari Akaun Pembelian Runcit Pemegang Kad."""
    ]
    
    for paragraph in test_paragraphs:
        pages = find_page_by_paragraph(content_list, paragraph)
        if pages:
            if len(pages) == 1:
                print(f"Paragraph found on page {pages[0]}")
            else:
                print(f"Paragraph spans across pages: {', '.join(map(str, pages))}")
        else:
            print("Paragraph not found in the content list.")

Paragraph spans across pages: 3, 4


In [4]:
from helpers.document_processor import DocumentProcessor

document_processor = DocumentProcessor()
if __name__ == "__main__":
    # Example content list
    content_list = [
  {
    "page": 1,
    "content": "TERMA DAN SYARAT KAD DEBIT HONG LEONG (English Version)\nDikemaskini pada 6 Januari 2025\nSebagai pertimbangan Hong Leong Bank Berhad atau Hong Leong Islamic Bank Berhad (\"Bank\")bersetuju\nuntuk menyediakan Perkhidmatan dan bersetuju untuk mengeluarkan Kad Debit (seperti yang ditentukan\ndalam Klausa 1) termasuk Gabungan (sebagaimana didefinisikan dalam Klausa 1.2) kepada Pemegang\nKad atas permohonan Pemegang Kad, Pemegang Kad berjanji dan bersetuju bahawa penurunan\ntandatangan pada Kad Debit, penggunaan Kad Debit dan/atau pengekalan Akaun oleh Pemegang Kad\nakan membentuk perjanjian Pemegang Kad dengan terma dan syarat (\u201cT&S\u201d) di bawah.\nT&S ini hendaklah dibaca bersama sebagai keseluruhan dengan Terma dan Syarat Am Akaun Bank,\nTerma dan Syarat Penggunaan HLB Connect dan sebarang terma dan syarat akaun yang berkaitan, serta\nsyarat dan peraturan lain yang mengikat Bank. Definisi berikut berkuatkuasa kecuali dinyatakan sebaliknya:\n1.0\nDefinisi dan Interpretasi\n1.1\n\u201cAkaun\u201d bermaksud akaun atau akaun-akaun Pelanggan dengan HLB / HLISB dan hendaklah\ntermasuk mana-mana akaun baharu lain yang dibuka dari semasa ke semasa\n1.2\n\u201cGabungan\u201d bermaksud Kad Debit yang ditawarkan oleh institusi kewangan dalam perkongsian\ndengan institusi lain.\n1.3\n\u201cFi Tahunan\u201d bermaksud fi yang dikenakan ke atas Pemegang Kad setiap tahun.\n1.4\n\"Pihak Berkuasa Berkenaan\" bermaksud mana-mana kerajaan atau pihak berkuasa percukaian.\n1.5\n\u201cATM\u201d bermaksud mesin juruwang automatik:\n(a) yang dipasang oleh Bank atau mana-mana ahli Rangkaian ATM Kongsi (Shared ATM Network\n(SAN)); dan/atau\n(b) ditetapkan oleh Bank atau Visa/Mastercard,\nuntuk penggunaan Pemegang Kad.\n1.6\n\u201cTransaksi Kad ATM\u201d bermaksud penggunaan Kad Debit untuk pengeluaran tunai dan transaksi\nelektronik atau mana-mana kad yang mungkin diluluskan oleh Bank dari semasa ke semasa.\n1.7\n\u201cRangkaian Tunai Dibenarkan\u201d bermaksud cawangan, pejabat dan/atau lokasi yang ditetapkan\noleh Rangkaian ATM Kongsi, Visa/Mastercard untuk melaksanakan pengeluaran wang tunai.\n1.8\n\u201cPeniaga Sah\u201d bermaksud mana-mana peruncit atau perbadanan yang mana selaras dengan\nPerjanjian Peniaga bersetuju untuk menerima atau menyebabkan outletnya menerima Kad Debit\nuntuk\npembayaran\natau\nmenurut\nkepada\nperkiraan\nundang-undang\ndengan\nMyDebit/Visa/Mastercard bersetuju untuk menerima atau menyebabkan rangkaiannya untuk\nmenerima kemudahan yang ditawarkan bergabung atau berkongsi jenama Visa/Mastercard untuk\npembayaran."
  },
  {
    "page": 2,
    "content": "1.9\n\u201cTransaksi Auto Debit\u201d bermaksud bayaran berulang melalui Kad Debit untuk caj utiliti, insurans\natau takaful yang diluluskan oleh Bank sahaja.\n1.10\n\u201cBank\u201d bermaksud sama ada Hong Leong Bank Berhad (193401000023 (97141-X)) atau Hong\nLeong Islamic Bank Berhad (200501009144 (686191-W)) dan termasuk pewaris-namanya dan\npemegang serah-hak dibenarkan.\n1.11\n\u201cHari Perniagaan\u201d bermaksud hari apabila HLB / HLISB dibuka untuk perniagaan di semenanjung\nMalaysia, Sabah dan Sarawak (tidak termasuk hari kelepasan bank, negeri dan hari kelepasan am),\nmengikut mana yang berkenaan, dan hari apabila transaksi yang jenisnya dicadang bagi Akaun\ndijalankan.\n1.12\n\u201cPemegang Kad\u201d bermaksud Pemegang Kad Bank yang mana Kad Debit diterbitkan kepadanya.\n1.13\n\u201cFi Penggantian Kad\u201d bermaksud fi yang dikenakan ke atas Pemegang Kad jika kad hilang, dicuri\natau rosak.\n1.14\n\u201cTransaksi Kad\u201d bermaksud transaksi yang dilaksanakan dengan menggunakan Kad Debit untuk\nurus niaga tempatan dan luar negara; Transaksi dengan Kad secara bersemuka sertaTransaksi\nTanpa Kad dan Transaksi Tanpa Sentuhan secara tidak bersemuka (di mana berkenaan).\n1.15\n\u201cTransaksi Card-Present (CP)\u201d bermaksud pembayaran Transaksi Runcit di mana Pemegang Kad\ndan Kad Debit hadir secara fizikal di Peniaga Sah apabila pembayaran dilakukan.\n1.16\n\u201cTransaksi Card-Not-Present (CNP)\u201d bermaksud pembayaran Transaksi Runcit di mana\nPemegang Kad dan Kad Debit tidak hadir secara fizikal di Peniaga Sah apabila pembayaran\ndilakukan.\n1.17\n\u201cBaki Semasa\u201d bermaksud baki atau tunai terkini yang ada di dalam akaun simpanan atau\nsemasa Pemegang Kad.\n1.18\n\u201cFi Pengeluaran Tunai\u201d bermaksud fi yang dikenakan ke atas Pemegang Kad untuk pengeluaran\ntunai berjaya dari ATM.\n1.19\n\u201cTransaksi Tanpa Sentuhan\u201d bermaksud kaedah pembayaran yang cepat dan mudah yang\nmenggunakan teknologi komunikasi medan dekat (NFC) bagi membuat pembayaran dengan\nmenyentuh/mengimbas Kad Debit pada pembaca selamat.\n1.20\n\u201cKad Karakter\u201d merujuk kepada Kad Debit seperti Kad Debit Hello Kitty atau Transformers yang\ndikeluarkan oleh Bank.\n1.21\n\u201cHad Pengeluaran Tunai Harian\u201d bermaksud had maksimum harian dibenarkan yang ditetapkan\noleh Bank berhubung pengeluaran tunai melalui ATM.\n1.22\n\u201cHad Belian Dalam Talian Harian\u201d bermaksud had maksimum belian harian dalam talian\ndibenarkan yang ditetapkan oleh Bank dalam Klausa 12.4 di sini.\n1.23\n\u201cHad Belian Runcit Harian\u201d bermaksud had maksimum belian runcit harian dibenarkan yang\nditetapkan oleh Bank dalam Klausa 12.3 di sini."
  },
  {
    "page": 3,
    "content": "1.24\n\u201cHad Pemindahan Harian\u201d bermaksud had maksimum harian yang ditetapkan oleh Bank\nberkenaan dengan pemindahan wang melalui ATM dan/atau Hong Leong Connect.\n1.25\n\u201cKad Debit\u201d bermaksud Kad Debit MyDebit/Visa/Mastercard Hong Leong yang dikeluarkan oleh\nBank.\n1.26\n\u201cDCC\u201d merujuk kepada Penukaran Mata Wang Dinamik, iaitu perkhidmatan pilihan yang\nditawarkan oleh peniaga atau tersedia untuk pembelian luar negara tertentu seperti yang\nditentukan dalam Klausa 20.1 di bawah (termasuk Transaksi Kad ATM), yang memberi pilihan\npembayaran / pengeluaran tunai dalam Ringgit Malaysia (termasuk pembelian di luar negara dan\npembelian dalam talian dalam mata wang asing) untuk transaksi luar negara, sebagaimana yang\ndihuraikan dalam Klausa 20.\n1.27\n\u201cFi\u201d termasuk fi perlu dibayar oleh Pemegang Kad semasa permohonan, tahunan atau sebarang\njarak waktu yang mungkin ditentukan oleh Bank untuk menggunakan Perkhidmatan yang akan\ndidebit dari Akaun pada setiap tarikh ulangtahun Kad Debit dikeluarkan dan akan merangkumi\nsemua fi lain, caj perkhidmatan, komisen dan lain-lain bayaran yang dicaj oleh Bank di bawah T&S\nini. Bank berhak untuk mengubah Fi dengan memberi notis awal dua puluh satu (21) hari kalendar\nterlebih dahulu kepada Pemegang Kad.\n1.28\n\u201cPemegang Kad Biasa\u201d bermaksud Pemegang Kad bukan Perbankan Prioriti.\n1.29\n\u201cTarikh Baik Sehingga\u201d bermaksud tarikh luput yang dicetak pada Kad Debit.\n1.30\n\u201cBarangan dan Perkhidmatan yang mematuhi Syariah\u201d bermaksud barangan & perkhidmatan\nyang boleh diterima dan mematuhi Syariah.\n1.31\n\u201cHLB Connect App\u201d bermaksud Perbankan Mudah Alih Hong Leong Connect.\n1.32\n\u201cHong Leong Connect\u201d bermaksud perkhidmatan perbankan internet dan perkhidmatan\naplikasi mudah alih yang disediakan oleh HLB / HLISB.\n1.33\n\u201cPemegang Akaun Bersama\u201d bermaksud mana-mana Akaun yang ada dua (2) atau lebih nama\nbersama atau mempunyai dua (2) atau lebih yang diberi kuasa penandatangan.\n1.34\n\u201cMastercard\u201d bermaksud MasterCard International Incorporated, sebuah syarikat yang\ndikendalikan di bawah Negeri Delaware dengan pejabat dan pusat perniagaan utamanya di 2000\nPurchase Street, Purchase, N.Y. 10577-2509, Amerika Syarikat di mana Bank adalah ahli\ninstitusinya.\n1.35\n\"MCCS\" bermaksud Spesifikasi Kad Cip Malaysia, iaitu ialah skim kad debit EMV (Euro,\nMastercard, dan Visa) domestik yang menyokong kedua-dua aplikasi sentuhan dan tanpa\nsentuhan.\n1.36\n\u201cAkaun Diaktifkan MCF\u201d bermaksud Akaun Pemegang Kad di mana MCF (seperti yang ditakrifkan\ndalam Klausa 1.45 di sini) telah diupayakan, yang mungkin termasuk akaun yang dipautkanke Kad\nDebit. telah diaktifkan, yang mungkin termasuk Akaun Pembelian Runcit.\n1.37\n\u201cMEPS\u201d bermaksud Malaysian Electronic Payment System Sdn Bhd, pembekal perkhidmatan\nantara bank di Malaysia."
  },
  {
    "page": 4,
    "content": "1.38\n\u201cPerjanjian Peniaga\u201d bermaksud perjanjian yang ditandatangani di antara Peniaga Sah dan Bank.\n1.39\n\u201cFi Penyata Bulanan\u201d bermaksud fi bulanan yang dikenakan ke atas Pemegang Kad untuk\npenyata bulanan Kad Debit yang dicetak.\n1.40\n\"MOTO\" ertinya Pesanan Mel Pesanan Telefon yang merupakan jenis Transaksi Kad-Tidak-Hadir\ndi mana Perkhidmatan diselesaikan dan dibayar melalui telefon, mel, faks atau e-mel.\n1.41\n\u201cMyKad\u201d bermaksud kad yang dikeluarkan kepada Pemegang Kad oleh Jabatan Pendaftaran\nNegara Malaysia kepada Pemegang Kad untuk mengenalpasti and megesahkan identiti\nPemegang Kad.\n1.42\n\"MyDebit\" bermaksud suatu skim kad debit domestik yang membenarkan Pemegang Kad untuk\nmembuat bayaran bagi pembelian barangan dan/atau perkhidmatan dengan mendebit secara\nlangsung dari Akaun Pembelian Runcit Pemegang Kad.\n1.43\n\u201cKad Debit Multi Currency\u201d atau \u201cKad MCF\u201d atau \u201cKad Debit Multi Currency Visa Hong\nLeong\u201d merujuk kepada Kad Debit dengan asas warna hitam yang dikeluarkan oleh Bank kepada\npemegang Akaun Pay&Save Hong Leong dan hanya boleh dipautkan kepada Akaun Pembelian\nRuncit ke Akaun Pay&Save Hong Leong.\n1.44\n\u201cCiri Mata Wang Pelbagai\u201d atau \u201cMCF\u201d bermaksud mata wang asing yang disimpan dalam Akaun\nyang dibenarkan oleh Bank untuk MCF yang membolehkan Pemegang Kad, antara lain:\n(a) memegang mata wang asing yang dibenarkan oleh Bank dari semasa ke semasa;\n(b) melakukan Transaksi Luar Negara dalam mata wang asing yang dipilih oleh Pemegang Kad;\n(c) menukar satu (1) mata wang asing ke mata wang asing yang lain dalam Akaun Diaktifkan MCF\nPemegang Kad melalui cawangan Bank atau Hong Leong Connect.\nBagi mengelakkan keraguan, MCF hanya tersedia untuk Akaun Diaktifkan MCF yang dipegang\noleh Pemegang Akaun tunggal. Mata wang asing terkini yang diluluskan untuk MCF boleh didapati\nhttps://www.hlb.com.my/ms/personal-banking/deposits/savings-account/pay-and-save-\naccount.html.\n1.45\n\u201cPaparan Notifikasi HLB Connect App\u201d bermaksud ciri notifikasi tolak yang membolehkan\nPemegang Kad menerima notifikasi melalui HLB Connect App.\n1.46\n\u201cOne-Time Password (OTP)\u201d bermaksud kod 6-digit yang unik dan sekali guna yang dihantar\nmelalui SMS atau Kad OTP dalam Aplikasi HLB Connect untuk melakukan transaksi dalam talian\ntertentu atau tinkdakan.\n1.47\n\u201cHad Belian Dalam Talian\u201d bermaksud had transaksi yang terpakai ke atas Transaksi Kad yang\ndilaksanakan dalam talian melalui internet.\n1.48\n\u201cPembelian Petrol\u201d bermaksud semua transaksi yang dibuat dengan Kad Debit untuk\npembayaran petrol dan mana-mana pembayaran lain yang dilakukan di kiosk petrol.\n1.49\n\u201cPIN\u201d bermaksud kod berangka yang bertindak sebagai kata laluan yang digunakan untuk\nmengesahkan Pemegang Kad ke akaun mereka\n1.50\n\u201cTarikh Pengeposan\u201d bermaksud tarikh di mana sebarang Transaksi Kad yang dilakukan oleh"
  },
  {
    "page": 5,
    "content": "Pemegang Kad didebitkan dan direkodkan dalam Penyata.\n1.51\n\u201cPemegang Kad Perbankan Prioriti\u201d bermaksud Pemegang Kad dengan status Perbankan\nPrioriti yang diperakui oleh Bank.\n1.52\n\u201cNotis Privasi\u201d bermaksud dasar dan prinsip Bank yang berkaitan dengan pengumpulan,\npenggunaan dan penyimpanan maklumat peribadi individu dan entiti sedia ada dan prospektif yang\nberurusan dengan Bank yang mana mungkin akan dipinda dari semasa ke semasa dan diterbitkan\ndi laman sesawang Bank atau dengan cara yang dianggap wajar oleh Bank.\n1.53\n\u201cHad Belian Runcit\u201d bermaksud had transaksi yang dikuatkuasakan bagi Transaksi Kad termasuk\nTransaksi Tanpa Sentuhan yang dibuat di kedai runcit, atau mana-mana kedai lain di mana Kad\nDebit boleh diterima untuk pembayaran.\n1.54\n\u201cAkaun Pembelian Runcit\u201d bermaksud akaun yang dikaitkan dengan Kad Debit untuk membuat\nTransaksi Runcit.\n1.55\n\"Transaksi Runcit\" bermaksud transaksi yang menggunakan Kad Debit di Peniaga Sah di mana\nKad Debit boleh diterima untuk pembayaran di dalam dan luar negara termasuk Pembelian Dalam\nTalian, tambah nilai Touch 'n Go secara automatik, Pembelian Petrol, Auto Bill/transaksi urusniaga\nyang berulang dan tidak termasuk pengeluaran wang tunai, pemindahan dana, pembayaran yuran\ntahunan dan perkhidmatan lain/yuran pelbagai seperti yang ditakrifkan oleh Bank dari semasa ke\nsemasa dengan memberi notis terlebih dahulu.\n1.56\n\u201cFi Pencarian Rekod Jualan\u201d bermaksud fi yang dikenakan ke atas Pemegang Kad apabila\nPemegang Kad memohon untuk mendapatkan rekod jualan yang diisu oleh Peniaga Sah dari\nBank. Fi ini juga dikenakan semasa penyiasatan kes pertikaian apabila Pemegang Kad memohon\nuntuk melihat rekod jualan yang berkaitan.\n1.57\n\u201cKod Keselamatan\u201d bermaksud kod keselamatan yang diberikan oleh Bank kepada Pemegang\nKad untuk mengakses Perkhidmatan berkaitan yang merangkumi PIN (untuk Perkhidmatan ATM),\nIPIN (untuk Hong Leong Connect), HLB Connect Code (untuk Hong Leong Connect) dan termasuk\nlain-lain nama pengguna, kata laluan, nombor pengenalan peribadi, sijil digital atau lain-lain kod\nkeselamatan yang mungkin dikeluarkan oleh Bank dari semasa ke semasa untuk mengakses\nsemua atau mana-mana Perkhidmatan dan rujukan untuk terma \u201cKod Keselamatan\u201d\nakanbermaksud kod atau kod-kod keselamatan berkaitan dengan Perkhidmatan tersebut\nsebagaimanadiperlukan.\n1.58\n\u201cPerkhidmatan\u201d bermaksud perkhidmatan yang disediakan oleh Bank untuk memudahkan\nTransaksi Kad, Transaksi Tanpa Sentuhan, transaksi Cip dan PIN, Perbankan Telefon,\nPerkhidmatan Pembayaran Bil, atau mana-mana perkhidmatan elektronik, digital, visual, video\natau lain-lain seperti perkhidmatan baru atau tambahan yang ditawarkan oleh Bank dari semasa\nke semasa bagi membolehkan Pemegang Kad untuk melaksanakan transaksi perbankan dan lain-\nlain transaksi dan Perkhidmatan itu boleh ditambah, diubah atau ditarik balik oleh Bank pada bila-\nbila masa dan dari semasa ke semasa dengan memberi notis awal dua puluh satu (21) hari\nkalendar terlebih dahulu.\n1.59\n\u201cRangkaian ATM Kongsi\u201d bermaksud bank dan institusi kewangan yang mengambil bahagian di\ndalam Rangkaian ATM Kongsi yang dikendalikan oleh Payments Network Malaysia Sdn Bhd.\n1.60\n\u201cPenyata\u201d bermaksud penyata akaun mencerminkan urusniaga yang dilaksanakan dengan Kad"
  },
  {
    "page": 6,
    "content": "Debit.\n1.61\n\"Cukai\" bermaksud sebarang cukai semasa atau masa hadapan, langsung atau tidak langsung,\ncukai, levi, impos, duti, caj, yuran, sebarang bentuk potongan atau pegangan dalam sebarang\nbentuk, Malaysia atau asing, yang dikenakan oleh Pihak Berkuasa Berkenaan, termasuk tanpa\nhad, sebarang cukai penggunaan dan lain-lain cukai dengan apa sahaja nama yang dipanggil, dan\nsebarang faedah, denda atau penalti yang berkenaan dengannya.\n1.62\n\"Tarikh Transaksi\" bermaksud tarikh di mana Transaksi Pembelian Runcit tersebut\nmenggunakan Kad Debit di Peniaga Sah di mana Kad Debit adalah diterima untuk pembayaran.\n1.63\n\u201cResit Transaksi\u201d bermaksud slip bayaran yang berkaitan, borang atau kertas yang dibekalkan\nkepada/oleh Bank kepada/oleh Peniaga Sah atau dijana oleh ATM untuk tujuan merekod,\nmengesah dan membuktikan pembelian atau transaksi lain yang dilakukan oleh Pemegang Kad\nmelalui penggunaan Kad Debit yang akan dicajkan dan didebitkan daripada Akaun atau\npengesahan yang diberikan kepada Pemegang Kad untuk transaksi kewangan yang dilakukan\nmelalui Hong Leong Connect.\n1.64\n\u201cVisa\u201d bermaksud Visa International Service Association, sebuah syarikat yang dikendalikan di\nbawah Negeri Delaware dengan ibu pejabatnya di 3125 Clearview Way, San Mateo, CA 94402,\nAmerika Syarikat di mana Bank adalah ahli institusinya.\n1.65\nPerkataan yang bermaksud tunggal juga bermaksud majmuk dan sebaliknya serta serta perkataan\nyang bermaksud jantina maskulin hendaklah termasuk jantina feminin dan neuter dan sebaliknya.\n2.0\nTanggungjawab Pemegang Kad\n2.1\nDengan menandatangani Kad Debit, menggunakan Kad Debit dan/atau mengekalkan Akaun,\nPemegang Kad bersetuju untuk terikat sepenuhnya dengan T&S ini yang mungkin diubah oleh\nBank dari semasa ke semasa dengan memberi notis awal dua puluh satu (21) hari kalendar. Notis\nmengenai perubahan atau pindaan tersebut akan dimaklumkan kepada Pemegang Kad dalam\nsebarang bentuk atau kaedah sebagaimana yang ditetapkan oleh Bank.\n2.2\nPemegang Kad harus mengambil semua langkah wajar untuk memastikan dan mencegah\npenggunaan tanpa kebenaran dan secara penipuan dan meyalahi undang-undang ke atas\nPerkhidmatan atau sebahagian daripadanya dan Kod Keselamatan Pemegang Kad, termasuk\ntetapi tidak terhad kepada yang berikut:\n(a) Kod Keselamatan hendaklah dirahsiakan dan apabila Pemegang Kad menerima Kod\nKeselamatan ia hendaklah ditukar sertamerta apabila telah diterima dan dibaca, dan hanya\nboleh digunakan oleh Pemegang Kad sahaja dan bukan orang lain. Jika Kod Keselamatan\ntidak dikeluarkan kepada Pemegang Kad, Pemegang Kad dinasihatkan agar mencipta Kod\nKeselamatan sendiri sebagai syarat untuk mengakses Perkhidmatan.\n(b) Bahawa tidak ada orang lain selain daripada Pemegang Kad mempunyai akses atau mampu\nmempunyai akses kepada Kod Keselamatan untuk membolehkan dia melakukan sebarang\ntransaksi melalui mana-mana Perkhidmatan dan Kod Keselamatan tidak ditulis dalam apa jua\nbentuk atau cara termasuk tetapi tidak terhad kepada e-mel yang tidak diminta atau pada\nmana-mana laman sesawang selain daripada laman sesawang Bank yang boleh ditafsirkan\noleh pihak ketiga atau disimpan bersama atau didedahkan kepada mana-mana orang\n(termasuk pekerja Bank) di bawah apa-apa keadaan atau pada bila-bila masa."
  },
  {
    "page": 7,
    "content": "(c) Pemegang Kad tidak boleh mendedahkan Kod Keselamatan kepada mana-mana pihak di\nbawah apa jua sebab atau cara sama ada secara sukarela atau sebaliknya dan harus\nmengambil segala langkah wajar untuk mencegah Kod Keselamatan daripada diketahui oleh\npihak lain. Pemegang Kad memahami dan bersetuju bahawa kegagalan untuk mematuhi\nperaturan ini boleh mendedahkan Pemegang Kad kepada akibat kecurian dan/atau\npenggunaan tanpa kebenaran Kad Debit, yang mana Bank tidak akan bertanggungjawab.\nPemegang Kad mesti melaporkan sebarang pelanggaran Kod Keselamatan atau kehilangan\nKod Keselamatan kepada Bank dengan secepat yang mungkin, apabila Pemegang Kad\nmenyedari tentang pelanggaran atau kerugian masing-masing. Pemegang Kad dengan ini\nbersetuju untuk membayar balik dan membayar kepada Bank atas permintaan bertulis Bank\nsemua tuntutan dan liabiliti yang ditanggung oleh Bank yang timbul daripada penggunaan\ntanpa kebenaran tersebut.\n(d) Bahawa baki Akaun, Penyata Akaun dan Transaksi Kad diperiksa dan dipantau oleh\nPemegang Kad secara kerap dan teliti dan diperiksa setiap kali sebelum memberikan apa-apa\narahan kepada Bank. Pemegang Kad hendaklah memaklumkan kepada Bank dengan segera\ntentang apa-apa yang tidak dibenarkan, penipuan, menyalahi undang-undang urus niaga,\nkesilapan atau percanggahan dalam Akaun.\n(e) Pemegang Kad harus memastikan bahawa dana di dalam Akaun adalah mencukupi untuk\nmelaksanakan sebarang Transaksi Kad yang diarahkan oleh Pemegang Kad.\n(f) Pemegang Kad akan mencipta PIN sendiri pada ATM Bank untuk tujuan melaksanakan\ntransaksi ATM atau transaksi elektronik. Pemegang Kad dikehendaki untuk tidak mendedahkan\nPIN kepada mana-mana pihak ketiga dan akan bertanggungjawab kepada Bank untuk sebarang\nkemasukan debit di dalam Akaun dengan Bank yang diakibatkan oleh Transaksi Kad tanpa\nkebenaran.\n(g) Untuk melakukan pembelian dengan menggunakan Kad Debit di mana-mana Peniaga Sah,\nPemegang Kad boleh mengesahkan Transaksi Kad dengan (i) menandatangan Resit\nTransaksi yang disediakan oleh Peniaga Sah; (ii) memasukkan PIN pada terminal Tempat\nJualan; atau (iii) melaksanakan Transaksi Tanpa Sentuhan. Penerimaan Resit Transaksi atau\napa-apa dokumen pulangan dengan tandatangan Pemegang Kad oleh Bank (jika ada) akan\nmenjadi bukti yang mengikat dan muktamad bahawa Transaksi Kad dan amaun yang direkod\npadanya adalah dilakukan oleh Pemegang Kad selaras dengan T&S ini. Untuk pembelian atau\npenggunaan Kad Debit yang dibuat secara transaksi dalam talian, atau transaksi bukan\nberasaskan tandatangan termasuk tetapi tidak terhad kepada pembelian petrol, transaksi\ndalam talian, Pemegang Kad bersetuju bahawa resit pengesahan dan/atau penerimaan yang\ndikeluarkan oleh mana-mana Peniaga Sah atau sekutunya untuk transaksi tersebut akan\ndianggap sebagai bukti dokumen yang memuaskan sebagai penggunaan dan Pemegang Kad\nharus mematuhi T&S ini.\n(h) Walapun peruntukan yang dinyatakan di bawah Klausa 2.2(g) di atas, tandatangan/PIN bukan\nsatu-satunya syarat yang boleh didahulukan kepada liability Pemegang Kad berkenaan\ndengan perkhidmatan yang dibekalkan, dan Pemegang Kad akan bertanggungjawab untuk\nTransaksi Kad jika Bank berpendapat, berdasarkan bukti dokumen yang memuaskan, bahawa\nketidaksempurnaan tandatangan atau ketiadaan tandatangan/kemasukan PIN adalah\ndisebabkan oleh keadaan dan/atau kaedah transaksi yang digunakan atau akibat kesilapan\ndari pihak Pemegang Kad dan/atau Peniaga Sah dan ahli bank MyDebit/Visa/ Mastercard.\n(i) Pemegang Kad harus mematuhi semua keperluan, arahan, suruhan, dan garis panduan"
  },
  {
    "page": 8,
    "content": "tentang penggunaan Kad Debit yang dikeluarkan oleh Bank dari semasa ke semasa\nberkenaan dengan semua Perkhidmatan kepada Pemegang Kad.\n(j) Pemegang Kad bersetuju untuk menandatangan bahagian belakang Kad Debit sebaik sahaja\nkad tersebut diterima dan sebelum ia digunakan, sebagai langkah mencegah penggunaan\ntanpa kebenaran Kad Debit tersebut.\n(k) Kad Debit adalah sah sehingga Tarikh Baik Sehingga. Surat pembaharuan kepada Pemegang\nKad akan dikirimkan tiga puluh (30) hari sebelum Tarikh Baik Sehingga. Bank akan mengirim\nsurat pembaharuan kepada Pemegang Kad untuk memaklumkan Pemegang Kad untuk\nmengunjungi cawangan Bank yang terdekat atau mana-mana cawangan Bank untuk\nmemperolehi Kad Debit Penggantian. Pemegang Kad harus memastikan bahawa sebaik\nsahaja Kad Debit luput, ia dimusnahkan, dengan memotong dua secara melintang dan\nmengembalikannya kepada Bank untuk penggantian Kad Debit.\n(l) Kad Debit tidak boleh digunakan selepas pembatalan, tamat tempoh, atau pengeluaran atau\napabila Pemegang Kad tidak lagi menjadi Pemegang Kad Bank.\n(m) Pemegang Kad tidak boleh menggunakan Kad Debit untuk pengeluaran tunai, bayaran atau\npemindahan dana melainkan terdapat dana yang mencukupi di dalam Akaun. Sebaran\npengeluaran tunai, bayaran atau pindahan dana akan ditolak jika tidak terdapat dana yang\nmencukupi di dalam Akaun.\n(n) Sekiranya tiada terdapat dana mencukupi di dalam Akaun dan Pemegang Kad tidak\nmempunyai kemudahan overdraf atau mana-mana kemudahan kredit tersedia, semua\nTransaksi Kad di Peniaga Sah akan ditolak.\n(o) Pemegang Kad bersetuju untuk tidak menggunakan Kad Debit untuk sebarang aktiviti yang\nmenyalahi undang-undang. Jika Bank mendapati, mengesyaki atau mempunyai sebab untuk\nmempercayai bahawa Kad Debit telah digunakan untuk sebarang aktiviti yang menyalahi\nundang-undang, Bank boleh mengambil apa-apa tindakan dianggap sesuai untuk memenuhi\napa-apa kewajipan yang berkaitan dengan pencegahan apa-apa aktiviti haram termasuk tetapi\ntidak terhad kepada penipuan, pengubahan wang haram, aktiviti/pembiayaan pengganas,\nrasuah, penyalahgunaan wang dan/atau pengelakan cukai.\n(p) Sekiranya Pemegang Kad menjadikan Akaun Simpanan-i atau Akaun Semasa-i sebagai\nAkaun Pembelian Runcit, Pemegang Kad mengakui dan bersetuju bahawa Kad Debit tidak\nboleh digunakan untuk sebarang aktiviti atau tujuan tidak patuh Syariah seperti yang\ndinyatakan di Klausa 13 di bawah dan yang boleh menyebabkan transaksi tersebut ditolak.\n(q) Pemegang Kad dengan ini memberi representasi dan memberi waranti bahawa Pemegang\nKad telah mendapat persetujuan semua orang yang dinamakan dalam permohonan\nPemegang Kad untuk Akaun atau apa-apa dokumen lain yang dihantar kepada Bank bagi\nmenyokong permohonan itu dan/atau wakil yang diberi kuasa mereka, termasuk tetapi tidak\nterhad kepada pengarah, pemegang saham, penandatangan yang diberi kuasa Pemegang\nKad atau mana-mana orang lain yang ditetapkan oleh Bank (\"Subjek Data Relevan\"), untuk\nkoleksi Bank, pegangan dan penggunaan maklumat peribadi daripada subjek Data Relevan\nmengikut Notis Privasi Bank sebagaimana yang dipinda dari semasa ke semasa.\n(r) Sekiranya Pemegang Kad menggantikan Kad Debit beliau dengan Kad Debit baru kerana\nhilang/dicuri/pembaharuan atau apa-apa sebab lain, Pemegang Kad bertanggungjawab untuk\nmemaklumkan kepada semua Peniaga Sah berkaitan yang beliau telah melanggan auto\ndebit/transaksi debit langsung untuk mengemaskini nombor Kad Debit baru untuk"
  },
  {
    "page": 9,
    "content": "mengelakkan transaksi daripada ditolak.\n(s) Pemegang Kad dengan ini bersetuju dan memberi kebenaran kepada pegangan,\npengumpulan dan penggunaan semua data peribadi yang diberikan kepada Bank oleh\nPemegang Kad dan/atau diperoleh oleh Bank dari domain awam, dan juga data peribadi yang\ntimbul akibat daripada penyediaan perkhidmatan kepada Pemegang Kad berhubung dengan\nAkaun-Akaun yang selaras dengan Notis Privasi Bank sebagaimana yang dipinda dari semasa\nke semasa.\n(t) Pemegang Kad yang ingin dikecualikan daripada pekongsian data peribadi mereka di dalam\nBank dan/atau kumpulan syarikat Hong Leong Financial Group Berhad bagi tujuan pemasaran\ndan promosi perlu mengunjungi mana-mana cawangan Bank atau menghubungi Pusat\nPanggilan Bank di 03-7626 8899 untuk mendaftar permintaan mengecualikan pekongsian\ntersebut.\n(u) Pemegang Kad hendaklah segera memberitahu Bank jika ada apa-apa perubahan dalam\nbutir-butir perhubungan dan/atau peribadi Pemegang Kad sama ada secara bertulis,\nmengunjungi cawangan-cawangan Bank atau menghubungi Pusat Hubungan Bank di 03-7626\n8899.\n(v) Bank tidak menyatakan atau menjamin bahawa penggunaan Kad Debit tidak akan berlaku\ntanpa gangguan, gangguan atau kesilapan. Bank tidak akan bertanggungjawab ke atas apa-\napa kerugian, kerosakan, kesulitan, keaiban, kos atau perbelanjaan dalam apa jua bentuk yang\ndalam apa-apa cara mungkin dialami atau ditanggung oleh Pemegang Kad atau oleh mana-\nmana orang lain berkenaan dengan atau berkaitan dengan penggunaan Perkhidmatan\ndisebabkan oleh sebarang kejadian di luar kawalan munasabah Bank dan Pemegang Kad\ntidak akan mempunyai tuntutan terhadap Bank jika Transaksi Kad dan/atau Transaksi Tanpa\nSentuhan tidak diterima atau dihormati oleh mana-mana Peniaga Sah atau atas apa jua sebab\nsekalipun.\n(w) Pemegang Kad mengakui dan bersetuju bahawa penggunaan Kad Debit atau prestasi\nTransaksi Kad dan/atau Transaksi Tanpa Sentuhan adalah atas risiko mutlak Pemegang Kad.\nBank tidak akan bertanggungjawab ke atas sebarang kerosakan dan/atau kerugian yang\nditanggung berhubung dengan penggunaan Kad Debit atau prestasi Transaksi Kad dan/atau\nTransaksi Tanpa Sentuhan bagi barangan atau perkhidmatan yang dibekalkan oleh mana-\nmana Peniaga Sah atau kualiti atau pelaksanaan apa-apa barangan atau perkhidmatan\nselaras dengan atau berhubung dengan apa-apa transaksi yang dibuat pada Kad Debit.\nPemegang Kad akan menyelesaikan secara langsung dengan Peniaga Sah bagi apa-apa\naduan yang timbul daripada Transaksi Kad dan/atau Transaksi Tanpa Sentuhan.\n(x) Pemegang Kad harus memastikan bahawa jika beliau mempunyai lebih daripada satu (1) kad\natau instrumen pembayaran dengan teknologi tanpa sentuh, iaitu instrumen pembayaran\ndengan keupayaan untuk melaksanakan Transaksi Tanpa Sentuhan, Pemegang Kad harus\nmemastikan bahawa kad tersebut disimpan secara berasingan daripada Kad Debit untuk\nmengelakkan urus niaga perlanggaran yang akan menyebabkan kedua-dua kad didebitkan\nuntuk jumlah urus niaga yang berkaitan.\n(y) Di mana debit atau kedudukan terlebih pengeluaran yang tidak diatur sebelumnya timbul dalam\nakaun semasa atas apa jua sebab, Pelanggan bertanggungjawab dan hendaklah membayar\namaun terlebih keluar ditambah faedah yang boleh dikenakan iaitu 4.0% p.a. melebihi BLR pada\nbaki harian atas baki yang terlebih keluar."
  },
  {
    "page": 10,
    "content": "(z) Pemegang Kad harus memastikan semua Transaksi Kad (termasuk Transaksi Luar Negara\nseperti yang ditakrifkan di bawah Klausa 20.1 di bawah) mematuhi Notis Dasar Pertukaran\nAsing (\u201cNotis FEP\u201d). Pemegang Kad disarankan untuk membaca dan memahami Notis FEP\ndi laman sesawang Bank Negara Malaysia (\u201cBNM\u201d) di http://www.bnm.gov.my \u2013 tab \u2018Notis\nDasar Pertukaran Asing\u2019 untuk memastikan semua aktiviti yang dilakukan menggunakan Kad\nDebit mematuhi Notis FEP setiap masa.\n3.0\nTransaksi Tanpa Sentuhan\n3.1\nTerdapat tiga (3) jenis Transaksi Tanpa Sentuhan iaitu:\n(a) MCCS Tanpa Sentuhan \u2013 kaedah pembayaran tanpa sentuhan melalui rangkaian MyDebit\nyang boleh dilakukan di mana-mana terminal Point-of-Sales yang memaparkan logo MCCS\nTanpa Sentuhan;\n(b) Visa Tanpa Sentuhan \u2013 kaedah pembayaran tanpa sentuhan melalui rangkaian Visa yang\nboleh dilakukan di mana-mana terminal Point-of-Sales yang memaparkan logo Visa Tanpa\nSentuhan; dan\n(c) Mastercard Tanpa Sentuhan \u2013 kaedah pembayaran tanpa sentuhan melalui rangkaian\nMastercard yang boleh dilakukan di mana-mana terminal Point-of-Sales yang memaparkan\nlogo Mastercard Tanpa Sentuhan.\n3.2\nTransaksi Tanpa Sentuhan tanpa pengesahan PIN dihadkan kepada Ringgit Malaysia Dua Ratus\nLima Puluh (RM250) setiap transaksi (\u201cHad Transaksi Tanpa Sentuhan\u201d). Pemegang Kad akan\ndikehendaki mengesahkan PIN bagi Transaksi Tanpa Sentuhan melebihi Ringgit Malaysia Dua\nRatus Lima Puluh (RM250).\n3.3\nTransaksi Tanpa Sentuhan tanpa pengesahan PIN boleh dilakukan secara berasingan tetapi\ntransaksi kumulatif dalam satu (1) hari hendaklah dihadkan kepada Ringgit Malaysia Lapan Ratus\n(RM800) sahaja atau had lain sebagaimana yang ditetapkan oleh Pemegang Kad mengikut Klausa\n3.4 (\u201cHad Kumulatif Transaksi Harian Tanpa Sentuhan\u201d). Apa-apa Transaksi Tanpa Sentuhan\ntanpa pengesahan PIN yang melebihi Had Kumulatif Transaksi Harian Tanpa Sentuhan akan\ndibatalkan dan Pemegang Kad akan diminta untuk melakukan pengesahan PIN untuk meneruskan\nbayaran. Had Kumulatif Transaksi Harian Tanpa Sentuhan akan ditetapkan semula setiap hari atau\napabila Pemegang Kad melakukan transaksi disahkan PIN pada terminal Point-of-Sales.\n3.4\nHad Kumulatif Transaksi Harian Tanpa Sentuhan ditetapkan kepada Ringgit Malaysia Lapan Ratus\n(RM800). Pemegang Kad boleh menukarkan Had Kumulatif Transaksi Harian Tanpa Sentuhan\nmereka bila-bila masa dari RM0 \u2013 RM800 melalui Hong Leong Connect atau di mana-mana\ncawangan Bank di seluruh negara. Jika Pemegang Kad tidak ingin menggunakan fungsi Transaksi\nTanpa Sentuhan tanpa pengesahan PIN, Had Kumulatif Transaksi Harian Tanpa Sentuhan\nhendaklah ditetapkan kepada sifar, iaitu RM0.\n3.5\nHad Kumulatif Transaksi Harian Tanpa Sentuhan Pemegang Kad adalah subset Had Belian Runcit\nPemegang Kad. Oleh sebab itu, Had Kumulatif Transaksi Harian Tanpa Sentuhan tidak akan lebih\ndaripada Had Belian Runcit yang ditetapkan oleh Pemegang Kad. Had Kumulatif Transaksi Harian\nTanpa Sentuhan Pemegang Kad secara automatik akan ditetapkan kepada Had Belian Runcit\nsekiranya Had Belian Runcit kurang daripada Had Kumulatif Transaksi Harian Tanpa Sentuhan.\nContohnya, jika Pemegang Kad telah menetapkan Had Belian Runcit kepada Ringgit Malaysia\nSeratus (RM100), Had Kumulatif Transaksi Harian Tanpa Sentuhan akan menjadi Ringgit Malaysia\nSeratus (RM100)."
  },
  {
    "page": 11,
    "content": "4.0\nTransaksi Pembelian Dalam Talian\n4.1\nApabila Pemegang Kad membuat pembelian dalam talian dengan Kad Debit, Pemegang Kad akan\nmenerima OTP (One-Time Password) 6-digit dengan salah satu cara berikut:\n(i)\nMelalui OTP Kad Debit dalam HLB Connect App (jika Pemegang Kad berdaftar) atau\n(ii)\nMelalui SMS (jika Pemegang Kad tidak berdaftar dengan HLB Connect App)\n4.2\nUntuk melengkapkan pembelian dalan talian, Pemegang Kad dikehendaki::\n(i)\nSemak Kad Debit OTP melalui HLB Connect App atau SMS; dan\n(ii)\nMasukkan kod OTP 6-digit pada halaman pembayaran Peniaga\n5.0\nPenyata Pemegang Kad\n5.1\nPenyata akan dikemas kini ke dalam penyata/buku Akaun Simpanan/Akaun Simpanan-i atau\nAkaun Semasa/Akaun Semasa-i yang ditetapkan, dan Pemegang Kad boleh menyemak\nPenyatanya secara percuma melalui Hong Leong Connect yang meliputi, antara lain, Transaksi\nKad dan Tarikh Pengeposan Transaksi Kad yang dilakukan oleh Pemegang Kad untuk tempoh\nyang dinyataka.\n5.2\nAtas permintaan oleh Pemegang Kad, Penyata bulanan bercetak akan dihantar ke alamat surat-\nmenyurat Pemegang Kad yang terakhir dicatatkan di dalam rekod Bank. Caj perkhidmatan bulanan akan\ndidebitkan dari dalam Akaun. Penyata bulanan yang dicetak menunjukkan semua TransaksiKad,\nTarikh Pengeposan dan Tarikh Transaksi untuk bulan yang berkenaan. Pemegang Kad boleh\nmeminta Penyata dicetak pada bila-bila masa dan Bank akan mengenakan bayaran perkhidmatan\nsebanyak Ringgit Malaysia Sepuluh (RM10) dan Ringgit Malaysia Dua (RM2) bagi setiap halaman\nuntuk Penyata sehingga satu (1) tahun yang lalu atau Ringgit Malaysia Sepuluh (RM10) dan\nRinggit Malaysia Lima (RM5) bagi setiap halaman untuk Penyata lebih daripada satu (1) tahun\nyang lalu untuk setiap permintaan.\n5.3\nRekod dan catatan dalam Akaun yang ditetapkan dengan Bank yang terdapat pada Penyata\nbulanan akan dianggap sebagai tepat dan mengikat ke atas Pemegang Kad kecuali makluman\nbertulis bertentangan dengannya diberi oleh Pemegang Kad kepada Bank dalam masa empat\nbelas (14) hari kalendar selepas menerima.\n5.4\nJika atas sebarang sebab Pemegang Kad tidak memaklumkan kepada Bank secara bertulis dalam\nmasa empat belas (14) hari tentang sebarang percanggahan di dalam Penyata dan sekiranya\ntiada sebarang kesilapan yang jelas pada muka penyata atau penipuan oleh Bank,maka\nPemegangKad akan dianggap telah menerima rekod dan catatan di dalam Penyata sebagai betul,\nakhir danmuktamad. Penyata itu harus dianggap sebagai muktamad dan terikat kepada Pemegang\nKad, wakil perundangan dan pengganti Pemegang Kad, dan sebarang tuntutan atau dakwa\nterhadap Bank yang mendakwa Penyata itu adalah salah adalah tidak sah.\n5.5\nApabila Bank menerima notis bertulis yang diberi oleh Pemegang Kad dalam tempoh yang\ndinyatakan, Bank akan menyiasat Akaun untuk membuat pelarasan dan pembetulan yang\nsewajarnya, jika ada. Sebarang wang yang perlu dibayar kepada atau dibayar oleh Pemegang Kad\nakan dikredit atau didebit ke dalam Akaun berkaitan.\n5.6\nHanya Transaksi Kad yang telah disahkan dengan sewajarnya oleh Bank akan muncul di dalam\nPenyata Akaun Pemegang Kad.\n5.7\nPemegang Kad juga boleh mendapatkan rekod sepuluh (10) transaksi terakhir Akaun Simpanan"
  },
  {
    "page": 12,
    "content": "atau Semasa/Akaun Simpanan-i atau Semasa -i melalui ATM. Di dalam rekod transaksi ATM yang\ndikeluarkan oleh ATM, semua Transaksi Belian Runcit diwakili oleh perkataan tiga (3)-huruf \u201cPOS\u201d.\n6.0\nPemegang Akaun Bersama\n6.1\nUntuk Pemegang Akaun Bersama, salah seorang pemegang akaun boleh mengeluarkan arahan\ndan membenarkan Bank untuk melaksanakan sebarang Transaksi Kad ke atas akaun bersama.\nSemua Transaksi Kad yang dilakukan akan mengikat ke atas semua pemegang kad, yang\nbertanggungjawab secara bersama dan bersendiri. Mandat pemegang akaun bersama tidak akan\nditarik-balik kecuali dan sehingga pemegang akaun bersama yang mana mandatnya akan ditarik-\nbalik menyerahkan Kad Debitnya kepada Bank.\n7.0\nAkaun Pegangan\n7.1\nBank mungkin akan mendebit atau mengenakan pegangan ke atas Akaun Pembelian Runcit\n(\u201cAkaun Pegangan\u201d) untuk amaun Transaksi Kad sama ada pada hari ianya diserahkan kepada\nBank untuk bayaran atau pada hari Bank menerima makluman mengenai Transaksi Kad, yang\nmana lebih awal.\n7.2\nAkaun Pegangan ke atas akaun adalah sah untuk semua transaksi runcit yang dilakukan dengan\nPeniaga Sah yang memerlukan pegangan sedemikian. Jika Peniaga Sah meminta kebenaran\nuntuk sebarang Transaksi Kad, Bank mungkin akan mengenakan pegangan ke atas Akaun\nPegangan untuk amaun Transaksi Kad yang ditetapkan oleh Peniaga Sah and dipersetujui oleh\nPemegang Kad. Baki tersedia di dalam Akaun Pembelian Runcit untuk digunakan oleh Pemegang\nKad akan berkurangan sejajar dengan amaun yang dipegang.\n7.3\nTertakluk kepada Had Pembelian Runcit Harian dan Baki Semasa dalam Akaun Belian Runcit dan\nTerma & Syarat di sini, Bank hendaklah meletakkan Akaun Pegangan untuk pembelian petrol oleh\nPemegang Kad di dispenser bahan api automatik di stesen minyak di mana sistem akan secara\nautomatik memperuntukkan Ringgit Malaysia Dua Ratus (RM200). Amaun pra-kuasa tersebut\nakan dikeluarkan secara automatik sebaik sahaja bank pemeroleh menuntut nilai pembelian\nsebenar kepada bank dalam tempoh tiga (3) Hari Perniagaan atau tempoh lain yang mungkin\ndimaklumkan oleh Bank.\n7.4\nSelain daripada yang dinyatakan seperti Klausa 7.3 di atas, Bank mempunyai budi bicara untuk\nmenahan jumlah tersebut sehingga dua puluh satu (21) hari kalendar atau untuk tempoh yang\ndifikirkan sesuai. Bank hendaklah mendebitkan jumlah yang ditahan ke Akaun Belian Runcit\napabila Transaksi Kad yang sepadan dikemukakan kepada Bank untuk pembayaran. Bank\nhendaklah mengeluarkan jumlah yang ditahan jika Transaksi Kad yang sepadan tidak\ndikemukakan kepada Bank untuk pembayaran dalam tempoh yang dianggap sesuai oleh Bank.\nPemegang Kad selanjutnya dengan nyata bersetuju bahawa Bank berhak untuk menahan Akaun\nBelian Runcit dan mendebit Akaun Belian Runcit jika Transaksi Kad berkemungkinan akan atau\ndibentangkan untuk pembayaran kemudiannya oleh Pedagang Dibenarkan apabila tamat tempoh\ndua puluh satu satu (21) hari kalendar. Pemegang Kad bersetuju bahawa semua resit yang\ndikemukakan oleh Pedagang Sah untuk Urus Niaga Kad melalui Resit Transaksi termasuk resit\npengesahan dan pengakuan hendaklah dianggap sebagai bukti penggunaan muktamad dan\nkonklusif oleh Pemegang Kad.\n7.5\nApabila Bank menerima bukti jumlah transaksi sebenar daripada Peniaga Sah, Bank akan\nmelepaskan Jumlah Pra-Kuasa dan Akaun Pembelian Runcit Pemegang Kad akan didebitkan"
  },
  {
    "page": 13,
    "content": "dengan jumlah transaksi sebenar tersebut.\n7.6\nPemegang Kad tidak boleh memberhentikan pembayaran ke atas Transaksi Kad. Untuk\nmengelakkan keraguan, dengan ini dinyatakan bahawa hak Bank untuk mendebit Akaun\nPembelian Runcit berhubung sebarang Transaksi Kad tidak terhad kepada amaun yang diletakkan\ndalam pegangan yang berkaitan dengan Transaksi Kad dan Bank berhak untuk mendebit Akaun\nPembelian Runcit untuk amaun penuh atas Transaksi Kad. Bank juga berhak untuk menaikkan\namaun yang perlu dipegang berkaitan mana-mana Transaksi Kad yang berdenominasi dalam mata\nwang lain selain daripada Ringgit Malaysia jika Bank berpendapat amaun asal yang dipegang tidak\nmencukupi untuk membayar Transaksi Kad sepenuhnya apabila ditukar kepada mata wang asing\ntersebut.\n7.7\nPemegang Kad tidak boleh menggunakan jumlah yang ditahan, walau apa pun terma dan syarat\nlain yang mengatur Akaun Pembelian Runcit.\n7.8\nBank hanya akan mengkreditkan Akaun Pembelian Runcit dengan kembalian yang dibuat oleh\nPeniaga Sah atau penentuan yang berkaitan dengan mana-mana transaksi runcit/transaksi dalam\ntalian selepas Bank menerima baucer kredit yang dikeluarkan dengan betul. Kembalian ini\ntermasuk tetapi tidak terhad kepada Pemegang Kad memulangkan semula barangan kepada\nperuncit selepas pembelian.\n8.0\nPemilikan dan Kad Debit Hilang atau Dicuri\n8.1\nPemegang Kad harus mengambil segala langkah keselamatan untuk mengelakkan Kad Debit\ndaripada kehilangan atau kecurian dan Pemegang Kad tidak boleh meninggalkan Kad Debit tanpa\ndijaga atau mendedahkan PIN dan/atau butiran Kad Debit kepada mana-mana pihak ketiga.\nSekiranya berlaku kehilangan dan/atau kecurian Kad Debit dan/atau pendedahan PIN dan/atau\nbutiran kepada pihak yang tidak dibenarkan, Pemegang Kad apabila menyedarinya hendaklah\nmemaklumkan kepada Bank dengan secepat yang munasabah boleh dilaksanakan selepas itu\n(jika perkara tersebut berlaku di Malaysia) atau Visa Travel Service Centre atau mana-mana ahli\nMastercard atau sekutu terdekatnya (jika perkara tersebut berlaku di luar negara). Pemegang Kad\nmemahami sepenuhnya bahawa kegagalan untuk menjaga dan mengambil langkah keselamatan\nyang munasabah dalam penyimpanan Kad Debit boleh mendedahkan Pemegang Kad kepada\nrisiko kecurian dan/atau penggunaan tanpa kebenaran Kad Debit.\n8.2\nBank berhak bertindak atas arahan telefon yang diterima oleh Bank daripada Pemegang Kad\ndan/atau wakilnya yang diberikuasa oleh Pemegang Kad (jika berkenaan dan mengikut mana-\nmana yang berkenaan) (\"Orang yang Diberi Kuasa\") dengan niat yang baik dan mengikut operasi\nyang biasa. Berhubung dengan ini, Bank tidak akan bertanggungjawab untuk bertindak atas arahan\ntelefon yang diberikan oleh Orang yang Diberi Kuasa tersebut tidak kira keadaan yang wujud pada\nmasa arahan tersebut.\n8.3\nJika berlaku kecurian/kehilangan Kad Debit dan Kad Debit tersebut telah digunakan oleh orang\nyang tidak dibenarkan untuk sebarang Transaksi Kad, Pemegang Kad bersetuju bahawa beliau\nakan bertanggungjawab kepada Bank ke atas semua caj tanpa kebenaran yang ditanggung\ntermasuk caj kewangan untuk sebarang barangan atau perkhidmatan yang diberikan oleh Peniaga\nSah, akibat penggunaan Kad Debit oleh pihak yang tidak dibenarkan tersebut sehingga tarikh dan\nmasa penerimaan oleh Bank makluman dari Pemegang Kad yang dinyatakan di dalam Klausa 8.1\ndi atas dan tertakluk kepada terma dan syarat Klausa 8.6 di dalam ini.\n8.4\nPemegang Kad bersetuju untuk memberi kuasa kepada Bank supaya tidak memperakui Kad Debit"
  },
  {
    "page": 14,
    "content": "asal yang tersebut di atas sekiranya Kad Debit yang sama diserahkan kepada Bank untuk\npembayaran setelah penerimaan oleh Bank pemberitahuan Pemegang Kad yang dinyatakan di\nbawah Klausa 8.1 diatas.\n8.5\nSelepas pengeluaran penggantian Kad Debit asal atas permintaan Pemegang Kad, Pemegang Kad\nbersetuju untuk membayar balik dan membayar kepada Bank jika Kad Debit asal digunakan dalam\napa jua cara, yang mana Pemegang Kad bersetuju bahawa Bank boleh menolak atau mendebit\ndaripada Akaun Pemegang Kad untuk jumlah tersebut.\n8.6\nPemegang Kad tidak akan dipertanggungjawab ke atas urus niaga tanpa kebenaran kad-hadir\nyang memerlukan pengesahan PIN atau tandatangan yang telah disahkan atau penggunaan kad\ntanpa-sentuh, dengan syarat Pemegang Kad tidak:\n8.6.1 melakukan penipuan;\n8.6.2 tangguh dalam memaklum Bank secepat mungkin setelah diketahui kehilangan atau\npenggunaan tanpa kebenaran Kad Kredit;\n8.6.3 secara sukarela mendedahkan PIN kepada orang lain;\n8.6.4 mencatatkan PIN pada Kad Kredit atau pada apa-apa yang disimpan berdekatan dengan Kad;\n8.6.5 meninggalkan Kad Kredit atau apa-apa yang mengandungi Kad Kredit tanpa jagaan di mana-\nmana tempat yang boleh dilihat dan diakses oleh orang lain; atau\n8.6.6 secara sukarela membenarkan orang lain untuk menggunakan Kad.\nUntuk mengelakkan keraguan, transaksi tanpa kebenaran yang dinyatakan di atas bermaksud\ntransaksi pembelian runcit dan bukan transaksi ATM. Urus niaga ATM akan merangkumi kedua-\ndua pengeluaran dan transaksi wang antara bank yang dilakukan dengan Kad Debit. Adalah\ndipersetujui dengan nyata bahawa Bank tidak bertanggungjawab untuk memastikan Baki Semasa\nyang ditetapkan oleh Bank tidak melebihi.\n8.7\nBank boleh, atas permintaan Pemegang Kad, tetapi tanpa diwajibkan di sisi undang-undang,\nmengganti Kad Debit yang hilang, dicuri atau rosak dengan mengenakan bayaran fi. Kad Debit\ngentian adalah tertakluk kepada terma dan syarat di dalam T&S ini sebagaimana Kad Debit asal.\n8.8\nSetelah Kad Debit tamat tempoh atau apabila diminta oleh Bank selepas pembatalan, pemansuhan\natau penggantungan Kad Debit oleh Bank atau apabila diketahui Kad Debit selepas pemberitahuan\nkehilangannya, Pemegang Kad tidak mempunyai apa-apa hak lagi penggunaan Kad Debit dan\nakan segera dipotong setengah dan memulangkan Kad Debit tersebut kepada Bank.\n9.0\nPenamatan oleh Pemegang Kad\n9.1\nPemegang Kad boleh pada bila-bila masa, menamatkan penggunaan Kad Debit dengan\nmemberikan makluman bertulis kepada Bank dan mengembalikan Kad Debit yang dipotong dua\nkepada Bank. Tiada kembalian Fi atau sebahagiannya akan dibuat kepada Pemegang Kad dan\nPemegang Kad akan dan kekal bertanggungjawab ke atas sebarang transaksi yang dilakukan\nmenggunakan Kad Debit sebelum Bank menerima makluman bertulis mengenai penamatan dan\npengembalian Kad Debit dipotong dua kepada Bank.\n10.0\nFi, Komisyen dan Caj\n10.1\nBank berhak untuk mengenakan Fi untuk Perkhidmatan termasuk tetapi tidak terhad kepada\npengeluaran Kad Debit, Penyata bulanan dicetak, Yuran Penggantian Kad, Fi Pengeluaran Tunai\nmelalui rangkaian MEPS, terminal Visa PLUS/Mastercard Cirrus dan Bank Antarabangsa, Dapatan\nSemula Draf Jualan dan Fi Bayaran Tahunan. Selain daripada Bayaran Fi tersebut, transaksi yang\ndijalankan melalui Perkhidmatan akan dikenakan caj dan komisen biasa. Bagi tujuan pengutipan\nYuran, komisen dan/atau caj, Pemegang Kad bersetuju untuk memberi kuasa kepada Bank untuk"
  },
  {
    "page": 15,
    "content": "mendebitkan Akaun dengan Yuran, komisyen dan/atau caj yang berkenaan. Sekiranya Pemegang\nKad tidak memberi kuasa kepada Bank untuk mendebitkan Akaun dengan Yuran, komisen\ndan/atau caj, Pemegang Kad bersetuju bahawa kegagalan pembayaran Yuran, komisen dan/atau\ncaj bagi Perkhidmatan boleh menjejaskan Perkhidmatan yang disediakan sehingga semua yuran,\nkomisen dan/atau caj yang telah dibayar dengan sewajarnya oleh Pemegang Kad.\n10.2\nPemegang Kad bersetuju untuk membayar semua fi, komisen dan/atau caj yang dikenakan dalam\nklausa ini dan membenarkan Bank untuk mendebit Akaun Pemegang Kad, tanpa mengambilkira\npendebitan tersebut mungkin menyebabkan Akaun terlebihguna. Bayaran, komisen dan/atau caj\nberikut dikenakan pada kadar yang dinyatakan atau kadar lain yang ditetapkan, yang boleh dipinda\noleh Bank untuk membuat pemindahan dari semasa ke semasa dengan memberi dua puluh satu\n(21) hari kalendar notis terlebih dahulu kepada Pemegang Kad untuk transaksi yang dilaksanakan\nmelalui penggunaan Kad Debit. Untuk senarai fi dan caj yang lengkap, sila layari laman web kami\nwww.hlb.com.my/dc2 atau imbas di sini:\n10.3\nFi Tahunan tidak akan dikenakan semasa pengeluaran Kad Debit dan hanya akan dikenakan pada\ntarikh ulang tahun. Fi Tahunan boleh diubah oleh Bank dari semasa ke semasa. Fi Tahunan tidak\nakan dikembalikan.\n11.0\nCukai Kerajaan, Fi dan Caj Berkanun dan/atau Kawal Selia\n11.1\nSebarang yuran perkhidmatan dan sebarang pembayaran yang kena dibuat oleh Pemegang Kad\nkepada Bank dibawah T&S ini, termasuk sebarang jumlah yang mewakili pembayaran ganti untuk\ndibayar oleh Pemegang Kad kepada Bank, adalah tidak termasuk Cukai, dan perlu dijelaskan tanpa\nsebarang pengecualian, syarat dan tanpa sebarang potongan bagi sebarang jumlah tuntutan balas atau\nsebarang potongan atau sekatan.\n11.2\nPada sebarang masa atau keadaan dimana Pemegang Kad dituntut oleh undang-undang untuk\nmembuat sebarang potongan atau menahan dari yuran perkhidmatan dan/atau semua wang lain\nyang kena dibayar kepada di bawah T&S ini berkaitan dengan Cukai atau sebaliknya, jumlah yang\nkena dibayar oleh Pemegang Kad dalam hal di mana potongan atau penahanan cukai diperlukan\nakan ditambah supaya yuran perkhidmatan dan/atau jumlah bersih wang yang diterima oleh Bank\nkekal dengan apa yang Bank sepatutnya menerima jika sebarang potong atau pengecualian tidak\ndikenakan.\n11.3\nSebagai penambahan atas yuran perkhidmatan dan sebarang bayaran yang perlu dibayar,\nPemegang Kad kena bayar kepada Bank semua bayaran Cukai pada kadar terkini dan/atau jumlah\nyang\nditentukan\noleh\nBank\nuntuk\nmerangkumi\nsebarang\npembayaran\nCukai/liabilti\nCukai/kewajipan-kewajipan Cukai yang berhubung kait, tanpa sebarang penolakan, sekatan atau\nsyarat-syarat dan tanpa apa-apa potongan untuk atau sebarang jumlah tuntutan balas atau apa-\napa potongan atau pengecualian, selain sebarang Cukai-Cukai yang mana mungkin diperlukan\ndi bawah sebarang undang-undang kena dibayar oleh anda secara terus kepada Pihak Berkuasa\nYang Berkenaan, dimana anda akan membayar secara terus kepada Pihak Berkuasa Yang\nBerkenaan."
  },
  {
    "page": 16,
    "content": "11.4\nJika pada bila-bila masa satu pelarasan dibuat atau dikehendaki untuk dibuat antara Bank\ndan Pihak Berkuasa Berkenaan bagi sebarang jumlah Cukai berkaitan sebarang bekalan yang\ndilakukan atau dinilai sebagai telah berlaku, yang berkaitan dengan T&S ini daripada Bank atau\nsegala yang berkaitan dengannya, satu pelarasan, tertakluk pada budi bicara, boleh dibuat diantara\nBank and Pemegang Kad dan pada keadaan sedemikian, sebarang pembayaran berkaitan\npelarasan tersebut, haruslah dijelaskan sepenuhnya.\n11.5\nSemua Cukai yang mesti dibayar oleh Pelanggan kepada Bank seperti yang diperuntukkan di ini\nadalah perlu dibayar di masa-masa sedemikian dan di sedemikian cara seperti yang akan diminta\noleh Bank.\n11.6\nPemegang Kad dengan ini bersetuju untuk memenuhi segala permintaan oleh Bank bagi\nmembantu Bank dalam mematuhi dengan kewajipan-kewajipannya di bawah undang-undang\npercukaian yang berkaitan. Jika terdapat, Cukai baru yang telah berkuatkuasa dan Cukai tersebut\nadalah diwajibkan untuk dikenakan terhadap sebarang urus niaga yang termaktub di dalam T&S\nini, Pemegang Kad bersetuju untuk memberi kerjasama dengan sepenuhnya kepada Bank untuk\nmemudahkan Bank untuk mematuhi tanggungjawab Bank di bawah undang-undang tersebut.\n11.7\nBagi mengelakkan sebarang keraguan, kesemua pihak terbabit dengan ini bersetuju bahawa\nmana-mana jumlah yang perlu dibayar atau jumlah yang digunakan untuk menentukan jumlah yang\nperlu dibayar berdasarkan Terma dan Syarat ini adalah muktamad tanpa mengambil kira untuk\nCukai dan tidak termasuk jumlah-jumlah yang perlu ditambah di bawah Klausa ini bekenaan\ndengan Cukai.\n12.0\nHad Ke Atas Transaksi Kad\n12.1\nKecuali diarahkan sebaliknya oleh Pemegang Kad atau Pemegang Kad tidak memilih suatu had\nuntuk pelaksanaan transaksi melalui Perkhidmatan, Bank berhak untuk menentukan dan\nmenetapkan sebarang had sama ada dari segi amaun, kekerapan atau kegunaan mana-mana\nPerkhidmatan dan/atau Kod Keselamatan. Pemegang Kad boleh mengubah had ke atas Transaksi\nKad yang dilakukan dengan memberikan makluman awal secara bertulis kepada Bank. Bank tidak\nbertanggungjawab untuk memastikan Pemegang Kad tidak melebihi had yang ditetapkan\nmahupun mematuhi permintaan Pemegang Kad supaya Bank mengubah had tersebut.\n12.2\nHad Pengeluaran Tunai Harian untuk Pemegang Kad Biasa dan Perbankan Prioriti ditetapkan pada\nRinggit Malaysia Dua Ribu (RM2,000), dengan had maksimum dibenarkan sebanyak Ringgit\nMalaysia Sepuluh Ribu (RM10,000) untuk Pemegang Kad Biasa dan Pemegang Kad Perbankan\nPrioriti, atau suatu amaun lain yang ditentukan oleh Bank dari semasa ke semasa dengan memberi\ndua puluh satu (21) hari kalendar terlebih dahulu makluman awal kepada Pemegang Kad.\nPemegang Kad boleh melakukan penetapan Had Pengeluaran Tunai Harian di mana-mana\ncawangan Bank atau ATM atau melalui Hong Leong Connect.\n12.3\nHad Belian Runcit Harian untuk Pemegang Kad Biasa dan Perbankan Prioriti ditetapkan pada\nRinggit Malaysia Tiga Ribu (RM3,000), dengan had maksimum dibenarkan sebanyak Ringgit\nMalaysia Dua Puluh Ribu (RM20,000) untuk Pemegang Kad Biasa dan Pemegang Kad Perbankan\nPrioriti, atau suatu amaun lain yang ditentukan oleh Bank dari semasa ke semasa dengan\nmemberikan notis awal dua puluh satu (21) hari kepada Pemegang Kad. Pemegang Kad boleh\nmelakukan penetapan Had Belian Runcit Harian di mana-mana cawangan Bank atau melalui Hong\nLeong Connect. Selain daripada itu, Pemegang Kad boleh memohon Had Belian Runcit Harian\nyang lebih tinggi (\u201cHad Belian Runcit Harian Sementara\u201d) melalui Hong Leong Connect dan Had\nBelian Runcit Harian Sementara tersebut akan berkuat kuasa sepanjang tempoh masa tertentu"
  },
  {
    "page": 17,
    "content": "yang dipilih oleh Pemegang Kad (\u201cJulat Tarikh\u201d). Had Belian Runcit Harian Sementara\nmemberikan Pemegang Kad had maksimum yang dibenarkan iaitu Ringgit Malaysia Sembilan\nPuluh Sembilan Ribu Sembilan Ratus Sembilan Puluh Sembilan (RM99,999) semasa Julat Tarikh.\nHad Belian Runcit Harian Sementara akan ditamatkan apabila Julat Tarikh berakhir dan selepas\nitu, Had Belian Runcit akan kembali kepada Had Belian Runcit Harian.\n12.4\nHad Belian Dalam Talian Harian untuk Pemegang Kad Biasa dan Pemegang Kad Perbankan\nPrioriti ditetapkan pada Ringgit Malaysia Satu Ribu (RM1,000), dengan had maksimum dibenarkan\nsebanyak Ringgit Malaysia Dua Puluh Ribu (RM20,000) untuk Pemegang Kad Biasa dan\nPemegang Kad Perbankan Prioriti, atau suatu amaun lain yang ditentukan oleh Bank dari semasa\nke semasa dengan memberikan notis awal dua puluh satu (21) hari kalendar kepada Pemegang\nKad. Pemegang Kad boleh melakukan Had Belian Dalam Talian Harian di mana-mana cawangan\nBank atau melalui Hong Leong Connect. Selain daripada itu, berkuat kuasa 24 September 2022,\nPemegang Kad boleh memohon Had Belian Dalam Talian Harian yang lebih tinggi (\u201cHad Belian\nDalam Talian Harian Sementara\u201d) melalui Hong Leong Connect dan Had Belian Dalam Talian\nSeharian Sementara tersebut akan berkuat kuasa sepanjang tempoh masa tertentu yang dipilih\noleh Pemegang Kad (\u201cJulat Tarikh\u201d). Had Belian Dalam Talian Harian Sementara memberikan\nPemegang Kad had maksimum yang dibenarkan iaitu Ringgit Malaysia Tiga Puluh Ribu\n(RM30,000) semasa Julat Tarikh. Had Belian Dalam Talian Harian Sementara akan ditamatkan\napabila Julat Tarikh berakhir dan selepas itu, Had Belian Runcit akan Kembali kepada\nHad BelianDalam Talian Harian.\n12.5 Had Pemindahan Harian melalui ATM untuk Pemegang Kad Biasa dan Perbankan Prioriti\nditetapkan pada Ringgit Malaysia Dua Ribu (RM2,000), dengan had maksimum dibenarkan\nsebanyak Ringgit Malaysia Lima Puluh Ribu (RM50,000) untuk Pemegang Kad Biasa dan\nPerbankan Prioriti, atau suatu amaun lain yang ditentukan oleh Bank dari semasa ke semasa\ndengan memberikan makluman awal dua puluh satu (21) hari kalendar terlebih dahulu kepada\nPemegang Kad. Pemegang Kad boleh melakukan penetapan Had Pemindahan Harian di mana-\nmana cawangan Bank atau ATM daripada Bank.\n12.6 Bagi mengelakkan keraguan, Had Pemindahan Harian melalui ATM adalah berasingkan daripada\nHad Pemindahan Harian melalui Hong Leong Connect.\n12.7 Pemegang Kad yang ingin menukar had amaran transaksi melalui Paparan Notifikasi Aplikasi HLB\nConnect atau khidmat pesanan ringkas (\u201cSMS\u201d) dikehendaki membuat permintaan bertulis untuk\nmenukar had yang telah ditetapkan sebelum ini atau menarik diri daripada perkhidmatan amaran\ntransaksi melalui Paparan Notifikasi Aplikasi HLB Connect atau SMS tersebut. Pemegang Kad\nboleh mendapatkan Borang Penyelenggaraan Perkhidmatan Amaran Transaksi Kad Debit/Kad\nDebit-i (\u201cBorang\u201d) di mana-mana cawangan Bank atau di Laman Sesawang Bank di\nwww.hlb.com.my. Pemegang Kad perlu melengkapkan Borang tesebut dan mematuhi arahan yang\nterdapat di dalamnya untuk menukar had amaran transaksi melalui Paparan Notifikasi Aplikasi HLB\nConnect atau SMS atau menarik diri daripada perkhidmatan amaran transaksi melalui Paparan\nNotifikasi Aplikasi HLB Connect atau SMS tersebut.\n13.0\nPematuhan Syariah\n13.1\nApabila Akaun Pembelian Runcit yang dipautkan dengan Kad Debit adalah Akaun Simpanan atau\nAkaun Semasa-i, Kad Debit hanya boleh digunakan untuk pembelian barang dan perkhidmatan\nyang mematuhi Syariah. Kad Debit tidak boleh digunakan di mana-mana peniaga runcit yang\nmenyediakan barang dan perkhidmatan yang tidak mematuhi Syariah dan/atau untuk apa-apa"
  },
  {
    "page": 18,
    "content": "transaksi yang tidak mematuhi Syariah yang dikategorikan mengikut Kategori peniaga seperti yang\nberikut:\n(i) Bar, Ruang Koktel, Disko, Kelab Malam dan Kedai Minuman Keras\n(ii) Pakej Bir, Wain dan Minuman Keras\n(iii) Kedai dan gerai cerut\n(iv) Tranksaksi Perjudian\n(v) Petaruhan-Perlumbaan Kuda, Perlumbaan Anjing, Petaruhan Bukan Sukan Dalam Talian\nAntara Negeri\n(vi) Perkhidmatan Janji Temu dan Teman Social\nNota: Senarai peniaga runcit dan aktiviti tidak mematuhi Syariah di atas akan dikemas kini dari\nsemasa ke semasa.\n14.0\nKewajipan Pemegang Kad\n14.1\nKecuali jika kerugian tersebut disebabkan oleh salah laku sengaja atau kecuaian melampau oleh\nBank, Pemegang Kad dengan ini bersetuju dan wajib untuk membayar balik kepada Bank atas\npermintaan bertulis Bank semua tuntutan, kerugian dan perbelanjaan (termasuk kos perundangan\nsebagaimana peguam cara dengan pelanggan sendiri) dan semua liabiliti dalam apa jua bentuk atau\njenis yang ditanggung atau dialami oleh Bank berhubung dengan atau apa jua cara yang timbul\ndaripada peruntukan Perkhidmatan atau penerimaan sebarang arahan yang diberikan oleh\nPemegang Kad atau pelanggaran mana-mana Terma dan Syarat ini oleh Pemegang Kad. Liabiliti\nPemegang Kad adalah merupakan liabiliti berterusan dan akan kekal berkuat kuasa sepenuhnya\nsehingga liabiliti Bank, jika ada, dilepaskan sepenuhnya.\n15.0\nPenggangtungan, Pembatalan dan Penamatan\n15.1\nBank berhak menarik atau menggantung mengikut budi bicaranya Kad Debit dan/atau mana-mana\nPerkhidmatan dan dalam keadaan seperti itu jika Bank dianggap sesuai untuk menghentikan\npenggunaan Kad Debit oleh Pemegang Kad dengan pemberitahuan bertulis kepada Pemegang\nKad. Selanjutnya disepakati bahawa Bank tidak berkewajiban untuk menyatakan alasan\npenamatan penggunaan Kad Debit.\n16.0\nBudi Bicara Bank\n16.1\nBank berhak pada bila-bila masa menurut budi bicaranya tidak akan meluluskan sebarang\nTransaksi Kad walaupun Baki Semasa di dalam Akaun Pemegang Kad membolehkan mana-mana\nTransaksi Kad (iaitu terdapat dana yang mencukupi di dalam Akaun).\n16.2\nBank berhak untuk:\n(a) Menggantung hak Pemegang Kad untuk menggunakan Kad Debit sepenuhnya atau\nberkenaan keistimewaan tertentu.\n(b) Menolak untuk menerbit-semula, memperbaharui atau mengganti Kad Debit, tanpa dalam apa\njua perkara, menjejaskan tanggungjawab Pemegang Kad di bawah T&S ini yang akan terus\nberkuatkuasa dan tiada pengembalian akan dibuat untuk sebarang fi tahunan atau fi lain yang\ntelah dibayar jika hak untuk menggunakan Kad Debit digantung oleh Bank atau jika Kad Debit\ntidak diperbaharui atau diganti.\n16.3\nTanpa prejudis kepada peruntukan lain di dalam T&S ini, Bank berhak pada bila-bila masa dengan\nmemberi dua puluh satu (21) hari kalendar terlebih dahulu makluman awal, untuk\nmemperkenalkan, menukar, mengubah, mengehad atau menarik balik semua atau mana-mana"
  },
  {
    "page": 19,
    "content": "manfaat, perkhidmatan, kemudahan atau kelebihan yang berkaitan dengan atau berhubung\ndengan penggunaan Kad Debit dan/atau Terma dan Syarat ini.\n17.0\nAkaun Gantung\n17.1\nUntuk tujuan membolehkan Bank mengekalkan keseluruhan liabiliti Pemegang Kad, apabila writ\nsaman telah dikeluarkan atau untuk membuktikan kebankrapan atau ketakmampuan bayar\nPemegang Kad atau untuk tujuan lain yang dianggap wajar oleh Bank, Bank boleh pada bila-bila\nmasa dan tempat, menyimpan untuk suatu tempoh yang wajar, sebarang wang yang diterima,\ndiperoleh semula atau direalisasi selaras dengan T&S ini atau di bawah mana-mana cagaran atau\njaminan untuk dikreditkan dalam akaun berasingan yang tidak menawarkan keuntungan di bawah\nnama Pemegang Kad sebagaimana dianggap wajar oleh Bank. Bank tidak mempunyai kewajipan\nsegera untuk memanfaatkan wang tersebut atau sebahagian daripadanya untuk atau ke arah\nmelunaskan jumlah perlu bayar dan terhutang kepada Bank.\n18.0\nTolak Selesai\n18.1\nPemegang Kad bersetuju bahawa Bank boleh, pada bila-bila masa dan dengan memberikan\nmakluman awal, menyatu dan menggabung semua atau mana-mana Akaun Pemegang Kad\ndengan Bank dalam apa jua bentuk dan di mana-mana lokasi, sama ada di dalam Ringgit Malaysia\natau mata wang lain, atau menolak selesai atau memindah sebarang jumlah yang sedia untuk\ndikredit ke dalam mana-mana Akaun tersebut termasuk akaun bersama di mana akaun bersama\ntersebut adalah di atas nama Pemegang Kad untuk atau ke arah melunaskan semua jumlah\nterhutang kepada Bank di bawah mana-mana akaun Pemegang Kad dengan Bank, dalam apa jua\nbentuk dan di mana-mana lokasi, sama ada di dalam Ringgit Malaysia atau mata wang lain dan\nBank boleh berbuat demikian tanpa mengambil kira baki di dalam akaun tersebut dan jumlah\nterhutang mungkin bukan di dalam mata wang yang sama, dan Pemegang Kad dengan ini\nmembenarkan Bank menguatkuasakan sebarang penyatuan, gabungan, tolak selesai atau\npemindahan dengan pertukaran mata wang yang sewajarnya pada kadar tukaran semasa Bank\nyang akan ditentukan oleh Bank mengikut budi bicaranya.\n19.0\nCaj-Balik\n19.1\nBank tidak boleh dipertanggungjawabkan atas sebarang tindakan atau pengabaian oleh mana-\nmana peniaga runcit atau dalam talian walau apapun puncanya, termasuk tetapi tidak terhad\nkepada keengganan menerima Kad Debit atau sebarang Penyata atau komunikasi lain yang dibuat\nberkaitan dengannya, atau sebarang kecacatan atau kekurangan pada produk atau perkhidmatan.\nSekiranya terdapat sebarang kecacatan atau kekurangan pada produk atau perkhidmatan,\nPemegang Kad hendaklah berhubung secara langsung dengan Peniaga Sah mengenai sebarang\ntuntutan atau pertikaian dan tidak akan mempertikaikan sebarang bayaran yang dibuat oleh Bank\ndari Akaun Pemegang Kad ke atas tuntutan atau pertikaian tersebut.\n19.2\nDalam keadaan tertentu, peraturan skim kad membenarkan Pemegang Kad untuk mencaj-balik\nTransaksi Kad kepada Peniaga Sah bagi transaksi yang telah dibuat (jika hak caj-balik disediakan)\ntertakluk kepada aduan atau pertikaian telah dibuat oleh Pemegang Kad kepada Bank secara lisan\natau bertulis dalam masa dua puluh satu (21) hari kalendar daripada Tarikh Pengeposan\nTransaksi Kad tersebut seperti dalam Penyata dan Pemegang Kad telah menyediakan maklumat\nyang diperlukan oleh Bank (iaitu: nama Pemegang Kad, Akaun yang terlibat, tarikh dan jumlah Kad\ntransaksi yang dipertikaian dan sebab Pemegang Kad percaya bahawa ia adalah Transaksi Kad\nyang dipertikaikan) untuk menyokong caj balik dalam tempoh masa yang ditetapkan."
  },
  {
    "page": 20,
    "content": "19.3\nSekiranya Bank melanjutkan tempoh masa untuk menyelesaikan siasatan selepas empat belas\n(14) hari kalendar dari tarikh laporan terhadap Transaksi Kad yang dipertikaikan, sama ada secara\nlisan atau bertulis, oleh Pemegang Kad kepada Bank, Bank boleh: sekurang-kurangnya,\nsementara mengkreditkan jumlah penuh transaksi yang dipertikaikan atau Ringgit Malaysia Lima\nRibu (RM5,000), yang mana lebih rendah (termasuk apa-apa keuntungan yang berkenaan), ke\ndalam Akaun Pembelian Runcit tidak lewat daripada empat belas (14) hari kalendar dari tarikh\nPemegang Kad mengemukakan maklumat yang dinyatakandalam Klausa 19.2 kepada Bank;\n(a) mengkredit baki Transaksi Kad yang dipertikaikan (termasuk apa-apa keuntungan yang\nberkenaan) tidak lewat daripada tiga puluh (30) hari kalendar daripada tarikh perkreditan\npertama dana sementara ke dalam Akaun Pembelian Runcit oleh Bank mengikut Klausa 19.3\n(a) di mana dana tersebut adalah kurang daripada amaun Transaksi Kad yang dipertikaikan;\ndan\n(b) membenarkan Pemegang Kad menggunakan sepenuhnya dana sementara yang dikreditkan.\n19.4\nSekiranya Pemegang Kad dikehendaki membayar jumlah transaksi atau Peniaga Sah telah\nmengembalikan jumlah transkasi tersebut secara langsung kepada Pemegang Kad, Bank berhak\nmendebitkan amaun berkenaan daripada Akaun Pembelian Runcit.\n19.5\nTanpa mengambilkira dan tanpa prejudis kepada sebutan umum Klausa di dalam T&S ini,\nPemegang Kad bersetuju bahawa:\n(a) penggunaan Kad Debit adalah atas risiko Pemegang Kad sendiri dan Pemegang Kad harus\nmenanggung semua risiko yang berkaitan atau yang timbul daripada penggunaan Kad Debit;\ndan\n(b) Pemegang Kad tidak boleh membuat atau cuba untuk membuat apa-apa dakwaan palsu,\npenipuan atau menyalahi undang-undang berkenaan dengan Transaksi Kad yang\ndipertikaikan sama ada secara langsung atau tidak langsung. Bank berhak pada bila-bila masa\nuntuk memulakan dan memulakan dan mengambil tindakan undang-undang dan/atau apa-apa\nprosiding lain sebagaimana yang Bank fikirkan perlu terhadap Pemegang Kad setelah Bank\nmenemui atau menyedari tentang apa-apa tuntutan yang palsu atau penipuan atau menyalahi\nundang-undang berkaitan dengan atau yang timbul daripada Transaksi Kad yang dipertikaikan.\n19.6\nBank mempunyai hak untuk mengambil tindakan perundangan dan/atau apa-apa prosiding lain\nyang dianggap perlu oleh Bank termasuk melaporkan apa-apa laporan yang sesuai atau perlu\ndengan pihak berkuasa yang berkaitan, terhadap Pemegang Kad yang melengahkan, menghalang\ndan/atau menyembunyikan maklumat penting daripada Bank, membuat atau cuba untuk membuat\ntuntutan palsu berkenaan dengan apa-apa transaksi, menyiarkan tuntutan palsu di media\ntradisional atau sosial, dan/atau membuat laporan polis palsu mengenai apa-apa transaksi.\n19.7\nTertakluk kepada kepatuhan Pemegang Kad kepada tanggungjawab yang dinyatakan dalam\nKlausa 19.2, sekiranya berlaku caj balik akibat aduan atau pertikaian oleh Pemegang Kad\nmengenai Transaksi Luar Negara (seperti yang ditakrifkan di bawah Klausa 19.1 di bawah) yang\ndilakukan dalam mata wang asing melalui Akaun Yang Diupayakan Dengan MCF, amaun yang\ndicaj balik akan dikreditkan ke dalam Akaun Yang Diupayakan Dengan MCF Pemegang Kad dalam\nmata wang transaksi asal.\n20.0\nTransaksi Luar Negara\n20.1\nPemegang Kad boleh menggunakan Kad Debit untuk melakukan Transaksi Kad dan Transaksi"
  },
  {
    "page": 21,
    "content": "Kad ATM di luar Malaysia (\u201cTransaksi Luar Negara\u201d) di mana terdapat Peniaga Sah dan/atau\nSaluaran Tunai yang dibenarkan dengan syarat bahawa Pemegang Kad telah memilih untuk\nmembenarkan Transaksi Luar Negara yang akan dilaksanakan pada Kad Debit yang berkaitan\nKlausa 20.1 di dalam ini.\n20.2\nSemua Transaksi Luar Negara melalui transaksi ATM dan Transaksi Kad adalah tertakluk kepada\nundang-undang sedia ada di negara di mana transaksi dilakukan. Untuk semua Transaksi Kad,\nkadar tukaran, di mana berkaitan, adalah kadar tukaran serta-merta semasa pada tarikh transaksi.\n20.3\nPemegang Kad boleh menggunakan Kad Debit untuk pengeluaran tunai melalui ATM tertentu yang\ndipasang di negara yang diluluskan sebagaimana ditetapkan oleh Bank dan/atau mana-mana\nVisa/Mastercard tertakluk bahawa semua caj yang dikenakan oleh Visa International/MasterCard\nInternational dan/atau Bank termasuk semua caj yang dinyatakan di dalam Klausa 20.5 dan 20.6.\n20.4\nSekiranya Pemegang Kad menggunakan Kad Debit di ATM dan outlet Peniaga di bawah rangkaian\nVisa/Mastercard di luar Malaysia, transaksi akan dicaj menggunakan matawang rasmi negara\ntersebut dan ditukar ke Ringgit Malaysia pada kadar tukaran yang mungkin ditentukan oleh Visa\n/Mastercard.\n20.5\nTransaksi Kad yang dibuat dalam mata wang selain dari Ringgit Malaysia akan ditukarkan kepada\nRinggit Malaysia pada tarikh ia diproses oleh Visa atau Mastercard pada kadar pertukaran asing\nsebagaimana yang ditentukan oleh Visa atau Mastercard. Fi tambahan berikut akan dikenakan ke\natas penukaran:\n(i) 1% fi transaksi dikenakan oleh Visa atau Mastercard; dan\n(ii) 1% penambahan kepada penukaran tukaran asing dikenakan oleh Bank\n20.6\nUntuk Transaksi Luar Negara tertentu, Pemegang Kad boleh memilih untuk menggunakan\nperkhidmatan DCC untuk menukarkan transaksi mata wang asing kepada Ringgit Malaysia. Jika\nDCC dipilih:\n(i) Kadar pertukaran asing yang digunakan oleh peniaga luar negara bagi urusniaga DCCmungkin\nlebih tinggi daripada kadar pertukaran yang ditentukan oleh Visa atau Mastercard.\n(ii) Pemegang Kad akan dikenakan fi transaksi sebanyak 1% oleh Visa atau Mastercard ke atas\namaun Ringgit Malaysia yang ditukarkan, yang merupakan fi yang dikenakan oleh Visa atau\nMastercard.\n20.7\nSekiranya Pemegang Kad memilih untuk melakukan transaksi dalam mata wang asing\nmenggunakan Akaun Yang Diupayakan Dengan MCF, transaksi tersebut akan berjaya sekiranya\nmata wang asing yang dipilih mempunyai baki yang mencukupi di dalam Akaun Yang Diupayakan\nDengan MCF. Transaksi tersebut tidak akan dikenakan fi yang dinyatakan dalam Klausa 20.5.\nWalau bagaimanapun, sekiranya mata wang asing yang dipilih oleh Pemegang Kad tidak\nmencukupi, transaksi tersebut akan disempurnakan dalam Ringgit Malaysia dan akan didebit dari\nbaki Ringgit Malaysia yang terdapat dalam Akaun Yang Diupayakan Dengan MCF Pemegang Kad\ndan fi yang dinyatakan dalam Klausa 20.5 akan dikenakan.\n20.8\nMeskipun\nadanya\nKlausa\n20.7,\nsekiranya\ninstitusi\nkewangan\npeniaga\nluar\nnegara\nmenyempurnakan / membenarkan Transaksi Luar Negara dalam Ringgit Malaysia, bukannya\ndalam mata wang asing yang dipilih oleh Pemegang Kad, transaksi tersebut akan didebit dari"
  },
  {
    "page": 22,
    "content": "Akaun Pembelian Runcit Pemegang Kad dalam Ringgit Malaysia walaupun Pemegang\nKadmempunyai baki mata wang asing yang mencukupi dalam Akaun Yang Diupayakan\nDengan MCF\n20.9\nBagi Transaksi Kad yang disempurnakan dari Akaun Yang Diupayakan Dengan MCF dalam mata\nwang asing:\n(i) sekiranya terdapat kembalian oleh Peniaga Sah dalam tempoh enam puluh (60) hari daripada\ntarikh transaksi, dan Bank berjaya memadankan kembalian tersebut kepada transaksi asal,\nkembalian tersebut akan dikreditkan ke dalam Akaun Yang Diupayakan Dengan MCF dalam\nmata wang transaksi asal. Sekiranya padanan tidak berjaya, kembalian tersebut akan\nditukarkan kepada Ringgit Malaysia pada kadar tukaran semasa Bank pada masa Bank\nmengkredit kembalian tersebut ke dalam Akaun Yang Diupayakan Dengan MCF Pemegang\nKad.\n(ii)\nsekiranya Peniaga Sah mengemukakan permintaan untuk kembalian selepas tempoh enam\npuluh (60) hari daripada tarikh transaksi, kembalian kepada Pemegang Kad akan\ndisempurnakan dalam Ringgit Malaysia and tukaran daripada mata wang yang berkenaan\nkepada Ringgit Malaysia adalah pada kadar tukaran semasa Bank pada masa Bank\nmengkredit kembalian tersebut ke dalam Akaun Yang Diupayakan Dengan MCF Pemegang\nKad.\n20.10\nPemegang Kad perlu mematuhi Notis FE yang dikeluarkan oleh BNM berkenaan dengan sebarang\ntransaksi, termasuk Transaksi Luar Negara, yang boleh didapati di laman sesawang BNM.\nPemegang Kad adalah bertanggungjawab atas sebarang pengingkaran atau ketidakpatuhan\nperaturan tersebut dan pindaan yang dibuat terhadapnya.\n21.0\nKeperluan Pilih Masuk bagi Transaksi Luar Negara dan Transaksi Tanpa Kad\n21.1\nPemegang Kad tidak dibenarkan untuk melaksanakan apa-apa Transaksi Luar Negara dan/atau\nTransaksi Tanpa Kad melainkan Pemegang Kad telah mendaftarkan arahannya untuk memilih\nmasuk untuk melaksanakan Transaksi Luar Negara dan/atau Transaksi Tanpa Kad melalui Hong\nLeong Connect, ATM Bank, kunjungi mana-mana cawangan Bank atau hubungi Pusat Panggilan\nBank di 03-7626 8899, jika tidak, Transaksi Luar Negara dan/atau Transaksi Tanpa Kad akan\nditolak.\n22.0\nPlatform Pembayaran atau Rangkaian\n22.1\nWalaupun Kad Debit Bank menyokong MyDebit yang hanya boleh digunakan di dalam Malaysia,\ndan Visa/Mastercard yang diterima di dalam Malaysia dan di luar negara, apabila Kad Debit\ndigunakan di Peniaga Sah di Malaysia, ia adalah tertakluk kepada pilihan rangkaian oleh Peniaga\nSah tersebut (iaitu sama ada MyDebit atau Visa/Mastercard) bagi tujuan memproses pembayaran.\n22.2\nPemegang Kad tidak dibenarkan untuk meminta Peniaga Sah yang dibenarkan untuk menukar\nrangkaian Kad Debit yang telah dipilih.\n23.0\nPengganti\n23.1\nT&S ini akan digunakan pakai untuk waris, wakil peribadi dan pengganti\nPemegang Kad dan Bank masing-masing."
  },
  {
    "page": 23,
    "content": "24.0\nUndang-undang dan Bidang Kuasa yang Terpakai\n24.1\nT&S ini akan tertakluk kepada dan ditafsirkan menurut undang-undang Malaysia tetapi dalam\nmenguatkuasakan haknya, Bank bebas untuk memulakan dan mengambil tindakan atau prosiding\natau selainnya terhadap Pemegang Kad dan mana-mana pihak di Malaysia dan/atau di tempat lain\nseperti yang dianggap sesuai oleh Bank dan Pemegang Kad bersetuju bahawa apabila apa-apa\ntindakan atau prosiding dimulakan dan diambil di Malaysia, ia hendaklah mematuhi bidang kuasa\neksklusif Mahkamah Malaysia dalam semua hal yang berkaitan dengan kewajipan dan liabiliti di\nbawah atau yang timbul daripada Akaun dan T&S ini.\nJika anda mempunyai sebarang pertanyaan mengenai T&S ini, anda boleh mendapatkan penjelasan\ndaripada kakitangan kami yang membantu anda. Sebagai alternatif, sila e-mel kepada kami di\nhlonine@hlbb.hongleong.com.my atau hubungi 03-7626 8899\n[TAMAT]"
  }
]
    
    # Test cases
    test_paragraphs = [
      """1.36 “Akaun Diaktifkan MCF” bermaksud Akaun Pemegang Kad di mana MCF (seperti yang ditakrifkan dalam Klausa 1.45 di sini) telah diupayakan, yang mungkin termasuk akaun yang dipautkanke Kad Debit. telah diaktifkan, yang mungkin termasuk Akaun Pembelian Runcit.
      1.37 “MEPS” bermaksud Malaysian Electronic Payment System Sdn Bhd, pembekal perkhidmatan antara bank di Malaysia.
      1.38 “Perjanjian Peniaga” bermaksud perjanjian yang ditandatangani di antara Peniaga Sah dan Bank.
      1.39 “Fi Penyata Bulanan” bermaksud fi bulanan yang dikenakan ke atas Pemegang Kad untuk penyata bulanan Kad Debit yang dicetak.
      1.40 "MOTO" ertinya Pesanan Mel Pesanan Telefon yang merupakan jenis Transaksi Kad-Tidak-Hadir di mana Perkhidmatan diselesaikan dan dibayar melalui telefon, mel, faks atau e-mel.
      1.41 “MyKad” bermaksud kad yang dikeluarkan kepada Pemegang Kad oleh Jabatan Pendaftaran Negara Malaysia kepada Pemegang Kad untuk mengenalpasti and megesahkan identiti Pemegang Kad.
      1.42 "MyDebit" bermaksud suatu skim kad debit domestik yang membenarkan Pemegang Kad untuk membuat bayaran bagi pembelian barangan dan/atau perkhidmatan dengan mendebit secara langsung dari Akaun Pembelian Runcit Pemegang Kad."""
    ]
    
    for paragraph in test_paragraphs:
        pages = document_processor.find_page_by_paragraph(content_list, paragraph, 0.8)
        if pages:
            if len(pages) == 1:
                print(f"Paragraph found on page {pages[0]}")
            else:
                print(f"Paragraph spans across pages: {', '.join(map(str, pages))}")
        else:
            print("Paragraph not found in the content list.")

Paragraph not found in the content list.


In [6]:
content = [
  {
    "page": 1,
    "content": "TERMA DAN SYARAT KAD DEBIT HONG LEONG (English Version)\nDikemaskini pada 6 Januari 2025\nSebagai pertimbangan Hong Leong Bank Berhad atau Hong Leong Islamic Bank Berhad (\"Bank\")bersetuju\nuntuk menyediakan Perkhidmatan dan bersetuju untuk mengeluarkan Kad Debit (seperti yang ditentukan\ndalam Klausa 1) termasuk Gabungan (sebagaimana didefinisikan dalam Klausa 1.2) kepada Pemegang\nKad atas permohonan Pemegang Kad, Pemegang Kad berjanji dan bersetuju bahawa penurunan\ntandatangan pada Kad Debit, penggunaan Kad Debit dan/atau pengekalan Akaun oleh Pemegang Kad\nakan membentuk perjanjian Pemegang Kad dengan terma dan syarat (\u201cT&S\u201d) di bawah.\nT&S ini hendaklah dibaca bersama sebagai keseluruhan dengan Terma dan Syarat Am Akaun Bank,\nTerma dan Syarat Penggunaan HLB Connect dan sebarang terma dan syarat akaun yang berkaitan, serta\nsyarat dan peraturan lain yang mengikat Bank. Definisi berikut berkuatkuasa kecuali dinyatakan sebaliknya:\n1.0\nDefinisi dan Interpretasi\n1.1\n\u201cAkaun\u201d bermaksud akaun atau akaun-akaun Pelanggan dengan HLB / HLISB dan hendaklah\ntermasuk mana-mana akaun baharu lain yang dibuka dari semasa ke semasa\n1.2\n\u201cGabungan\u201d bermaksud Kad Debit yang ditawarkan oleh institusi kewangan dalam perkongsian\ndengan institusi lain.\n1.3\n\u201cFi Tahunan\u201d bermaksud fi yang dikenakan ke atas Pemegang Kad setiap tahun.\n1.4\n\"Pihak Berkuasa Berkenaan\" bermaksud mana-mana kerajaan atau pihak berkuasa percukaian.\n1.5\n\u201cATM\u201d bermaksud mesin juruwang automatik:\n(a) yang dipasang oleh Bank atau mana-mana ahli Rangkaian ATM Kongsi (Shared ATM Network\n(SAN)); dan/atau\n(b) ditetapkan oleh Bank atau Visa/Mastercard,\nuntuk penggunaan Pemegang Kad.\n1.6\n\u201cTransaksi Kad ATM\u201d bermaksud penggunaan Kad Debit untuk pengeluaran tunai dan transaksi\nelektronik atau mana-mana kad yang mungkin diluluskan oleh Bank dari semasa ke semasa.\n1.7\n\u201cRangkaian Tunai Dibenarkan\u201d bermaksud cawangan, pejabat dan/atau lokasi yang ditetapkan\noleh Rangkaian ATM Kongsi, Visa/Mastercard untuk melaksanakan pengeluaran wang tunai.\n1.8\n\u201cPeniaga Sah\u201d bermaksud mana-mana peruncit atau perbadanan yang mana selaras dengan\nPerjanjian Peniaga bersetuju untuk menerima atau menyebabkan outletnya menerima Kad Debit\nuntuk\npembayaran\natau\nmenurut\nkepada\nperkiraan\nundang-undang\ndengan\nMyDebit/Visa/Mastercard bersetuju untuk menerima atau menyebabkan rangkaiannya untuk\nmenerima kemudahan yang ditawarkan bergabung atau berkongsi jenama Visa/Mastercard untuk\npembayaran."
  },
  {
    "page": 2,
    "content": "1.9\n\u201cTransaksi Auto Debit\u201d bermaksud bayaran berulang melalui Kad Debit untuk caj utiliti, insurans\natau takaful yang diluluskan oleh Bank sahaja.\n1.10\n\u201cBank\u201d bermaksud sama ada Hong Leong Bank Berhad (193401000023 (97141-X)) atau Hong\nLeong Islamic Bank Berhad (200501009144 (686191-W)) dan termasuk pewaris-namanya dan\npemegang serah-hak dibenarkan.\n1.11\n\u201cHari Perniagaan\u201d bermaksud hari apabila HLB / HLISB dibuka untuk perniagaan di semenanjung\nMalaysia, Sabah dan Sarawak (tidak termasuk hari kelepasan bank, negeri dan hari kelepasan am),\nmengikut mana yang berkenaan, dan hari apabila transaksi yang jenisnya dicadang bagi Akaun\ndijalankan.\n1.12\n\u201cPemegang Kad\u201d bermaksud Pemegang Kad Bank yang mana Kad Debit diterbitkan kepadanya.\n1.13\n\u201cFi Penggantian Kad\u201d bermaksud fi yang dikenakan ke atas Pemegang Kad jika kad hilang, dicuri\natau rosak.\n1.14\n\u201cTransaksi Kad\u201d bermaksud transaksi yang dilaksanakan dengan menggunakan Kad Debit untuk\nurus niaga tempatan dan luar negara; Transaksi dengan Kad secara bersemuka sertaTransaksi\nTanpa Kad dan Transaksi Tanpa Sentuhan secara tidak bersemuka (di mana berkenaan).\n1.15\n\u201cTransaksi Card-Present (CP)\u201d bermaksud pembayaran Transaksi Runcit di mana Pemegang Kad\ndan Kad Debit hadir secara fizikal di Peniaga Sah apabila pembayaran dilakukan.\n1.16\n\u201cTransaksi Card-Not-Present (CNP)\u201d bermaksud pembayaran Transaksi Runcit di mana\nPemegang Kad dan Kad Debit tidak hadir secara fizikal di Peniaga Sah apabila pembayaran\ndilakukan.\n1.17\n\u201cBaki Semasa\u201d bermaksud baki atau tunai terkini yang ada di dalam akaun simpanan atau\nsemasa Pemegang Kad.\n1.18\n\u201cFi Pengeluaran Tunai\u201d bermaksud fi yang dikenakan ke atas Pemegang Kad untuk pengeluaran\ntunai berjaya dari ATM.\n1.19\n\u201cTransaksi Tanpa Sentuhan\u201d bermaksud kaedah pembayaran yang cepat dan mudah yang\nmenggunakan teknologi komunikasi medan dekat (NFC) bagi membuat pembayaran dengan\nmenyentuh/mengimbas Kad Debit pada pembaca selamat.\n1.20\n\u201cKad Karakter\u201d merujuk kepada Kad Debit seperti Kad Debit Hello Kitty atau Transformers yang\ndikeluarkan oleh Bank.\n1.21\n\u201cHad Pengeluaran Tunai Harian\u201d bermaksud had maksimum harian dibenarkan yang ditetapkan\noleh Bank berhubung pengeluaran tunai melalui ATM.\n1.22\n\u201cHad Belian Dalam Talian Harian\u201d bermaksud had maksimum belian harian dalam talian\ndibenarkan yang ditetapkan oleh Bank dalam Klausa 12.4 di sini.\n1.23\n\u201cHad Belian Runcit Harian\u201d bermaksud had maksimum belian runcit harian dibenarkan yang\nditetapkan oleh Bank dalam Klausa 12.3 di sini."
  },
  {
    "page": 3,
    "content": "1.24\n\u201cHad Pemindahan Harian\u201d bermaksud had maksimum harian yang ditetapkan oleh Bank\nberkenaan dengan pemindahan wang melalui ATM dan/atau Hong Leong Connect.\n1.25\n\u201cKad Debit\u201d bermaksud Kad Debit MyDebit/Visa/Mastercard Hong Leong yang dikeluarkan oleh\nBank.\n1.26\n\u201cDCC\u201d merujuk kepada Penukaran Mata Wang Dinamik, iaitu perkhidmatan pilihan yang\nditawarkan oleh peniaga atau tersedia untuk pembelian luar negara tertentu seperti yang\nditentukan dalam Klausa 20.1 di bawah (termasuk Transaksi Kad ATM), yang memberi pilihan\npembayaran / pengeluaran tunai dalam Ringgit Malaysia (termasuk pembelian di luar negara dan\npembelian dalam talian dalam mata wang asing) untuk transaksi luar negara, sebagaimana yang\ndihuraikan dalam Klausa 20.\n1.27\n\u201cFi\u201d termasuk fi perlu dibayar oleh Pemegang Kad semasa permohonan, tahunan atau sebarang\njarak waktu yang mungkin ditentukan oleh Bank untuk menggunakan Perkhidmatan yang akan\ndidebit dari Akaun pada setiap tarikh ulangtahun Kad Debit dikeluarkan dan akan merangkumi\nsemua fi lain, caj perkhidmatan, komisen dan lain-lain bayaran yang dicaj oleh Bank di bawah T&S\nini. Bank berhak untuk mengubah Fi dengan memberi notis awal dua puluh satu (21) hari kalendar\nterlebih dahulu kepada Pemegang Kad.\n1.28\n\u201cPemegang Kad Biasa\u201d bermaksud Pemegang Kad bukan Perbankan Prioriti.\n1.29\n\u201cTarikh Baik Sehingga\u201d bermaksud tarikh luput yang dicetak pada Kad Debit.\n1.30\n\u201cBarangan dan Perkhidmatan yang mematuhi Syariah\u201d bermaksud barangan & perkhidmatan\nyang boleh diterima dan mematuhi Syariah.\n1.31\n\u201cHLB Connect App\u201d bermaksud Perbankan Mudah Alih Hong Leong Connect.\n1.32\n\u201cHong Leong Connect\u201d bermaksud perkhidmatan perbankan internet dan perkhidmatan\naplikasi mudah alih yang disediakan oleh HLB / HLISB.\n1.33\n\u201cPemegang Akaun Bersama\u201d bermaksud mana-mana Akaun yang ada dua (2) atau lebih nama\nbersama atau mempunyai dua (2) atau lebih yang diberi kuasa penandatangan.\n1.34\n\u201cMastercard\u201d bermaksud MasterCard International Incorporated, sebuah syarikat yang\ndikendalikan di bawah Negeri Delaware dengan pejabat dan pusat perniagaan utamanya di 2000\nPurchase Street, Purchase, N.Y. 10577-2509, Amerika Syarikat di mana Bank adalah ahli\ninstitusinya.\n1.35\n\"MCCS\" bermaksud Spesifikasi Kad Cip Malaysia, iaitu ialah skim kad debit EMV (Euro,\nMastercard, dan Visa) domestik yang menyokong kedua-dua aplikasi sentuhan dan tanpa\nsentuhan.\n1.36\n\u201cAkaun Diaktifkan MCF\u201d bermaksud Akaun Pemegang Kad di mana MCF (seperti yang ditakrifkan\ndalam Klausa 1.45 di sini) telah diupayakan, yang mungkin termasuk akaun yang dipautkanke Kad\nDebit. telah diaktifkan, yang mungkin termasuk Akaun Pembelian Runcit.\n1.37\n\u201cMEPS\u201d bermaksud Malaysian Electronic Payment System Sdn Bhd, pembekal perkhidmatan\nantara bank di Malaysia."
  },
  {
    "page": 4,
    "content": "1.38\n\u201cPerjanjian Peniaga\u201d bermaksud perjanjian yang ditandatangani di antara Peniaga Sah dan Bank.\n1.39\n\u201cFi Penyata Bulanan\u201d bermaksud fi bulanan yang dikenakan ke atas Pemegang Kad untuk\npenyata bulanan Kad Debit yang dicetak.\n1.40\n\"MOTO\" ertinya Pesanan Mel Pesanan Telefon yang merupakan jenis Transaksi Kad-Tidak-Hadir\ndi mana Perkhidmatan diselesaikan dan dibayar melalui telefon, mel, faks atau e-mel.\n1.41\n\u201cMyKad\u201d bermaksud kad yang dikeluarkan kepada Pemegang Kad oleh Jabatan Pendaftaran\nNegara Malaysia kepada Pemegang Kad untuk mengenalpasti and megesahkan identiti\nPemegang Kad.\n1.42\n\"MyDebit\" bermaksud suatu skim kad debit domestik yang membenarkan Pemegang Kad untuk\nmembuat bayaran bagi pembelian barangan dan/atau perkhidmatan dengan mendebit secara\nlangsung dari Akaun Pembelian Runcit Pemegang Kad.\n1.43\n\u201cKad Debit Multi Currency\u201d atau \u201cKad MCF\u201d atau \u201cKad Debit Multi Currency Visa Hong\nLeong\u201d merujuk kepada Kad Debit dengan asas warna hitam yang dikeluarkan oleh Bank kepada\npemegang Akaun Pay&Save Hong Leong dan hanya boleh dipautkan kepada Akaun Pembelian\nRuncit ke Akaun Pay&Save Hong Leong.\n1.44\n\u201cCiri Mata Wang Pelbagai\u201d atau \u201cMCF\u201d bermaksud mata wang asing yang disimpan dalam Akaun\nyang dibenarkan oleh Bank untuk MCF yang membolehkan Pemegang Kad, antara lain:\n(a) memegang mata wang asing yang dibenarkan oleh Bank dari semasa ke semasa;\n(b) melakukan Transaksi Luar Negara dalam mata wang asing yang dipilih oleh Pemegang Kad;\n(c) menukar satu (1) mata wang asing ke mata wang asing yang lain dalam Akaun Diaktifkan MCF\nPemegang Kad melalui cawangan Bank atau Hong Leong Connect.\nBagi mengelakkan keraguan, MCF hanya tersedia untuk Akaun Diaktifkan MCF yang dipegang\noleh Pemegang Akaun tunggal. Mata wang asing terkini yang diluluskan untuk MCF boleh didapati\nhttps://www.hlb.com.my/ms/personal-banking/deposits/savings-account/pay-and-save-\naccount.html.\n1.45\n\u201cPaparan Notifikasi HLB Connect App\u201d bermaksud ciri notifikasi tolak yang membolehkan\nPemegang Kad menerima notifikasi melalui HLB Connect App.\n1.46\n\u201cOne-Time Password (OTP)\u201d bermaksud kod 6-digit yang unik dan sekali guna yang dihantar\nmelalui SMS atau Kad OTP dalam Aplikasi HLB Connect untuk melakukan transaksi dalam talian\ntertentu atau tinkdakan.\n1.47\n\u201cHad Belian Dalam Talian\u201d bermaksud had transaksi yang terpakai ke atas Transaksi Kad yang\ndilaksanakan dalam talian melalui internet.\n1.48\n\u201cPembelian Petrol\u201d bermaksud semua transaksi yang dibuat dengan Kad Debit untuk\npembayaran petrol dan mana-mana pembayaran lain yang dilakukan di kiosk petrol.\n1.49\n\u201cPIN\u201d bermaksud kod berangka yang bertindak sebagai kata laluan yang digunakan untuk\nmengesahkan Pemegang Kad ke akaun mereka\n1.50\n\u201cTarikh Pengeposan\u201d bermaksud tarikh di mana sebarang Transaksi Kad yang dilakukan oleh"
  },
  {
    "page": 5,
    "content": "Pemegang Kad didebitkan dan direkodkan dalam Penyata.\n1.51\n\u201cPemegang Kad Perbankan Prioriti\u201d bermaksud Pemegang Kad dengan status Perbankan\nPrioriti yang diperakui oleh Bank.\n1.52\n\u201cNotis Privasi\u201d bermaksud dasar dan prinsip Bank yang berkaitan dengan pengumpulan,\npenggunaan dan penyimpanan maklumat peribadi individu dan entiti sedia ada dan prospektif yang\nberurusan dengan Bank yang mana mungkin akan dipinda dari semasa ke semasa dan diterbitkan\ndi laman sesawang Bank atau dengan cara yang dianggap wajar oleh Bank.\n1.53\n\u201cHad Belian Runcit\u201d bermaksud had transaksi yang dikuatkuasakan bagi Transaksi Kad termasuk\nTransaksi Tanpa Sentuhan yang dibuat di kedai runcit, atau mana-mana kedai lain di mana Kad\nDebit boleh diterima untuk pembayaran.\n1.54\n\u201cAkaun Pembelian Runcit\u201d bermaksud akaun yang dikaitkan dengan Kad Debit untuk membuat\nTransaksi Runcit.\n1.55\n\"Transaksi Runcit\" bermaksud transaksi yang menggunakan Kad Debit di Peniaga Sah di mana\nKad Debit boleh diterima untuk pembayaran di dalam dan luar negara termasuk Pembelian Dalam\nTalian, tambah nilai Touch 'n Go secara automatik, Pembelian Petrol, Auto Bill/transaksi urusniaga\nyang berulang dan tidak termasuk pengeluaran wang tunai, pemindahan dana, pembayaran yuran\ntahunan dan perkhidmatan lain/yuran pelbagai seperti yang ditakrifkan oleh Bank dari semasa ke\nsemasa dengan memberi notis terlebih dahulu.\n1.56\n\u201cFi Pencarian Rekod Jualan\u201d bermaksud fi yang dikenakan ke atas Pemegang Kad apabila\nPemegang Kad memohon untuk mendapatkan rekod jualan yang diisu oleh Peniaga Sah dari\nBank. Fi ini juga dikenakan semasa penyiasatan kes pertikaian apabila Pemegang Kad memohon\nuntuk melihat rekod jualan yang berkaitan.\n1.57\n\u201cKod Keselamatan\u201d bermaksud kod keselamatan yang diberikan oleh Bank kepada Pemegang\nKad untuk mengakses Perkhidmatan berkaitan yang merangkumi PIN (untuk Perkhidmatan ATM),\nIPIN (untuk Hong Leong Connect), HLB Connect Code (untuk Hong Leong Connect) dan termasuk\nlain-lain nama pengguna, kata laluan, nombor pengenalan peribadi, sijil digital atau lain-lain kod\nkeselamatan yang mungkin dikeluarkan oleh Bank dari semasa ke semasa untuk mengakses\nsemua atau mana-mana Perkhidmatan dan rujukan untuk terma \u201cKod Keselamatan\u201d\nakanbermaksud kod atau kod-kod keselamatan berkaitan dengan Perkhidmatan tersebut\nsebagaimanadiperlukan.\n1.58\n\u201cPerkhidmatan\u201d bermaksud perkhidmatan yang disediakan oleh Bank untuk memudahkan\nTransaksi Kad, Transaksi Tanpa Sentuhan, transaksi Cip dan PIN, Perbankan Telefon,\nPerkhidmatan Pembayaran Bil, atau mana-mana perkhidmatan elektronik, digital, visual, video\natau lain-lain seperti perkhidmatan baru atau tambahan yang ditawarkan oleh Bank dari semasa\nke semasa bagi membolehkan Pemegang Kad untuk melaksanakan transaksi perbankan dan lain-\nlain transaksi dan Perkhidmatan itu boleh ditambah, diubah atau ditarik balik oleh Bank pada bila-\nbila masa dan dari semasa ke semasa dengan memberi notis awal dua puluh satu (21) hari\nkalendar terlebih dahulu.\n1.59\n\u201cRangkaian ATM Kongsi\u201d bermaksud bank dan institusi kewangan yang mengambil bahagian di\ndalam Rangkaian ATM Kongsi yang dikendalikan oleh Payments Network Malaysia Sdn Bhd.\n1.60\n\u201cPenyata\u201d bermaksud penyata akaun mencerminkan urusniaga yang dilaksanakan dengan Kad"
  },
  {
    "page": 6,
    "content": "Debit.\n1.61\n\"Cukai\" bermaksud sebarang cukai semasa atau masa hadapan, langsung atau tidak langsung,\ncukai, levi, impos, duti, caj, yuran, sebarang bentuk potongan atau pegangan dalam sebarang\nbentuk, Malaysia atau asing, yang dikenakan oleh Pihak Berkuasa Berkenaan, termasuk tanpa\nhad, sebarang cukai penggunaan dan lain-lain cukai dengan apa sahaja nama yang dipanggil, dan\nsebarang faedah, denda atau penalti yang berkenaan dengannya.\n1.62\n\"Tarikh Transaksi\" bermaksud tarikh di mana Transaksi Pembelian Runcit tersebut\nmenggunakan Kad Debit di Peniaga Sah di mana Kad Debit adalah diterima untuk pembayaran.\n1.63\n\u201cResit Transaksi\u201d bermaksud slip bayaran yang berkaitan, borang atau kertas yang dibekalkan\nkepada/oleh Bank kepada/oleh Peniaga Sah atau dijana oleh ATM untuk tujuan merekod,\nmengesah dan membuktikan pembelian atau transaksi lain yang dilakukan oleh Pemegang Kad\nmelalui penggunaan Kad Debit yang akan dicajkan dan didebitkan daripada Akaun atau\npengesahan yang diberikan kepada Pemegang Kad untuk transaksi kewangan yang dilakukan\nmelalui Hong Leong Connect.\n1.64\n\u201cVisa\u201d bermaksud Visa International Service Association, sebuah syarikat yang dikendalikan di\nbawah Negeri Delaware dengan ibu pejabatnya di 3125 Clearview Way, San Mateo, CA 94402,\nAmerika Syarikat di mana Bank adalah ahli institusinya.\n1.65\nPerkataan yang bermaksud tunggal juga bermaksud majmuk dan sebaliknya serta serta perkataan\nyang bermaksud jantina maskulin hendaklah termasuk jantina feminin dan neuter dan sebaliknya.\n2.0\nTanggungjawab Pemegang Kad\n2.1\nDengan menandatangani Kad Debit, menggunakan Kad Debit dan/atau mengekalkan Akaun,\nPemegang Kad bersetuju untuk terikat sepenuhnya dengan T&S ini yang mungkin diubah oleh\nBank dari semasa ke semasa dengan memberi notis awal dua puluh satu (21) hari kalendar. Notis\nmengenai perubahan atau pindaan tersebut akan dimaklumkan kepada Pemegang Kad dalam\nsebarang bentuk atau kaedah sebagaimana yang ditetapkan oleh Bank.\n2.2\nPemegang Kad harus mengambil semua langkah wajar untuk memastikan dan mencegah\npenggunaan tanpa kebenaran dan secara penipuan dan meyalahi undang-undang ke atas\nPerkhidmatan atau sebahagian daripadanya dan Kod Keselamatan Pemegang Kad, termasuk\ntetapi tidak terhad kepada yang berikut:\n(a) Kod Keselamatan hendaklah dirahsiakan dan apabila Pemegang Kad menerima Kod\nKeselamatan ia hendaklah ditukar sertamerta apabila telah diterima dan dibaca, dan hanya\nboleh digunakan oleh Pemegang Kad sahaja dan bukan orang lain. Jika Kod Keselamatan\ntidak dikeluarkan kepada Pemegang Kad, Pemegang Kad dinasihatkan agar mencipta Kod\nKeselamatan sendiri sebagai syarat untuk mengakses Perkhidmatan.\n(b) Bahawa tidak ada orang lain selain daripada Pemegang Kad mempunyai akses atau mampu\nmempunyai akses kepada Kod Keselamatan untuk membolehkan dia melakukan sebarang\ntransaksi melalui mana-mana Perkhidmatan dan Kod Keselamatan tidak ditulis dalam apa jua\nbentuk atau cara termasuk tetapi tidak terhad kepada e-mel yang tidak diminta atau pada\nmana-mana laman sesawang selain daripada laman sesawang Bank yang boleh ditafsirkan\noleh pihak ketiga atau disimpan bersama atau didedahkan kepada mana-mana orang\n(termasuk pekerja Bank) di bawah apa-apa keadaan atau pada bila-bila masa."
  },
  {
    "page": 7,
    "content": "(c) Pemegang Kad tidak boleh mendedahkan Kod Keselamatan kepada mana-mana pihak di\nbawah apa jua sebab atau cara sama ada secara sukarela atau sebaliknya dan harus\nmengambil segala langkah wajar untuk mencegah Kod Keselamatan daripada diketahui oleh\npihak lain. Pemegang Kad memahami dan bersetuju bahawa kegagalan untuk mematuhi\nperaturan ini boleh mendedahkan Pemegang Kad kepada akibat kecurian dan/atau\npenggunaan tanpa kebenaran Kad Debit, yang mana Bank tidak akan bertanggungjawab.\nPemegang Kad mesti melaporkan sebarang pelanggaran Kod Keselamatan atau kehilangan\nKod Keselamatan kepada Bank dengan secepat yang mungkin, apabila Pemegang Kad\nmenyedari tentang pelanggaran atau kerugian masing-masing. Pemegang Kad dengan ini\nbersetuju untuk membayar balik dan membayar kepada Bank atas permintaan bertulis Bank\nsemua tuntutan dan liabiliti yang ditanggung oleh Bank yang timbul daripada penggunaan\ntanpa kebenaran tersebut.\n(d) Bahawa baki Akaun, Penyata Akaun dan Transaksi Kad diperiksa dan dipantau oleh\nPemegang Kad secara kerap dan teliti dan diperiksa setiap kali sebelum memberikan apa-apa\narahan kepada Bank. Pemegang Kad hendaklah memaklumkan kepada Bank dengan segera\ntentang apa-apa yang tidak dibenarkan, penipuan, menyalahi undang-undang urus niaga,\nkesilapan atau percanggahan dalam Akaun.\n(e) Pemegang Kad harus memastikan bahawa dana di dalam Akaun adalah mencukupi untuk\nmelaksanakan sebarang Transaksi Kad yang diarahkan oleh Pemegang Kad.\n(f) Pemegang Kad akan mencipta PIN sendiri pada ATM Bank untuk tujuan melaksanakan\ntransaksi ATM atau transaksi elektronik. Pemegang Kad dikehendaki untuk tidak mendedahkan\nPIN kepada mana-mana pihak ketiga dan akan bertanggungjawab kepada Bank untuk sebarang\nkemasukan debit di dalam Akaun dengan Bank yang diakibatkan oleh Transaksi Kad tanpa\nkebenaran.\n(g) Untuk melakukan pembelian dengan menggunakan Kad Debit di mana-mana Peniaga Sah,\nPemegang Kad boleh mengesahkan Transaksi Kad dengan (i) menandatangan Resit\nTransaksi yang disediakan oleh Peniaga Sah; (ii) memasukkan PIN pada terminal Tempat\nJualan; atau (iii) melaksanakan Transaksi Tanpa Sentuhan. Penerimaan Resit Transaksi atau\napa-apa dokumen pulangan dengan tandatangan Pemegang Kad oleh Bank (jika ada) akan\nmenjadi bukti yang mengikat dan muktamad bahawa Transaksi Kad dan amaun yang direkod\npadanya adalah dilakukan oleh Pemegang Kad selaras dengan T&S ini. Untuk pembelian atau\npenggunaan Kad Debit yang dibuat secara transaksi dalam talian, atau transaksi bukan\nberasaskan tandatangan termasuk tetapi tidak terhad kepada pembelian petrol, transaksi\ndalam talian, Pemegang Kad bersetuju bahawa resit pengesahan dan/atau penerimaan yang\ndikeluarkan oleh mana-mana Peniaga Sah atau sekutunya untuk transaksi tersebut akan\ndianggap sebagai bukti dokumen yang memuaskan sebagai penggunaan dan Pemegang Kad\nharus mematuhi T&S ini.\n(h) Walapun peruntukan yang dinyatakan di bawah Klausa 2.2(g) di atas, tandatangan/PIN bukan\nsatu-satunya syarat yang boleh didahulukan kepada liability Pemegang Kad berkenaan\ndengan perkhidmatan yang dibekalkan, dan Pemegang Kad akan bertanggungjawab untuk\nTransaksi Kad jika Bank berpendapat, berdasarkan bukti dokumen yang memuaskan, bahawa\nketidaksempurnaan tandatangan atau ketiadaan tandatangan/kemasukan PIN adalah\ndisebabkan oleh keadaan dan/atau kaedah transaksi yang digunakan atau akibat kesilapan\ndari pihak Pemegang Kad dan/atau Peniaga Sah dan ahli bank MyDebit/Visa/ Mastercard.\n(i) Pemegang Kad harus mematuhi semua keperluan, arahan, suruhan, dan garis panduan"
  },
  {
    "page": 8,
    "content": "tentang penggunaan Kad Debit yang dikeluarkan oleh Bank dari semasa ke semasa\nberkenaan dengan semua Perkhidmatan kepada Pemegang Kad.\n(j) Pemegang Kad bersetuju untuk menandatangan bahagian belakang Kad Debit sebaik sahaja\nkad tersebut diterima dan sebelum ia digunakan, sebagai langkah mencegah penggunaan\ntanpa kebenaran Kad Debit tersebut.\n(k) Kad Debit adalah sah sehingga Tarikh Baik Sehingga. Surat pembaharuan kepada Pemegang\nKad akan dikirimkan tiga puluh (30) hari sebelum Tarikh Baik Sehingga. Bank akan mengirim\nsurat pembaharuan kepada Pemegang Kad untuk memaklumkan Pemegang Kad untuk\nmengunjungi cawangan Bank yang terdekat atau mana-mana cawangan Bank untuk\nmemperolehi Kad Debit Penggantian. Pemegang Kad harus memastikan bahawa sebaik\nsahaja Kad Debit luput, ia dimusnahkan, dengan memotong dua secara melintang dan\nmengembalikannya kepada Bank untuk penggantian Kad Debit.\n(l) Kad Debit tidak boleh digunakan selepas pembatalan, tamat tempoh, atau pengeluaran atau\napabila Pemegang Kad tidak lagi menjadi Pemegang Kad Bank.\n(m) Pemegang Kad tidak boleh menggunakan Kad Debit untuk pengeluaran tunai, bayaran atau\npemindahan dana melainkan terdapat dana yang mencukupi di dalam Akaun. Sebaran\npengeluaran tunai, bayaran atau pindahan dana akan ditolak jika tidak terdapat dana yang\nmencukupi di dalam Akaun.\n(n) Sekiranya tiada terdapat dana mencukupi di dalam Akaun dan Pemegang Kad tidak\nmempunyai kemudahan overdraf atau mana-mana kemudahan kredit tersedia, semua\nTransaksi Kad di Peniaga Sah akan ditolak.\n(o) Pemegang Kad bersetuju untuk tidak menggunakan Kad Debit untuk sebarang aktiviti yang\nmenyalahi undang-undang. Jika Bank mendapati, mengesyaki atau mempunyai sebab untuk\nmempercayai bahawa Kad Debit telah digunakan untuk sebarang aktiviti yang menyalahi\nundang-undang, Bank boleh mengambil apa-apa tindakan dianggap sesuai untuk memenuhi\napa-apa kewajipan yang berkaitan dengan pencegahan apa-apa aktiviti haram termasuk tetapi\ntidak terhad kepada penipuan, pengubahan wang haram, aktiviti/pembiayaan pengganas,\nrasuah, penyalahgunaan wang dan/atau pengelakan cukai.\n(p) Sekiranya Pemegang Kad menjadikan Akaun Simpanan-i atau Akaun Semasa-i sebagai\nAkaun Pembelian Runcit, Pemegang Kad mengakui dan bersetuju bahawa Kad Debit tidak\nboleh digunakan untuk sebarang aktiviti atau tujuan tidak patuh Syariah seperti yang\ndinyatakan di Klausa 13 di bawah dan yang boleh menyebabkan transaksi tersebut ditolak.\n(q) Pemegang Kad dengan ini memberi representasi dan memberi waranti bahawa Pemegang\nKad telah mendapat persetujuan semua orang yang dinamakan dalam permohonan\nPemegang Kad untuk Akaun atau apa-apa dokumen lain yang dihantar kepada Bank bagi\nmenyokong permohonan itu dan/atau wakil yang diberi kuasa mereka, termasuk tetapi tidak\nterhad kepada pengarah, pemegang saham, penandatangan yang diberi kuasa Pemegang\nKad atau mana-mana orang lain yang ditetapkan oleh Bank (\"Subjek Data Relevan\"), untuk\nkoleksi Bank, pegangan dan penggunaan maklumat peribadi daripada subjek Data Relevan\nmengikut Notis Privasi Bank sebagaimana yang dipinda dari semasa ke semasa.\n(r) Sekiranya Pemegang Kad menggantikan Kad Debit beliau dengan Kad Debit baru kerana\nhilang/dicuri/pembaharuan atau apa-apa sebab lain, Pemegang Kad bertanggungjawab untuk\nmemaklumkan kepada semua Peniaga Sah berkaitan yang beliau telah melanggan auto\ndebit/transaksi debit langsung untuk mengemaskini nombor Kad Debit baru untuk"
  },
  {
    "page": 9,
    "content": "mengelakkan transaksi daripada ditolak.\n(s) Pemegang Kad dengan ini bersetuju dan memberi kebenaran kepada pegangan,\npengumpulan dan penggunaan semua data peribadi yang diberikan kepada Bank oleh\nPemegang Kad dan/atau diperoleh oleh Bank dari domain awam, dan juga data peribadi yang\ntimbul akibat daripada penyediaan perkhidmatan kepada Pemegang Kad berhubung dengan\nAkaun-Akaun yang selaras dengan Notis Privasi Bank sebagaimana yang dipinda dari semasa\nke semasa.\n(t) Pemegang Kad yang ingin dikecualikan daripada pekongsian data peribadi mereka di dalam\nBank dan/atau kumpulan syarikat Hong Leong Financial Group Berhad bagi tujuan pemasaran\ndan promosi perlu mengunjungi mana-mana cawangan Bank atau menghubungi Pusat\nPanggilan Bank di 03-7626 8899 untuk mendaftar permintaan mengecualikan pekongsian\ntersebut.\n(u) Pemegang Kad hendaklah segera memberitahu Bank jika ada apa-apa perubahan dalam\nbutir-butir perhubungan dan/atau peribadi Pemegang Kad sama ada secara bertulis,\nmengunjungi cawangan-cawangan Bank atau menghubungi Pusat Hubungan Bank di 03-7626\n8899.\n(v) Bank tidak menyatakan atau menjamin bahawa penggunaan Kad Debit tidak akan berlaku\ntanpa gangguan, gangguan atau kesilapan. Bank tidak akan bertanggungjawab ke atas apa-\napa kerugian, kerosakan, kesulitan, keaiban, kos atau perbelanjaan dalam apa jua bentuk yang\ndalam apa-apa cara mungkin dialami atau ditanggung oleh Pemegang Kad atau oleh mana-\nmana orang lain berkenaan dengan atau berkaitan dengan penggunaan Perkhidmatan\ndisebabkan oleh sebarang kejadian di luar kawalan munasabah Bank dan Pemegang Kad\ntidak akan mempunyai tuntutan terhadap Bank jika Transaksi Kad dan/atau Transaksi Tanpa\nSentuhan tidak diterima atau dihormati oleh mana-mana Peniaga Sah atau atas apa jua sebab\nsekalipun.\n(w) Pemegang Kad mengakui dan bersetuju bahawa penggunaan Kad Debit atau prestasi\nTransaksi Kad dan/atau Transaksi Tanpa Sentuhan adalah atas risiko mutlak Pemegang Kad.\nBank tidak akan bertanggungjawab ke atas sebarang kerosakan dan/atau kerugian yang\nditanggung berhubung dengan penggunaan Kad Debit atau prestasi Transaksi Kad dan/atau\nTransaksi Tanpa Sentuhan bagi barangan atau perkhidmatan yang dibekalkan oleh mana-\nmana Peniaga Sah atau kualiti atau pelaksanaan apa-apa barangan atau perkhidmatan\nselaras dengan atau berhubung dengan apa-apa transaksi yang dibuat pada Kad Debit.\nPemegang Kad akan menyelesaikan secara langsung dengan Peniaga Sah bagi apa-apa\naduan yang timbul daripada Transaksi Kad dan/atau Transaksi Tanpa Sentuhan.\n(x) Pemegang Kad harus memastikan bahawa jika beliau mempunyai lebih daripada satu (1) kad\natau instrumen pembayaran dengan teknologi tanpa sentuh, iaitu instrumen pembayaran\ndengan keupayaan untuk melaksanakan Transaksi Tanpa Sentuhan, Pemegang Kad harus\nmemastikan bahawa kad tersebut disimpan secara berasingan daripada Kad Debit untuk\nmengelakkan urus niaga perlanggaran yang akan menyebabkan kedua-dua kad didebitkan\nuntuk jumlah urus niaga yang berkaitan.\n(y) Di mana debit atau kedudukan terlebih pengeluaran yang tidak diatur sebelumnya timbul dalam\nakaun semasa atas apa jua sebab, Pelanggan bertanggungjawab dan hendaklah membayar\namaun terlebih keluar ditambah faedah yang boleh dikenakan iaitu 4.0% p.a. melebihi BLR pada\nbaki harian atas baki yang terlebih keluar."
  },
  {
    "page": 10,
    "content": "(z) Pemegang Kad harus memastikan semua Transaksi Kad (termasuk Transaksi Luar Negara\nseperti yang ditakrifkan di bawah Klausa 20.1 di bawah) mematuhi Notis Dasar Pertukaran\nAsing (\u201cNotis FEP\u201d). Pemegang Kad disarankan untuk membaca dan memahami Notis FEP\ndi laman sesawang Bank Negara Malaysia (\u201cBNM\u201d) di http://www.bnm.gov.my \u2013 tab \u2018Notis\nDasar Pertukaran Asing\u2019 untuk memastikan semua aktiviti yang dilakukan menggunakan Kad\nDebit mematuhi Notis FEP setiap masa.\n3.0\nTransaksi Tanpa Sentuhan\n3.1\nTerdapat tiga (3) jenis Transaksi Tanpa Sentuhan iaitu:\n(a) MCCS Tanpa Sentuhan \u2013 kaedah pembayaran tanpa sentuhan melalui rangkaian MyDebit\nyang boleh dilakukan di mana-mana terminal Point-of-Sales yang memaparkan logo MCCS\nTanpa Sentuhan;\n(b) Visa Tanpa Sentuhan \u2013 kaedah pembayaran tanpa sentuhan melalui rangkaian Visa yang\nboleh dilakukan di mana-mana terminal Point-of-Sales yang memaparkan logo Visa Tanpa\nSentuhan; dan\n(c) Mastercard Tanpa Sentuhan \u2013 kaedah pembayaran tanpa sentuhan melalui rangkaian\nMastercard yang boleh dilakukan di mana-mana terminal Point-of-Sales yang memaparkan\nlogo Mastercard Tanpa Sentuhan.\n3.2\nTransaksi Tanpa Sentuhan tanpa pengesahan PIN dihadkan kepada Ringgit Malaysia Dua Ratus\nLima Puluh (RM250) setiap transaksi (\u201cHad Transaksi Tanpa Sentuhan\u201d). Pemegang Kad akan\ndikehendaki mengesahkan PIN bagi Transaksi Tanpa Sentuhan melebihi Ringgit Malaysia Dua\nRatus Lima Puluh (RM250).\n3.3\nTransaksi Tanpa Sentuhan tanpa pengesahan PIN boleh dilakukan secara berasingan tetapi\ntransaksi kumulatif dalam satu (1) hari hendaklah dihadkan kepada Ringgit Malaysia Lapan Ratus\n(RM800) sahaja atau had lain sebagaimana yang ditetapkan oleh Pemegang Kad mengikut Klausa\n3.4 (\u201cHad Kumulatif Transaksi Harian Tanpa Sentuhan\u201d). Apa-apa Transaksi Tanpa Sentuhan\ntanpa pengesahan PIN yang melebihi Had Kumulatif Transaksi Harian Tanpa Sentuhan akan\ndibatalkan dan Pemegang Kad akan diminta untuk melakukan pengesahan PIN untuk meneruskan\nbayaran. Had Kumulatif Transaksi Harian Tanpa Sentuhan akan ditetapkan semula setiap hari atau\napabila Pemegang Kad melakukan transaksi disahkan PIN pada terminal Point-of-Sales.\n3.4\nHad Kumulatif Transaksi Harian Tanpa Sentuhan ditetapkan kepada Ringgit Malaysia Lapan Ratus\n(RM800). Pemegang Kad boleh menukarkan Had Kumulatif Transaksi Harian Tanpa Sentuhan\nmereka bila-bila masa dari RM0 \u2013 RM800 melalui Hong Leong Connect atau di mana-mana\ncawangan Bank di seluruh negara. Jika Pemegang Kad tidak ingin menggunakan fungsi Transaksi\nTanpa Sentuhan tanpa pengesahan PIN, Had Kumulatif Transaksi Harian Tanpa Sentuhan\nhendaklah ditetapkan kepada sifar, iaitu RM0.\n3.5\nHad Kumulatif Transaksi Harian Tanpa Sentuhan Pemegang Kad adalah subset Had Belian Runcit\nPemegang Kad. Oleh sebab itu, Had Kumulatif Transaksi Harian Tanpa Sentuhan tidak akan lebih\ndaripada Had Belian Runcit yang ditetapkan oleh Pemegang Kad. Had Kumulatif Transaksi Harian\nTanpa Sentuhan Pemegang Kad secara automatik akan ditetapkan kepada Had Belian Runcit\nsekiranya Had Belian Runcit kurang daripada Had Kumulatif Transaksi Harian Tanpa Sentuhan.\nContohnya, jika Pemegang Kad telah menetapkan Had Belian Runcit kepada Ringgit Malaysia\nSeratus (RM100), Had Kumulatif Transaksi Harian Tanpa Sentuhan akan menjadi Ringgit Malaysia\nSeratus (RM100)."
  },
  {
    "page": 11,
    "content": "4.0\nTransaksi Pembelian Dalam Talian\n4.1\nApabila Pemegang Kad membuat pembelian dalam talian dengan Kad Debit, Pemegang Kad akan\nmenerima OTP (One-Time Password) 6-digit dengan salah satu cara berikut:\n(i)\nMelalui OTP Kad Debit dalam HLB Connect App (jika Pemegang Kad berdaftar) atau\n(ii)\nMelalui SMS (jika Pemegang Kad tidak berdaftar dengan HLB Connect App)\n4.2\nUntuk melengkapkan pembelian dalan talian, Pemegang Kad dikehendaki::\n(i)\nSemak Kad Debit OTP melalui HLB Connect App atau SMS; dan\n(ii)\nMasukkan kod OTP 6-digit pada halaman pembayaran Peniaga\n5.0\nPenyata Pemegang Kad\n5.1\nPenyata akan dikemas kini ke dalam penyata/buku Akaun Simpanan/Akaun Simpanan-i atau\nAkaun Semasa/Akaun Semasa-i yang ditetapkan, dan Pemegang Kad boleh menyemak\nPenyatanya secara percuma melalui Hong Leong Connect yang meliputi, antara lain, Transaksi\nKad dan Tarikh Pengeposan Transaksi Kad yang dilakukan oleh Pemegang Kad untuk tempoh\nyang dinyataka.\n5.2\nAtas permintaan oleh Pemegang Kad, Penyata bulanan bercetak akan dihantar ke alamat surat-\nmenyurat Pemegang Kad yang terakhir dicatatkan di dalam rekod Bank. Caj perkhidmatan bulanan akan\ndidebitkan dari dalam Akaun. Penyata bulanan yang dicetak menunjukkan semua TransaksiKad,\nTarikh Pengeposan dan Tarikh Transaksi untuk bulan yang berkenaan. Pemegang Kad boleh\nmeminta Penyata dicetak pada bila-bila masa dan Bank akan mengenakan bayaran perkhidmatan\nsebanyak Ringgit Malaysia Sepuluh (RM10) dan Ringgit Malaysia Dua (RM2) bagi setiap halaman\nuntuk Penyata sehingga satu (1) tahun yang lalu atau Ringgit Malaysia Sepuluh (RM10) dan\nRinggit Malaysia Lima (RM5) bagi setiap halaman untuk Penyata lebih daripada satu (1) tahun\nyang lalu untuk setiap permintaan.\n5.3\nRekod dan catatan dalam Akaun yang ditetapkan dengan Bank yang terdapat pada Penyata\nbulanan akan dianggap sebagai tepat dan mengikat ke atas Pemegang Kad kecuali makluman\nbertulis bertentangan dengannya diberi oleh Pemegang Kad kepada Bank dalam masa empat\nbelas (14) hari kalendar selepas menerima.\n5.4\nJika atas sebarang sebab Pemegang Kad tidak memaklumkan kepada Bank secara bertulis dalam\nmasa empat belas (14) hari tentang sebarang percanggahan di dalam Penyata dan sekiranya\ntiada sebarang kesilapan yang jelas pada muka penyata atau penipuan oleh Bank,maka\nPemegangKad akan dianggap telah menerima rekod dan catatan di dalam Penyata sebagai betul,\nakhir danmuktamad. Penyata itu harus dianggap sebagai muktamad dan terikat kepada Pemegang\nKad, wakil perundangan dan pengganti Pemegang Kad, dan sebarang tuntutan atau dakwa\nterhadap Bank yang mendakwa Penyata itu adalah salah adalah tidak sah.\n5.5\nApabila Bank menerima notis bertulis yang diberi oleh Pemegang Kad dalam tempoh yang\ndinyatakan, Bank akan menyiasat Akaun untuk membuat pelarasan dan pembetulan yang\nsewajarnya, jika ada. Sebarang wang yang perlu dibayar kepada atau dibayar oleh Pemegang Kad\nakan dikredit atau didebit ke dalam Akaun berkaitan.\n5.6\nHanya Transaksi Kad yang telah disahkan dengan sewajarnya oleh Bank akan muncul di dalam\nPenyata Akaun Pemegang Kad.\n5.7\nPemegang Kad juga boleh mendapatkan rekod sepuluh (10) transaksi terakhir Akaun Simpanan"
  },
  {
    "page": 12,
    "content": "atau Semasa/Akaun Simpanan-i atau Semasa -i melalui ATM. Di dalam rekod transaksi ATM yang\ndikeluarkan oleh ATM, semua Transaksi Belian Runcit diwakili oleh perkataan tiga (3)-huruf \u201cPOS\u201d.\n6.0\nPemegang Akaun Bersama\n6.1\nUntuk Pemegang Akaun Bersama, salah seorang pemegang akaun boleh mengeluarkan arahan\ndan membenarkan Bank untuk melaksanakan sebarang Transaksi Kad ke atas akaun bersama.\nSemua Transaksi Kad yang dilakukan akan mengikat ke atas semua pemegang kad, yang\nbertanggungjawab secara bersama dan bersendiri. Mandat pemegang akaun bersama tidak akan\nditarik-balik kecuali dan sehingga pemegang akaun bersama yang mana mandatnya akan ditarik-\nbalik menyerahkan Kad Debitnya kepada Bank.\n7.0\nAkaun Pegangan\n7.1\nBank mungkin akan mendebit atau mengenakan pegangan ke atas Akaun Pembelian Runcit\n(\u201cAkaun Pegangan\u201d) untuk amaun Transaksi Kad sama ada pada hari ianya diserahkan kepada\nBank untuk bayaran atau pada hari Bank menerima makluman mengenai Transaksi Kad, yang\nmana lebih awal.\n7.2\nAkaun Pegangan ke atas akaun adalah sah untuk semua transaksi runcit yang dilakukan dengan\nPeniaga Sah yang memerlukan pegangan sedemikian. Jika Peniaga Sah meminta kebenaran\nuntuk sebarang Transaksi Kad, Bank mungkin akan mengenakan pegangan ke atas Akaun\nPegangan untuk amaun Transaksi Kad yang ditetapkan oleh Peniaga Sah and dipersetujui oleh\nPemegang Kad. Baki tersedia di dalam Akaun Pembelian Runcit untuk digunakan oleh Pemegang\nKad akan berkurangan sejajar dengan amaun yang dipegang.\n7.3\nTertakluk kepada Had Pembelian Runcit Harian dan Baki Semasa dalam Akaun Belian Runcit dan\nTerma & Syarat di sini, Bank hendaklah meletakkan Akaun Pegangan untuk pembelian petrol oleh\nPemegang Kad di dispenser bahan api automatik di stesen minyak di mana sistem akan secara\nautomatik memperuntukkan Ringgit Malaysia Dua Ratus (RM200). Amaun pra-kuasa tersebut\nakan dikeluarkan secara automatik sebaik sahaja bank pemeroleh menuntut nilai pembelian\nsebenar kepada bank dalam tempoh tiga (3) Hari Perniagaan atau tempoh lain yang mungkin\ndimaklumkan oleh Bank.\n7.4\nSelain daripada yang dinyatakan seperti Klausa 7.3 di atas, Bank mempunyai budi bicara untuk\nmenahan jumlah tersebut sehingga dua puluh satu (21) hari kalendar atau untuk tempoh yang\ndifikirkan sesuai. Bank hendaklah mendebitkan jumlah yang ditahan ke Akaun Belian Runcit\napabila Transaksi Kad yang sepadan dikemukakan kepada Bank untuk pembayaran. Bank\nhendaklah mengeluarkan jumlah yang ditahan jika Transaksi Kad yang sepadan tidak\ndikemukakan kepada Bank untuk pembayaran dalam tempoh yang dianggap sesuai oleh Bank.\nPemegang Kad selanjutnya dengan nyata bersetuju bahawa Bank berhak untuk menahan Akaun\nBelian Runcit dan mendebit Akaun Belian Runcit jika Transaksi Kad berkemungkinan akan atau\ndibentangkan untuk pembayaran kemudiannya oleh Pedagang Dibenarkan apabila tamat tempoh\ndua puluh satu satu (21) hari kalendar. Pemegang Kad bersetuju bahawa semua resit yang\ndikemukakan oleh Pedagang Sah untuk Urus Niaga Kad melalui Resit Transaksi termasuk resit\npengesahan dan pengakuan hendaklah dianggap sebagai bukti penggunaan muktamad dan\nkonklusif oleh Pemegang Kad.\n7.5\nApabila Bank menerima bukti jumlah transaksi sebenar daripada Peniaga Sah, Bank akan\nmelepaskan Jumlah Pra-Kuasa dan Akaun Pembelian Runcit Pemegang Kad akan didebitkan"
  },
  {
    "page": 13,
    "content": "dengan jumlah transaksi sebenar tersebut.\n7.6\nPemegang Kad tidak boleh memberhentikan pembayaran ke atas Transaksi Kad. Untuk\nmengelakkan keraguan, dengan ini dinyatakan bahawa hak Bank untuk mendebit Akaun\nPembelian Runcit berhubung sebarang Transaksi Kad tidak terhad kepada amaun yang diletakkan\ndalam pegangan yang berkaitan dengan Transaksi Kad dan Bank berhak untuk mendebit Akaun\nPembelian Runcit untuk amaun penuh atas Transaksi Kad. Bank juga berhak untuk menaikkan\namaun yang perlu dipegang berkaitan mana-mana Transaksi Kad yang berdenominasi dalam mata\nwang lain selain daripada Ringgit Malaysia jika Bank berpendapat amaun asal yang dipegang tidak\nmencukupi untuk membayar Transaksi Kad sepenuhnya apabila ditukar kepada mata wang asing\ntersebut.\n7.7\nPemegang Kad tidak boleh menggunakan jumlah yang ditahan, walau apa pun terma dan syarat\nlain yang mengatur Akaun Pembelian Runcit.\n7.8\nBank hanya akan mengkreditkan Akaun Pembelian Runcit dengan kembalian yang dibuat oleh\nPeniaga Sah atau penentuan yang berkaitan dengan mana-mana transaksi runcit/transaksi dalam\ntalian selepas Bank menerima baucer kredit yang dikeluarkan dengan betul. Kembalian ini\ntermasuk tetapi tidak terhad kepada Pemegang Kad memulangkan semula barangan kepada\nperuncit selepas pembelian.\n8.0\nPemilikan dan Kad Debit Hilang atau Dicuri\n8.1\nPemegang Kad harus mengambil segala langkah keselamatan untuk mengelakkan Kad Debit\ndaripada kehilangan atau kecurian dan Pemegang Kad tidak boleh meninggalkan Kad Debit tanpa\ndijaga atau mendedahkan PIN dan/atau butiran Kad Debit kepada mana-mana pihak ketiga.\nSekiranya berlaku kehilangan dan/atau kecurian Kad Debit dan/atau pendedahan PIN dan/atau\nbutiran kepada pihak yang tidak dibenarkan, Pemegang Kad apabila menyedarinya hendaklah\nmemaklumkan kepada Bank dengan secepat yang munasabah boleh dilaksanakan selepas itu\n(jika perkara tersebut berlaku di Malaysia) atau Visa Travel Service Centre atau mana-mana ahli\nMastercard atau sekutu terdekatnya (jika perkara tersebut berlaku di luar negara). Pemegang Kad\nmemahami sepenuhnya bahawa kegagalan untuk menjaga dan mengambil langkah keselamatan\nyang munasabah dalam penyimpanan Kad Debit boleh mendedahkan Pemegang Kad kepada\nrisiko kecurian dan/atau penggunaan tanpa kebenaran Kad Debit.\n8.2\nBank berhak bertindak atas arahan telefon yang diterima oleh Bank daripada Pemegang Kad\ndan/atau wakilnya yang diberikuasa oleh Pemegang Kad (jika berkenaan dan mengikut mana-\nmana yang berkenaan) (\"Orang yang Diberi Kuasa\") dengan niat yang baik dan mengikut operasi\nyang biasa. Berhubung dengan ini, Bank tidak akan bertanggungjawab untuk bertindak atas arahan\ntelefon yang diberikan oleh Orang yang Diberi Kuasa tersebut tidak kira keadaan yang wujud pada\nmasa arahan tersebut.\n8.3\nJika berlaku kecurian/kehilangan Kad Debit dan Kad Debit tersebut telah digunakan oleh orang\nyang tidak dibenarkan untuk sebarang Transaksi Kad, Pemegang Kad bersetuju bahawa beliau\nakan bertanggungjawab kepada Bank ke atas semua caj tanpa kebenaran yang ditanggung\ntermasuk caj kewangan untuk sebarang barangan atau perkhidmatan yang diberikan oleh Peniaga\nSah, akibat penggunaan Kad Debit oleh pihak yang tidak dibenarkan tersebut sehingga tarikh dan\nmasa penerimaan oleh Bank makluman dari Pemegang Kad yang dinyatakan di dalam Klausa 8.1\ndi atas dan tertakluk kepada terma dan syarat Klausa 8.6 di dalam ini.\n8.4\nPemegang Kad bersetuju untuk memberi kuasa kepada Bank supaya tidak memperakui Kad Debit"
  },
  {
    "page": 14,
    "content": "asal yang tersebut di atas sekiranya Kad Debit yang sama diserahkan kepada Bank untuk\npembayaran setelah penerimaan oleh Bank pemberitahuan Pemegang Kad yang dinyatakan di\nbawah Klausa 8.1 diatas.\n8.5\nSelepas pengeluaran penggantian Kad Debit asal atas permintaan Pemegang Kad, Pemegang Kad\nbersetuju untuk membayar balik dan membayar kepada Bank jika Kad Debit asal digunakan dalam\napa jua cara, yang mana Pemegang Kad bersetuju bahawa Bank boleh menolak atau mendebit\ndaripada Akaun Pemegang Kad untuk jumlah tersebut.\n8.6\nPemegang Kad tidak akan dipertanggungjawab ke atas urus niaga tanpa kebenaran kad-hadir\nyang memerlukan pengesahan PIN atau tandatangan yang telah disahkan atau penggunaan kad\ntanpa-sentuh, dengan syarat Pemegang Kad tidak:\n8.6.1 melakukan penipuan;\n8.6.2 tangguh dalam memaklum Bank secepat mungkin setelah diketahui kehilangan atau\npenggunaan tanpa kebenaran Kad Kredit;\n8.6.3 secara sukarela mendedahkan PIN kepada orang lain;\n8.6.4 mencatatkan PIN pada Kad Kredit atau pada apa-apa yang disimpan berdekatan dengan Kad;\n8.6.5 meninggalkan Kad Kredit atau apa-apa yang mengandungi Kad Kredit tanpa jagaan di mana-\nmana tempat yang boleh dilihat dan diakses oleh orang lain; atau\n8.6.6 secara sukarela membenarkan orang lain untuk menggunakan Kad.\nUntuk mengelakkan keraguan, transaksi tanpa kebenaran yang dinyatakan di atas bermaksud\ntransaksi pembelian runcit dan bukan transaksi ATM. Urus niaga ATM akan merangkumi kedua-\ndua pengeluaran dan transaksi wang antara bank yang dilakukan dengan Kad Debit. Adalah\ndipersetujui dengan nyata bahawa Bank tidak bertanggungjawab untuk memastikan Baki Semasa\nyang ditetapkan oleh Bank tidak melebihi.\n8.7\nBank boleh, atas permintaan Pemegang Kad, tetapi tanpa diwajibkan di sisi undang-undang,\nmengganti Kad Debit yang hilang, dicuri atau rosak dengan mengenakan bayaran fi. Kad Debit\ngentian adalah tertakluk kepada terma dan syarat di dalam T&S ini sebagaimana Kad Debit asal.\n8.8\nSetelah Kad Debit tamat tempoh atau apabila diminta oleh Bank selepas pembatalan, pemansuhan\natau penggantungan Kad Debit oleh Bank atau apabila diketahui Kad Debit selepas pemberitahuan\nkehilangannya, Pemegang Kad tidak mempunyai apa-apa hak lagi penggunaan Kad Debit dan\nakan segera dipotong setengah dan memulangkan Kad Debit tersebut kepada Bank.\n9.0\nPenamatan oleh Pemegang Kad\n9.1\nPemegang Kad boleh pada bila-bila masa, menamatkan penggunaan Kad Debit dengan\nmemberikan makluman bertulis kepada Bank dan mengembalikan Kad Debit yang dipotong dua\nkepada Bank. Tiada kembalian Fi atau sebahagiannya akan dibuat kepada Pemegang Kad dan\nPemegang Kad akan dan kekal bertanggungjawab ke atas sebarang transaksi yang dilakukan\nmenggunakan Kad Debit sebelum Bank menerima makluman bertulis mengenai penamatan dan\npengembalian Kad Debit dipotong dua kepada Bank.\n10.0\nFi, Komisyen dan Caj\n10.1\nBank berhak untuk mengenakan Fi untuk Perkhidmatan termasuk tetapi tidak terhad kepada\npengeluaran Kad Debit, Penyata bulanan dicetak, Yuran Penggantian Kad, Fi Pengeluaran Tunai\nmelalui rangkaian MEPS, terminal Visa PLUS/Mastercard Cirrus dan Bank Antarabangsa, Dapatan\nSemula Draf Jualan dan Fi Bayaran Tahunan. Selain daripada Bayaran Fi tersebut, transaksi yang\ndijalankan melalui Perkhidmatan akan dikenakan caj dan komisen biasa. Bagi tujuan pengutipan\nYuran, komisen dan/atau caj, Pemegang Kad bersetuju untuk memberi kuasa kepada Bank untuk"
  },
  {
    "page": 15,
    "content": "mendebitkan Akaun dengan Yuran, komisyen dan/atau caj yang berkenaan. Sekiranya Pemegang\nKad tidak memberi kuasa kepada Bank untuk mendebitkan Akaun dengan Yuran, komisen\ndan/atau caj, Pemegang Kad bersetuju bahawa kegagalan pembayaran Yuran, komisen dan/atau\ncaj bagi Perkhidmatan boleh menjejaskan Perkhidmatan yang disediakan sehingga semua yuran,\nkomisen dan/atau caj yang telah dibayar dengan sewajarnya oleh Pemegang Kad.\n10.2\nPemegang Kad bersetuju untuk membayar semua fi, komisen dan/atau caj yang dikenakan dalam\nklausa ini dan membenarkan Bank untuk mendebit Akaun Pemegang Kad, tanpa mengambilkira\npendebitan tersebut mungkin menyebabkan Akaun terlebihguna. Bayaran, komisen dan/atau caj\nberikut dikenakan pada kadar yang dinyatakan atau kadar lain yang ditetapkan, yang boleh dipinda\noleh Bank untuk membuat pemindahan dari semasa ke semasa dengan memberi dua puluh satu\n(21) hari kalendar notis terlebih dahulu kepada Pemegang Kad untuk transaksi yang dilaksanakan\nmelalui penggunaan Kad Debit. Untuk senarai fi dan caj yang lengkap, sila layari laman web kami\nwww.hlb.com.my/dc2 atau imbas di sini:\n10.3\nFi Tahunan tidak akan dikenakan semasa pengeluaran Kad Debit dan hanya akan dikenakan pada\ntarikh ulang tahun. Fi Tahunan boleh diubah oleh Bank dari semasa ke semasa. Fi Tahunan tidak\nakan dikembalikan.\n11.0\nCukai Kerajaan, Fi dan Caj Berkanun dan/atau Kawal Selia\n11.1\nSebarang yuran perkhidmatan dan sebarang pembayaran yang kena dibuat oleh Pemegang Kad\nkepada Bank dibawah T&S ini, termasuk sebarang jumlah yang mewakili pembayaran ganti untuk\ndibayar oleh Pemegang Kad kepada Bank, adalah tidak termasuk Cukai, dan perlu dijelaskan tanpa\nsebarang pengecualian, syarat dan tanpa sebarang potongan bagi sebarang jumlah tuntutan balas atau\nsebarang potongan atau sekatan.\n11.2\nPada sebarang masa atau keadaan dimana Pemegang Kad dituntut oleh undang-undang untuk\nmembuat sebarang potongan atau menahan dari yuran perkhidmatan dan/atau semua wang lain\nyang kena dibayar kepada di bawah T&S ini berkaitan dengan Cukai atau sebaliknya, jumlah yang\nkena dibayar oleh Pemegang Kad dalam hal di mana potongan atau penahanan cukai diperlukan\nakan ditambah supaya yuran perkhidmatan dan/atau jumlah bersih wang yang diterima oleh Bank\nkekal dengan apa yang Bank sepatutnya menerima jika sebarang potong atau pengecualian tidak\ndikenakan.\n11.3\nSebagai penambahan atas yuran perkhidmatan dan sebarang bayaran yang perlu dibayar,\nPemegang Kad kena bayar kepada Bank semua bayaran Cukai pada kadar terkini dan/atau jumlah\nyang\nditentukan\noleh\nBank\nuntuk\nmerangkumi\nsebarang\npembayaran\nCukai/liabilti\nCukai/kewajipan-kewajipan Cukai yang berhubung kait, tanpa sebarang penolakan, sekatan atau\nsyarat-syarat dan tanpa apa-apa potongan untuk atau sebarang jumlah tuntutan balas atau apa-\napa potongan atau pengecualian, selain sebarang Cukai-Cukai yang mana mungkin diperlukan\ndi bawah sebarang undang-undang kena dibayar oleh anda secara terus kepada Pihak Berkuasa\nYang Berkenaan, dimana anda akan membayar secara terus kepada Pihak Berkuasa Yang\nBerkenaan."
  },
  {
    "page": 16,
    "content": "11.4\nJika pada bila-bila masa satu pelarasan dibuat atau dikehendaki untuk dibuat antara Bank\ndan Pihak Berkuasa Berkenaan bagi sebarang jumlah Cukai berkaitan sebarang bekalan yang\ndilakukan atau dinilai sebagai telah berlaku, yang berkaitan dengan T&S ini daripada Bank atau\nsegala yang berkaitan dengannya, satu pelarasan, tertakluk pada budi bicara, boleh dibuat diantara\nBank and Pemegang Kad dan pada keadaan sedemikian, sebarang pembayaran berkaitan\npelarasan tersebut, haruslah dijelaskan sepenuhnya.\n11.5\nSemua Cukai yang mesti dibayar oleh Pelanggan kepada Bank seperti yang diperuntukkan di ini\nadalah perlu dibayar di masa-masa sedemikian dan di sedemikian cara seperti yang akan diminta\noleh Bank.\n11.6\nPemegang Kad dengan ini bersetuju untuk memenuhi segala permintaan oleh Bank bagi\nmembantu Bank dalam mematuhi dengan kewajipan-kewajipannya di bawah undang-undang\npercukaian yang berkaitan. Jika terdapat, Cukai baru yang telah berkuatkuasa dan Cukai tersebut\nadalah diwajibkan untuk dikenakan terhadap sebarang urus niaga yang termaktub di dalam T&S\nini, Pemegang Kad bersetuju untuk memberi kerjasama dengan sepenuhnya kepada Bank untuk\nmemudahkan Bank untuk mematuhi tanggungjawab Bank di bawah undang-undang tersebut.\n11.7\nBagi mengelakkan sebarang keraguan, kesemua pihak terbabit dengan ini bersetuju bahawa\nmana-mana jumlah yang perlu dibayar atau jumlah yang digunakan untuk menentukan jumlah yang\nperlu dibayar berdasarkan Terma dan Syarat ini adalah muktamad tanpa mengambil kira untuk\nCukai dan tidak termasuk jumlah-jumlah yang perlu ditambah di bawah Klausa ini bekenaan\ndengan Cukai.\n12.0\nHad Ke Atas Transaksi Kad\n12.1\nKecuali diarahkan sebaliknya oleh Pemegang Kad atau Pemegang Kad tidak memilih suatu had\nuntuk pelaksanaan transaksi melalui Perkhidmatan, Bank berhak untuk menentukan dan\nmenetapkan sebarang had sama ada dari segi amaun, kekerapan atau kegunaan mana-mana\nPerkhidmatan dan/atau Kod Keselamatan. Pemegang Kad boleh mengubah had ke atas Transaksi\nKad yang dilakukan dengan memberikan makluman awal secara bertulis kepada Bank. Bank tidak\nbertanggungjawab untuk memastikan Pemegang Kad tidak melebihi had yang ditetapkan\nmahupun mematuhi permintaan Pemegang Kad supaya Bank mengubah had tersebut.\n12.2\nHad Pengeluaran Tunai Harian untuk Pemegang Kad Biasa dan Perbankan Prioriti ditetapkan pada\nRinggit Malaysia Dua Ribu (RM2,000), dengan had maksimum dibenarkan sebanyak Ringgit\nMalaysia Sepuluh Ribu (RM10,000) untuk Pemegang Kad Biasa dan Pemegang Kad Perbankan\nPrioriti, atau suatu amaun lain yang ditentukan oleh Bank dari semasa ke semasa dengan memberi\ndua puluh satu (21) hari kalendar terlebih dahulu makluman awal kepada Pemegang Kad.\nPemegang Kad boleh melakukan penetapan Had Pengeluaran Tunai Harian di mana-mana\ncawangan Bank atau ATM atau melalui Hong Leong Connect.\n12.3\nHad Belian Runcit Harian untuk Pemegang Kad Biasa dan Perbankan Prioriti ditetapkan pada\nRinggit Malaysia Tiga Ribu (RM3,000), dengan had maksimum dibenarkan sebanyak Ringgit\nMalaysia Dua Puluh Ribu (RM20,000) untuk Pemegang Kad Biasa dan Pemegang Kad Perbankan\nPrioriti, atau suatu amaun lain yang ditentukan oleh Bank dari semasa ke semasa dengan\nmemberikan notis awal dua puluh satu (21) hari kepada Pemegang Kad. Pemegang Kad boleh\nmelakukan penetapan Had Belian Runcit Harian di mana-mana cawangan Bank atau melalui Hong\nLeong Connect. Selain daripada itu, Pemegang Kad boleh memohon Had Belian Runcit Harian\nyang lebih tinggi (\u201cHad Belian Runcit Harian Sementara\u201d) melalui Hong Leong Connect dan Had\nBelian Runcit Harian Sementara tersebut akan berkuat kuasa sepanjang tempoh masa tertentu"
  },
  {
    "page": 17,
    "content": "yang dipilih oleh Pemegang Kad (\u201cJulat Tarikh\u201d). Had Belian Runcit Harian Sementara\nmemberikan Pemegang Kad had maksimum yang dibenarkan iaitu Ringgit Malaysia Sembilan\nPuluh Sembilan Ribu Sembilan Ratus Sembilan Puluh Sembilan (RM99,999) semasa Julat Tarikh.\nHad Belian Runcit Harian Sementara akan ditamatkan apabila Julat Tarikh berakhir dan selepas\nitu, Had Belian Runcit akan kembali kepada Had Belian Runcit Harian.\n12.4\nHad Belian Dalam Talian Harian untuk Pemegang Kad Biasa dan Pemegang Kad Perbankan\nPrioriti ditetapkan pada Ringgit Malaysia Satu Ribu (RM1,000), dengan had maksimum dibenarkan\nsebanyak Ringgit Malaysia Dua Puluh Ribu (RM20,000) untuk Pemegang Kad Biasa dan\nPemegang Kad Perbankan Prioriti, atau suatu amaun lain yang ditentukan oleh Bank dari semasa\nke semasa dengan memberikan notis awal dua puluh satu (21) hari kalendar kepada Pemegang\nKad. Pemegang Kad boleh melakukan Had Belian Dalam Talian Harian di mana-mana cawangan\nBank atau melalui Hong Leong Connect. Selain daripada itu, berkuat kuasa 24 September 2022,\nPemegang Kad boleh memohon Had Belian Dalam Talian Harian yang lebih tinggi (\u201cHad Belian\nDalam Talian Harian Sementara\u201d) melalui Hong Leong Connect dan Had Belian Dalam Talian\nSeharian Sementara tersebut akan berkuat kuasa sepanjang tempoh masa tertentu yang dipilih\noleh Pemegang Kad (\u201cJulat Tarikh\u201d). Had Belian Dalam Talian Harian Sementara memberikan\nPemegang Kad had maksimum yang dibenarkan iaitu Ringgit Malaysia Tiga Puluh Ribu\n(RM30,000) semasa Julat Tarikh. Had Belian Dalam Talian Harian Sementara akan ditamatkan\napabila Julat Tarikh berakhir dan selepas itu, Had Belian Runcit akan Kembali kepada\nHad BelianDalam Talian Harian.\n12.5 Had Pemindahan Harian melalui ATM untuk Pemegang Kad Biasa dan Perbankan Prioriti\nditetapkan pada Ringgit Malaysia Dua Ribu (RM2,000), dengan had maksimum dibenarkan\nsebanyak Ringgit Malaysia Lima Puluh Ribu (RM50,000) untuk Pemegang Kad Biasa dan\nPerbankan Prioriti, atau suatu amaun lain yang ditentukan oleh Bank dari semasa ke semasa\ndengan memberikan makluman awal dua puluh satu (21) hari kalendar terlebih dahulu kepada\nPemegang Kad. Pemegang Kad boleh melakukan penetapan Had Pemindahan Harian di mana-\nmana cawangan Bank atau ATM daripada Bank.\n12.6 Bagi mengelakkan keraguan, Had Pemindahan Harian melalui ATM adalah berasingkan daripada\nHad Pemindahan Harian melalui Hong Leong Connect.\n12.7 Pemegang Kad yang ingin menukar had amaran transaksi melalui Paparan Notifikasi Aplikasi HLB\nConnect atau khidmat pesanan ringkas (\u201cSMS\u201d) dikehendaki membuat permintaan bertulis untuk\nmenukar had yang telah ditetapkan sebelum ini atau menarik diri daripada perkhidmatan amaran\ntransaksi melalui Paparan Notifikasi Aplikasi HLB Connect atau SMS tersebut. Pemegang Kad\nboleh mendapatkan Borang Penyelenggaraan Perkhidmatan Amaran Transaksi Kad Debit/Kad\nDebit-i (\u201cBorang\u201d) di mana-mana cawangan Bank atau di Laman Sesawang Bank di\nwww.hlb.com.my. Pemegang Kad perlu melengkapkan Borang tesebut dan mematuhi arahan yang\nterdapat di dalamnya untuk menukar had amaran transaksi melalui Paparan Notifikasi Aplikasi HLB\nConnect atau SMS atau menarik diri daripada perkhidmatan amaran transaksi melalui Paparan\nNotifikasi Aplikasi HLB Connect atau SMS tersebut.\n13.0\nPematuhan Syariah\n13.1\nApabila Akaun Pembelian Runcit yang dipautkan dengan Kad Debit adalah Akaun Simpanan atau\nAkaun Semasa-i, Kad Debit hanya boleh digunakan untuk pembelian barang dan perkhidmatan\nyang mematuhi Syariah. Kad Debit tidak boleh digunakan di mana-mana peniaga runcit yang\nmenyediakan barang dan perkhidmatan yang tidak mematuhi Syariah dan/atau untuk apa-apa"
  },
  {
    "page": 18,
    "content": "transaksi yang tidak mematuhi Syariah yang dikategorikan mengikut Kategori peniaga seperti yang\nberikut:\n(i) Bar, Ruang Koktel, Disko, Kelab Malam dan Kedai Minuman Keras\n(ii) Pakej Bir, Wain dan Minuman Keras\n(iii) Kedai dan gerai cerut\n(iv) Tranksaksi Perjudian\n(v) Petaruhan-Perlumbaan Kuda, Perlumbaan Anjing, Petaruhan Bukan Sukan Dalam Talian\nAntara Negeri\n(vi) Perkhidmatan Janji Temu dan Teman Social\nNota: Senarai peniaga runcit dan aktiviti tidak mematuhi Syariah di atas akan dikemas kini dari\nsemasa ke semasa.\n14.0\nKewajipan Pemegang Kad\n14.1\nKecuali jika kerugian tersebut disebabkan oleh salah laku sengaja atau kecuaian melampau oleh\nBank, Pemegang Kad dengan ini bersetuju dan wajib untuk membayar balik kepada Bank atas\npermintaan bertulis Bank semua tuntutan, kerugian dan perbelanjaan (termasuk kos perundangan\nsebagaimana peguam cara dengan pelanggan sendiri) dan semua liabiliti dalam apa jua bentuk atau\njenis yang ditanggung atau dialami oleh Bank berhubung dengan atau apa jua cara yang timbul\ndaripada peruntukan Perkhidmatan atau penerimaan sebarang arahan yang diberikan oleh\nPemegang Kad atau pelanggaran mana-mana Terma dan Syarat ini oleh Pemegang Kad. Liabiliti\nPemegang Kad adalah merupakan liabiliti berterusan dan akan kekal berkuat kuasa sepenuhnya\nsehingga liabiliti Bank, jika ada, dilepaskan sepenuhnya.\n15.0\nPenggangtungan, Pembatalan dan Penamatan\n15.1\nBank berhak menarik atau menggantung mengikut budi bicaranya Kad Debit dan/atau mana-mana\nPerkhidmatan dan dalam keadaan seperti itu jika Bank dianggap sesuai untuk menghentikan\npenggunaan Kad Debit oleh Pemegang Kad dengan pemberitahuan bertulis kepada Pemegang\nKad. Selanjutnya disepakati bahawa Bank tidak berkewajiban untuk menyatakan alasan\npenamatan penggunaan Kad Debit.\n16.0\nBudi Bicara Bank\n16.1\nBank berhak pada bila-bila masa menurut budi bicaranya tidak akan meluluskan sebarang\nTransaksi Kad walaupun Baki Semasa di dalam Akaun Pemegang Kad membolehkan mana-mana\nTransaksi Kad (iaitu terdapat dana yang mencukupi di dalam Akaun).\n16.2\nBank berhak untuk:\n(a) Menggantung hak Pemegang Kad untuk menggunakan Kad Debit sepenuhnya atau\nberkenaan keistimewaan tertentu.\n(b) Menolak untuk menerbit-semula, memperbaharui atau mengganti Kad Debit, tanpa dalam apa\njua perkara, menjejaskan tanggungjawab Pemegang Kad di bawah T&S ini yang akan terus\nberkuatkuasa dan tiada pengembalian akan dibuat untuk sebarang fi tahunan atau fi lain yang\ntelah dibayar jika hak untuk menggunakan Kad Debit digantung oleh Bank atau jika Kad Debit\ntidak diperbaharui atau diganti.\n16.3\nTanpa prejudis kepada peruntukan lain di dalam T&S ini, Bank berhak pada bila-bila masa dengan\nmemberi dua puluh satu (21) hari kalendar terlebih dahulu makluman awal, untuk\nmemperkenalkan, menukar, mengubah, mengehad atau menarik balik semua atau mana-mana"
  },
  {
    "page": 19,
    "content": "manfaat, perkhidmatan, kemudahan atau kelebihan yang berkaitan dengan atau berhubung\ndengan penggunaan Kad Debit dan/atau Terma dan Syarat ini.\n17.0\nAkaun Gantung\n17.1\nUntuk tujuan membolehkan Bank mengekalkan keseluruhan liabiliti Pemegang Kad, apabila writ\nsaman telah dikeluarkan atau untuk membuktikan kebankrapan atau ketakmampuan bayar\nPemegang Kad atau untuk tujuan lain yang dianggap wajar oleh Bank, Bank boleh pada bila-bila\nmasa dan tempat, menyimpan untuk suatu tempoh yang wajar, sebarang wang yang diterima,\ndiperoleh semula atau direalisasi selaras dengan T&S ini atau di bawah mana-mana cagaran atau\njaminan untuk dikreditkan dalam akaun berasingan yang tidak menawarkan keuntungan di bawah\nnama Pemegang Kad sebagaimana dianggap wajar oleh Bank. Bank tidak mempunyai kewajipan\nsegera untuk memanfaatkan wang tersebut atau sebahagian daripadanya untuk atau ke arah\nmelunaskan jumlah perlu bayar dan terhutang kepada Bank.\n18.0\nTolak Selesai\n18.1\nPemegang Kad bersetuju bahawa Bank boleh, pada bila-bila masa dan dengan memberikan\nmakluman awal, menyatu dan menggabung semua atau mana-mana Akaun Pemegang Kad\ndengan Bank dalam apa jua bentuk dan di mana-mana lokasi, sama ada di dalam Ringgit Malaysia\natau mata wang lain, atau menolak selesai atau memindah sebarang jumlah yang sedia untuk\ndikredit ke dalam mana-mana Akaun tersebut termasuk akaun bersama di mana akaun bersama\ntersebut adalah di atas nama Pemegang Kad untuk atau ke arah melunaskan semua jumlah\nterhutang kepada Bank di bawah mana-mana akaun Pemegang Kad dengan Bank, dalam apa jua\nbentuk dan di mana-mana lokasi, sama ada di dalam Ringgit Malaysia atau mata wang lain dan\nBank boleh berbuat demikian tanpa mengambil kira baki di dalam akaun tersebut dan jumlah\nterhutang mungkin bukan di dalam mata wang yang sama, dan Pemegang Kad dengan ini\nmembenarkan Bank menguatkuasakan sebarang penyatuan, gabungan, tolak selesai atau\npemindahan dengan pertukaran mata wang yang sewajarnya pada kadar tukaran semasa Bank\nyang akan ditentukan oleh Bank mengikut budi bicaranya.\n19.0\nCaj-Balik\n19.1\nBank tidak boleh dipertanggungjawabkan atas sebarang tindakan atau pengabaian oleh mana-\nmana peniaga runcit atau dalam talian walau apapun puncanya, termasuk tetapi tidak terhad\nkepada keengganan menerima Kad Debit atau sebarang Penyata atau komunikasi lain yang dibuat\nberkaitan dengannya, atau sebarang kecacatan atau kekurangan pada produk atau perkhidmatan.\nSekiranya terdapat sebarang kecacatan atau kekurangan pada produk atau perkhidmatan,\nPemegang Kad hendaklah berhubung secara langsung dengan Peniaga Sah mengenai sebarang\ntuntutan atau pertikaian dan tidak akan mempertikaikan sebarang bayaran yang dibuat oleh Bank\ndari Akaun Pemegang Kad ke atas tuntutan atau pertikaian tersebut.\n19.2\nDalam keadaan tertentu, peraturan skim kad membenarkan Pemegang Kad untuk mencaj-balik\nTransaksi Kad kepada Peniaga Sah bagi transaksi yang telah dibuat (jika hak caj-balik disediakan)\ntertakluk kepada aduan atau pertikaian telah dibuat oleh Pemegang Kad kepada Bank secara lisan\natau bertulis dalam masa dua puluh satu (21) hari kalendar daripada Tarikh Pengeposan\nTransaksi Kad tersebut seperti dalam Penyata dan Pemegang Kad telah menyediakan maklumat\nyang diperlukan oleh Bank (iaitu: nama Pemegang Kad, Akaun yang terlibat, tarikh dan jumlah Kad\ntransaksi yang dipertikaian dan sebab Pemegang Kad percaya bahawa ia adalah Transaksi Kad\nyang dipertikaikan) untuk menyokong caj balik dalam tempoh masa yang ditetapkan."
  },
  {
    "page": 20,
    "content": "19.3\nSekiranya Bank melanjutkan tempoh masa untuk menyelesaikan siasatan selepas empat belas\n(14) hari kalendar dari tarikh laporan terhadap Transaksi Kad yang dipertikaikan, sama ada secara\nlisan atau bertulis, oleh Pemegang Kad kepada Bank, Bank boleh: sekurang-kurangnya,\nsementara mengkreditkan jumlah penuh transaksi yang dipertikaikan atau Ringgit Malaysia Lima\nRibu (RM5,000), yang mana lebih rendah (termasuk apa-apa keuntungan yang berkenaan), ke\ndalam Akaun Pembelian Runcit tidak lewat daripada empat belas (14) hari kalendar dari tarikh\nPemegang Kad mengemukakan maklumat yang dinyatakandalam Klausa 19.2 kepada Bank;\n(a) mengkredit baki Transaksi Kad yang dipertikaikan (termasuk apa-apa keuntungan yang\nberkenaan) tidak lewat daripada tiga puluh (30) hari kalendar daripada tarikh perkreditan\npertama dana sementara ke dalam Akaun Pembelian Runcit oleh Bank mengikut Klausa 19.3\n(a) di mana dana tersebut adalah kurang daripada amaun Transaksi Kad yang dipertikaikan;\ndan\n(b) membenarkan Pemegang Kad menggunakan sepenuhnya dana sementara yang dikreditkan.\n19.4\nSekiranya Pemegang Kad dikehendaki membayar jumlah transaksi atau Peniaga Sah telah\nmengembalikan jumlah transkasi tersebut secara langsung kepada Pemegang Kad, Bank berhak\nmendebitkan amaun berkenaan daripada Akaun Pembelian Runcit.\n19.5\nTanpa mengambilkira dan tanpa prejudis kepada sebutan umum Klausa di dalam T&S ini,\nPemegang Kad bersetuju bahawa:\n(a) penggunaan Kad Debit adalah atas risiko Pemegang Kad sendiri dan Pemegang Kad harus\nmenanggung semua risiko yang berkaitan atau yang timbul daripada penggunaan Kad Debit;\ndan\n(b) Pemegang Kad tidak boleh membuat atau cuba untuk membuat apa-apa dakwaan palsu,\npenipuan atau menyalahi undang-undang berkenaan dengan Transaksi Kad yang\ndipertikaikan sama ada secara langsung atau tidak langsung. Bank berhak pada bila-bila masa\nuntuk memulakan dan memulakan dan mengambil tindakan undang-undang dan/atau apa-apa\nprosiding lain sebagaimana yang Bank fikirkan perlu terhadap Pemegang Kad setelah Bank\nmenemui atau menyedari tentang apa-apa tuntutan yang palsu atau penipuan atau menyalahi\nundang-undang berkaitan dengan atau yang timbul daripada Transaksi Kad yang dipertikaikan.\n19.6\nBank mempunyai hak untuk mengambil tindakan perundangan dan/atau apa-apa prosiding lain\nyang dianggap perlu oleh Bank termasuk melaporkan apa-apa laporan yang sesuai atau perlu\ndengan pihak berkuasa yang berkaitan, terhadap Pemegang Kad yang melengahkan, menghalang\ndan/atau menyembunyikan maklumat penting daripada Bank, membuat atau cuba untuk membuat\ntuntutan palsu berkenaan dengan apa-apa transaksi, menyiarkan tuntutan palsu di media\ntradisional atau sosial, dan/atau membuat laporan polis palsu mengenai apa-apa transaksi.\n19.7\nTertakluk kepada kepatuhan Pemegang Kad kepada tanggungjawab yang dinyatakan dalam\nKlausa 19.2, sekiranya berlaku caj balik akibat aduan atau pertikaian oleh Pemegang Kad\nmengenai Transaksi Luar Negara (seperti yang ditakrifkan di bawah Klausa 19.1 di bawah) yang\ndilakukan dalam mata wang asing melalui Akaun Yang Diupayakan Dengan MCF, amaun yang\ndicaj balik akan dikreditkan ke dalam Akaun Yang Diupayakan Dengan MCF Pemegang Kad dalam\nmata wang transaksi asal.\n20.0\nTransaksi Luar Negara\n20.1\nPemegang Kad boleh menggunakan Kad Debit untuk melakukan Transaksi Kad dan Transaksi"
  },
  {
    "page": 21,
    "content": "Kad ATM di luar Malaysia (\u201cTransaksi Luar Negara\u201d) di mana terdapat Peniaga Sah dan/atau\nSaluaran Tunai yang dibenarkan dengan syarat bahawa Pemegang Kad telah memilih untuk\nmembenarkan Transaksi Luar Negara yang akan dilaksanakan pada Kad Debit yang berkaitan\nKlausa 20.1 di dalam ini.\n20.2\nSemua Transaksi Luar Negara melalui transaksi ATM dan Transaksi Kad adalah tertakluk kepada\nundang-undang sedia ada di negara di mana transaksi dilakukan. Untuk semua Transaksi Kad,\nkadar tukaran, di mana berkaitan, adalah kadar tukaran serta-merta semasa pada tarikh transaksi.\n20.3\nPemegang Kad boleh menggunakan Kad Debit untuk pengeluaran tunai melalui ATM tertentu yang\ndipasang di negara yang diluluskan sebagaimana ditetapkan oleh Bank dan/atau mana-mana\nVisa/Mastercard tertakluk bahawa semua caj yang dikenakan oleh Visa International/MasterCard\nInternational dan/atau Bank termasuk semua caj yang dinyatakan di dalam Klausa 20.5 dan 20.6.\n20.4\nSekiranya Pemegang Kad menggunakan Kad Debit di ATM dan outlet Peniaga di bawah rangkaian\nVisa/Mastercard di luar Malaysia, transaksi akan dicaj menggunakan matawang rasmi negara\ntersebut dan ditukar ke Ringgit Malaysia pada kadar tukaran yang mungkin ditentukan oleh Visa\n/Mastercard.\n20.5\nTransaksi Kad yang dibuat dalam mata wang selain dari Ringgit Malaysia akan ditukarkan kepada\nRinggit Malaysia pada tarikh ia diproses oleh Visa atau Mastercard pada kadar pertukaran asing\nsebagaimana yang ditentukan oleh Visa atau Mastercard. Fi tambahan berikut akan dikenakan ke\natas penukaran:\n(i) 1% fi transaksi dikenakan oleh Visa atau Mastercard; dan\n(ii) 1% penambahan kepada penukaran tukaran asing dikenakan oleh Bank\n20.6\nUntuk Transaksi Luar Negara tertentu, Pemegang Kad boleh memilih untuk menggunakan\nperkhidmatan DCC untuk menukarkan transaksi mata wang asing kepada Ringgit Malaysia. Jika\nDCC dipilih:\n(i) Kadar pertukaran asing yang digunakan oleh peniaga luar negara bagi urusniaga DCCmungkin\nlebih tinggi daripada kadar pertukaran yang ditentukan oleh Visa atau Mastercard.\n(ii) Pemegang Kad akan dikenakan fi transaksi sebanyak 1% oleh Visa atau Mastercard ke atas\namaun Ringgit Malaysia yang ditukarkan, yang merupakan fi yang dikenakan oleh Visa atau\nMastercard.\n20.7\nSekiranya Pemegang Kad memilih untuk melakukan transaksi dalam mata wang asing\nmenggunakan Akaun Yang Diupayakan Dengan MCF, transaksi tersebut akan berjaya sekiranya\nmata wang asing yang dipilih mempunyai baki yang mencukupi di dalam Akaun Yang Diupayakan\nDengan MCF. Transaksi tersebut tidak akan dikenakan fi yang dinyatakan dalam Klausa 20.5.\nWalau bagaimanapun, sekiranya mata wang asing yang dipilih oleh Pemegang Kad tidak\nmencukupi, transaksi tersebut akan disempurnakan dalam Ringgit Malaysia dan akan didebit dari\nbaki Ringgit Malaysia yang terdapat dalam Akaun Yang Diupayakan Dengan MCF Pemegang Kad\ndan fi yang dinyatakan dalam Klausa 20.5 akan dikenakan.\n20.8\nMeskipun\nadanya\nKlausa\n20.7,\nsekiranya\ninstitusi\nkewangan\npeniaga\nluar\nnegara\nmenyempurnakan / membenarkan Transaksi Luar Negara dalam Ringgit Malaysia, bukannya\ndalam mata wang asing yang dipilih oleh Pemegang Kad, transaksi tersebut akan didebit dari"
  },
  {
    "page": 22,
    "content": "Akaun Pembelian Runcit Pemegang Kad dalam Ringgit Malaysia walaupun Pemegang\nKadmempunyai baki mata wang asing yang mencukupi dalam Akaun Yang Diupayakan\nDengan MCF\n20.9\nBagi Transaksi Kad yang disempurnakan dari Akaun Yang Diupayakan Dengan MCF dalam mata\nwang asing:\n(i) sekiranya terdapat kembalian oleh Peniaga Sah dalam tempoh enam puluh (60) hari daripada\ntarikh transaksi, dan Bank berjaya memadankan kembalian tersebut kepada transaksi asal,\nkembalian tersebut akan dikreditkan ke dalam Akaun Yang Diupayakan Dengan MCF dalam\nmata wang transaksi asal. Sekiranya padanan tidak berjaya, kembalian tersebut akan\nditukarkan kepada Ringgit Malaysia pada kadar tukaran semasa Bank pada masa Bank\nmengkredit kembalian tersebut ke dalam Akaun Yang Diupayakan Dengan MCF Pemegang\nKad.\n(ii)\nsekiranya Peniaga Sah mengemukakan permintaan untuk kembalian selepas tempoh enam\npuluh (60) hari daripada tarikh transaksi, kembalian kepada Pemegang Kad akan\ndisempurnakan dalam Ringgit Malaysia and tukaran daripada mata wang yang berkenaan\nkepada Ringgit Malaysia adalah pada kadar tukaran semasa Bank pada masa Bank\nmengkredit kembalian tersebut ke dalam Akaun Yang Diupayakan Dengan MCF Pemegang\nKad.\n20.10\nPemegang Kad perlu mematuhi Notis FE yang dikeluarkan oleh BNM berkenaan dengan sebarang\ntransaksi, termasuk Transaksi Luar Negara, yang boleh didapati di laman sesawang BNM.\nPemegang Kad adalah bertanggungjawab atas sebarang pengingkaran atau ketidakpatuhan\nperaturan tersebut dan pindaan yang dibuat terhadapnya.\n21.0\nKeperluan Pilih Masuk bagi Transaksi Luar Negara dan Transaksi Tanpa Kad\n21.1\nPemegang Kad tidak dibenarkan untuk melaksanakan apa-apa Transaksi Luar Negara dan/atau\nTransaksi Tanpa Kad melainkan Pemegang Kad telah mendaftarkan arahannya untuk memilih\nmasuk untuk melaksanakan Transaksi Luar Negara dan/atau Transaksi Tanpa Kad melalui Hong\nLeong Connect, ATM Bank, kunjungi mana-mana cawangan Bank atau hubungi Pusat Panggilan\nBank di 03-7626 8899, jika tidak, Transaksi Luar Negara dan/atau Transaksi Tanpa Kad akan\nditolak.\n22.0\nPlatform Pembayaran atau Rangkaian\n22.1\nWalaupun Kad Debit Bank menyokong MyDebit yang hanya boleh digunakan di dalam Malaysia,\ndan Visa/Mastercard yang diterima di dalam Malaysia dan di luar negara, apabila Kad Debit\ndigunakan di Peniaga Sah di Malaysia, ia adalah tertakluk kepada pilihan rangkaian oleh Peniaga\nSah tersebut (iaitu sama ada MyDebit atau Visa/Mastercard) bagi tujuan memproses pembayaran.\n22.2\nPemegang Kad tidak dibenarkan untuk meminta Peniaga Sah yang dibenarkan untuk menukar\nrangkaian Kad Debit yang telah dipilih.\n23.0\nPengganti\n23.1\nT&S ini akan digunakan pakai untuk waris, wakil peribadi dan pengganti\nPemegang Kad dan Bank masing-masing."
  },
  {
    "page": 23,
    "content": "24.0\nUndang-undang dan Bidang Kuasa yang Terpakai\n24.1\nT&S ini akan tertakluk kepada dan ditafsirkan menurut undang-undang Malaysia tetapi dalam\nmenguatkuasakan haknya, Bank bebas untuk memulakan dan mengambil tindakan atau prosiding\natau selainnya terhadap Pemegang Kad dan mana-mana pihak di Malaysia dan/atau di tempat lain\nseperti yang dianggap sesuai oleh Bank dan Pemegang Kad bersetuju bahawa apabila apa-apa\ntindakan atau prosiding dimulakan dan diambil di Malaysia, ia hendaklah mematuhi bidang kuasa\neksklusif Mahkamah Malaysia dalam semua hal yang berkaitan dengan kewajipan dan liabiliti di\nbawah atau yang timbul daripada Akaun dan T&S ini.\nJika anda mempunyai sebarang pertanyaan mengenai T&S ini, anda boleh mendapatkan penjelasan\ndaripada kakitangan kami yang membantu anda. Sebagai alternatif, sila e-mel kepada kami di\nhlonine@hlbb.hongleong.com.my atau hubungi 03-7626 8899\n[TAMAT]"
  }
]

search_text = """1.36 “Akaun Diaktifkan MCF” bermaksud Akaun Pemegang Kad di mana MCF (seperti yang ditakrifkan dalam Klausa 1.45 di sini) telah diupayakan, yang mungkin termasuk akaun yang dipautkanke Kad Debit. telah diaktifkan, yang mungkin termasuk Akaun Pembelian Runcit.
      1.37 “MEPS” bermaksud Malaysian Electronic Payment System Sdn Bhd, pembekal perkhidmatan antara bank di Malaysia.
      1.38 “Perjanjian Peniaga” bermaksud perjanjian yang ditandatangani di antara Peniaga Sah dan Bank.
      1.39 “Fi Penyata Bulanan” bermaksud fi bulanan yang dikenakan ke atas Pemegang Kad untuk penyata bulanan Kad Debit yang dicetak.
      1.40 "MOTO" ertinya Pesanan Mel Pesanan Telefon yang merupakan jenis Transaksi Kad-Tidak-Hadir di mana Perkhidmatan diselesaikan dan dibayar melalui telefon, mel, faks atau e-mel.
      1.41 “MyKad” bermaksud kad yang dikeluarkan kepada Pemegang Kad oleh Jabatan Pendaftaran Negara Malaysia kepada Pemegang Kad untuk mengenalpasti and megesahkan identiti Pemegang Kad.
      1.42 "MyDebit" bermaksud suatu skim kad debit domestik yang membenarkan Pemegang Kad untuk membuat bayaran bagi pembelian barangan dan/atau perkhidmatan dengan mendebit secara langsung dari Akaun Pembelian Runcit Pemegang Kad."""

for data in content:
  if search_text in data["content"].replace("\n", " "):
    print(data["page"])